In [1]:
#2025 06 01 売上消失商品 売上激減商品 空白部修正必要
import pandas as pd
import tkinter as tk
from tkinter import filedialog, ttk, messagebox
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
import openpyxl
from openpyxl.styles import Font, Alignment, PatternFill
from openpyxl.utils import get_column_letter
import re
import numpy as np

class SalesAnalysisApp:
    def __init__(self, root):
        self.root = root
        self.root.title("売上照会分析アプリ")
        self.root.geometry("1200x850")

        self.past_file_path = tk.StringVar()
        self.current_file_path = tk.StringVar()
        self.output_file_path = tk.StringVar()

        self.default_product_code_options = ["旧ｺｰﾄﾞ連結", "中分類１", "取引先", "取引先名"]
        self.product_code_column = tk.StringVar(value=self.default_product_code_options[0])
        self.amount_column = tk.StringVar(value="金額")

        self.past_summary = None
        self.current_summary = None
        self.merged_data = None
        self.lost_sales_data = None
        self.significant_decrease_data = None

        self.DECREASE_THRESHOLD_PERCENT = -80.0

        self.create_widgets()

    def create_widgets(self):
        main_frame = ttk.Frame(self.root)
        main_frame.pack(fill="both", expand=True, padx=10, pady=10)

        input_frame = ttk.LabelFrame(main_frame, text="ファイル選択")
        input_frame.pack(fill="x", pady=5)
        ttk.Label(input_frame, text="過去の売上照会ファイル:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        ttk.Entry(input_frame, textvariable=self.past_file_path, width=50).grid(row=0, column=1, padx=5, pady=5)
        ttk.Button(input_frame, text="参照", command=lambda: self.browse_file(self.past_file_path)).grid(row=0, column=2, padx=5)
        ttk.Label(input_frame, text="現在の売上照会ファイル:").grid(row=1, column=0, padx=5, pady=5, sticky="w")
        ttk.Entry(input_frame, textvariable=self.current_file_path, width=50).grid(row=1, column=1, padx=5, pady=5)
        ttk.Button(input_frame, text="参照", command=lambda: self.browse_file(self.current_file_path)).grid(row=1, column=2, padx=5)
        ttk.Label(input_frame, text="分析結果の出力先:").grid(row=2, column=0, padx=5, pady=5, sticky="w")
        ttk.Entry(input_frame, textvariable=self.output_file_path, width=50).grid(row=2, column=1, padx=5, pady=5)
        ttk.Button(input_frame, text="参照", command=self.browse_output_file).grid(row=2, column=2, padx=5)

        column_frame = ttk.LabelFrame(main_frame, text="列名設定")
        column_frame.pack(fill="x", pady=5)
        ttk.Label(column_frame, text="集計基準列名:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        self.product_code_combo_main = ttk.Combobox(column_frame, textvariable=self.product_code_column, width=18, values=self.default_product_code_options, state="readonly")
        self.product_code_combo_main.grid(row=0, column=1, padx=5, pady=5)
        self.product_code_column.set(self.default_product_code_options[0])
        ttk.Label(column_frame, text="金額列名:").grid(row=0, column=2, padx=5, pady=5, sticky="w")
        self.amount_entry_main = ttk.Entry(column_frame, textvariable=self.amount_column, width=20)
        self.amount_entry_main.grid(row=0, column=3, padx=5, pady=5)
        ttk.Label(column_frame, text="※「分析実行」でシート/列名詳細設定").grid(row=1, column=0, columnspan=4, padx=5, pady=5)

        buttons_frame = ttk.Frame(main_frame)
        buttons_frame.pack(fill="x", pady=5)
        ttk.Button(buttons_frame, text="分析実行", command=self.analyze_sales, width=20).pack(side="left", padx=5)
        ttk.Button(buttons_frame, text="Excelに出力", command=self.export_to_excel, width=20).pack(side="left", padx=5)
        ttk.Button(buttons_frame, text="クリア", command=self.clear_results, width=20).pack(side="left", padx=5)

        tab_control = ttk.Notebook(main_frame)
        table_tab = ttk.Frame(tab_control)
        graph_tab = ttk.Frame(tab_control)
        self.sales_trend_tab = ttk.Frame(tab_control)
        tab_control.add(table_tab, text='全体比較テーブル')
        tab_control.add(graph_tab, text='全体比較グラフ')
        tab_control.add(self.sales_trend_tab, text='売上傾向分析')
        tab_control.pack(expand=1, fill="both")

        cols_main = ("集計キー", "過去売上金額", "過去構成比", "現在売上金額", "現在構成比", "増減額", "増減率")
        self.tree = ttk.Treeview(table_tab, columns=cols_main, show="headings")
        for c in cols_main:
            self.tree.heading(c, text=c)
            self.tree.column(c, width=180 if c=="集計キー" else 130, anchor="center")
        ysb = ttk.Scrollbar(table_tab, orient="vertical", command=self.tree.yview)
        xsb = ttk.Scrollbar(table_tab, orient="horizontal", command=self.tree.xview)
        self.tree.configure(yscrollcommand=ysb.set, xscrollcommand=xsb.set)
        self.tree.pack(side="left", fill="both", expand=True)
        ysb.pack(side="right", fill="y"); xsb.pack(side="bottom", fill="x")

        self.graph_frame = graph_tab

        trend_frame = ttk.Frame(self.sales_trend_tab)
        trend_frame.pack(fill="both", expand=True, padx=5, pady=5)
        lost_sales_frame = ttk.LabelFrame(trend_frame, text="売上消失商品 (過去売上あり / 現在売上なし)")
        lost_sales_frame.pack(fill="both", pady=5, expand=True) # fill="both"に変更
        cols_lost = ("集計キー", "過去売上金額")
        self.lost_sales_tree = ttk.Treeview(lost_sales_frame, columns=cols_lost, show="headings", height=7)
        for c in cols_lost:
            self.lost_sales_tree.heading(c, text=c)
            self.lost_sales_tree.column(c, width=250 if c=="集計キー" else 150, anchor="w" if c=="集計キー" else "e")
        ls_ysb = ttk.Scrollbar(lost_sales_frame, orient="vertical", command=self.lost_sales_tree.yview)
        self.lost_sales_tree.configure(yscrollcommand=ls_ysb.set)
        self.lost_sales_tree.pack(side="left", fill="both", expand=True); ls_ysb.pack(side="right", fill="y")

        decrease_sales_frame = ttk.LabelFrame(trend_frame, text=f"売上激減商品 (増減率 {self.DECREASE_THRESHOLD_PERCENT}%以下)")
        decrease_sales_frame.pack(fill="both", pady=5, expand=True) # fill="both" に変更
        cols_decrease = ("集計キー", "過去売上金額", "現在売上金額", "増減額", "増減率")
        self.decrease_sales_tree = ttk.Treeview(decrease_sales_frame, columns=cols_decrease, show="headings", height=7)
        for c in cols_decrease:
            self.decrease_sales_tree.heading(c, text=c)
            self.decrease_sales_tree.column(c, width=200 if c=="集計キー" else 120, anchor="w" if c=="集計キー" else "e")
        ds_ysb = ttk.Scrollbar(decrease_sales_frame, orient="vertical", command=self.decrease_sales_tree.yview)
        self.decrease_sales_tree.configure(yscrollcommand=ds_ysb.set)
        self.decrease_sales_tree.pack(side="left", fill="both", expand=True); ds_ysb.pack(side="right", fill="y")

    def browse_file(self, var):
        f = filedialog.askopenfilename(title="Excelファイルを選択", filetypes=[("Excel files","*.xlsx *.xls"),("All","*.*")])
        if f: var.set(f)
        if var == self.past_file_path and f: self.update_main_product_code_combo()

    def update_main_product_code_combo(self):
        try:
            self.product_code_combo_main['values'] = self.default_product_code_options
            current_selection = self.product_code_column.get()
            if current_selection not in self.default_product_code_options or not current_selection: self.product_code_column.set(self.default_product_code_options[0])
            elif current_selection in self.default_product_code_options: self.product_code_combo_main.set(current_selection)
        except Exception as e:
            print(f"メイン画面のCombobox更新エラー: {e}")
            self.product_code_combo_main['values'] = self.default_product_code_options
            if self.product_code_column.get() not in self.default_product_code_options: self.product_code_column.set(self.default_product_code_options[0])

    def browse_output_file(self):
        f = filedialog.asksaveasfilename(title="保存先を選択", defaultextension=".xlsx", filetypes=[("Excel files","*.xlsx"),("All","*.*")])
        if f: self.output_file_path.set(f)

    def get_excel_sheets(self, path):
        try: return pd.ExcelFile(path).sheet_names
        except Exception as e: messagebox.showerror("エラー", f"シート取得失敗: {e}"); return []

    def normalize_column_name(self, name):
        if not isinstance(name, str): return str(name)
        return name.replace('　', ' ').strip()

    def find_best_match_column(self, df, target_name):
        if not target_name: return None
        if target_name in df.columns: return target_name
        norm_target = self.normalize_column_name(target_name)
        for col in df.columns:
            if self.normalize_column_name(col) == norm_target: return col
        if target_name in self.default_product_code_options:
            if target_name in df.columns: return target_name
            norm_default_opt = self.normalize_column_name(target_name)
            for col in df.columns:
                if self.normalize_column_name(col) == norm_default_opt: return col
        best_match, best_score = None, 0
        for col in df.columns:
            norm_col = self.normalize_column_name(col)
            if norm_target in norm_col or norm_col in norm_target:
                match_len = len(set(norm_target) & set(norm_col))
                if match_len > best_score: best_score, best_match = match_len, col
        # if best_match: print(f"部分一致: '{target_name}' -> '{best_match}' (スコア: {best_score})") # デバッグ用
        return best_match

    def show_sheet_selection_dialog(self, past_file, current_file):
        dialog = tk.Toplevel(self.root); dialog.title("シート選択と列設定"); dialog.geometry("500x470"); dialog.transient(self.root); dialog.grab_set()
        past_sheets, current_sheets = self.get_excel_sheets(past_file), self.get_excel_sheets(current_file)
        ttk.Label(dialog, text="過去ファイルのシート:").pack(anchor="w", padx=10, pady=5)
        past_sheet_var = tk.StringVar(value=past_sheets[0] if past_sheets else ""); past_combo = ttk.Combobox(dialog, textvariable=past_sheet_var, values=past_sheets, state="readonly", width=30); past_combo.pack(anchor="w", padx=10)
        ttk.Label(dialog, text="現在ファイルのシート:").pack(anchor="w", padx=10, pady=5)
        current_sheet_var = tk.StringVar(value=current_sheets[0] if current_sheets else ""); current_combo = ttk.Combobox(dialog, textvariable=current_sheet_var, values=current_sheets, state="readonly", width=30); current_combo.pack(anchor="w", padx=10)
        preview_frame = ttk.LabelFrame(dialog, text="列名設定とプレビュー"); preview_frame.pack(fill="both", expand=True, padx=10, pady=10)
        product_code_var_dialog, amount_var_dialog = tk.StringVar(value=self.product_code_column.get()), tk.StringVar(value=self.amount_column.get())
        ttk.Label(preview_frame, text="集計基準列:").grid(row=0, column=0, padx=5, pady=5, sticky="w"); product_code_combo_dialog = ttk.Combobox(preview_frame, textvariable=product_code_var_dialog, width=30); product_code_combo_dialog.grid(row=0, column=1, padx=5, pady=5)
        ttk.Label(preview_frame, text="金額列:").grid(row=1, column=0, padx=5, pady=5, sticky="w"); amount_combo_dialog = ttk.Combobox(preview_frame, textvariable=amount_var_dialog, width=30); amount_combo_dialog.grid(row=1, column=1, padx=5, pady=5)
        preview_text = tk.Text(preview_frame, height=8, width=45); preview_text.grid(row=3, column=0, columnspan=2, padx=5, pady=5)
        def preview_columns_and_set_defaults():
            try:
                selected_past_sheet = past_sheet_var.get()
                if not selected_past_sheet: messagebox.showwarning("注意", "過去ファイルのシートを選択してください."); return
                df = pd.read_excel(past_file, sheet_name=selected_past_sheet, nrows=5); file_cols = df.columns.tolist()
                dialog_pc_options = sorted(list(set(self.default_product_code_options + file_cols)))
                product_code_combo_dialog['values'], amount_combo_dialog['values'] = dialog_pc_options, file_cols
                current_pc_selection_in_dialog = product_code_var_dialog.get()
                if current_pc_selection_in_dialog in dialog_pc_options: product_code_var_dialog.set(current_pc_selection_in_dialog)
                else:
                    main_pc_selection = self.product_code_column.get()
                    if main_pc_selection in file_cols and main_pc_selection in dialog_pc_options: product_code_var_dialog.set(main_pc_selection)
                    else:
                        found_default_in_file = False
                        for opt in self.default_product_code_options:
                            if opt in file_cols and opt in dialog_pc_options: product_code_var_dialog.set(opt); found_default_in_file = True; break
                        if not found_default_in_file and dialog_pc_options: product_code_var_dialog.set(dialog_pc_options[0])
                        elif not found_default_in_file and not dialog_pc_options : product_code_var_dialog.set("")
                current_amount_selection_in_dialog = amount_var_dialog.get()
                if current_amount_selection_in_dialog in file_cols: amount_var_dialog.set(current_amount_selection_in_dialog)
                else:
                    target_amount_col_for_find = self.amount_column.get(); best_amount_match = self.find_best_match_column(df, target_amount_col_for_find)
                    if best_amount_match and best_amount_match in file_cols: amount_var_dialog.set(best_amount_match)
                    elif file_cols: amount_var_dialog.set(file_cols[0])
                    else: amount_var_dialog.set("")
                preview_text.delete("1.0", tk.END); preview_text.insert(tk.END, "使用可能な列一覧 (過去ファイルより):\n" + "\n".join(f"{i+1}. {c}" for i, c in enumerate(file_cols)))
                if len(df) > 0:
                    preview_text.insert(tk.END, "\n\n最初の行のデータ例:\n")
                    for col_idx, col_name in enumerate(file_cols[:5]): preview_text.insert(tk.END, f"{col_name}: {df.iloc[0, col_idx]}\n")
            except Exception as e: messagebox.showerror("エラー", f"列プレビュー失敗: {e}"); import traceback; traceback.print_exc()
        past_combo.bind("<<ComboboxSelected>>", lambda event: preview_columns_and_set_defaults()); dialog.after(100, preview_columns_and_set_defaults)
        ttk.Button(preview_frame, text="列名プレビュー更新", command=preview_columns_and_set_defaults).grid(row=2, column=0, columnspan=2, pady=5)
        def on_ok():
            selected_pc_in_dialog, selected_amount_in_dialog = product_code_var_dialog.get(), amount_var_dialog.get()
            if not past_sheet_var.get() or not current_sheet_var.get(): messagebox.showerror("エラー", "過去と現在の両方のシートを選択してください."); return
            if not selected_pc_in_dialog or not selected_amount_in_dialog: messagebox.showerror("エラー", "集計基準列と金額列を選択または入力してください."); return
            self.past_sheet, self.current_sheet = past_sheet_var.get(), current_sheet_var.get()
            self.product_code_column.set(selected_pc_in_dialog); self.amount_column.set(selected_amount_in_dialog)
            self.product_code_combo_main.set(selected_pc_in_dialog); self.amount_entry_main.delete(0, tk.END); self.amount_entry_main.insert(0, selected_amount_in_dialog)
            dialog.destroy(); self.perform_analysis()
        ttk.Button(dialog, text="OK", command=on_ok).pack(pady=10); self.root.wait_window(dialog)

    def analyze_sales(self):
        if not self.past_file_path.get() or not self.current_file_path.get(): messagebox.showerror("エラー", "過去と現在の両方のファイルを選択してください"); return
        self.product_code_column.set(self.product_code_combo_main.get()); self.amount_column.set(self.amount_entry_main.get())
        self.show_sheet_selection_dialog(self.past_file_path.get(), self.current_file_path.get())

    def perform_analysis(self):
        try:
            pc_user_choice, amt_user_choice = self.product_code_column.get(), self.amount_column.get()
            if not pc_user_choice or not amt_user_choice: messagebox.showerror("エラー", "集計基準列または金額列が指定されていません."); return
            past_df, curr_df = pd.read_excel(self.past_file_path.get(), sheet_name=self.past_sheet), pd.read_excel(self.current_file_path.get(), sheet_name=self.current_sheet)
            pc_past_actual, amt_past_actual = self.find_best_match_column(past_df, pc_user_choice), self.find_best_match_column(past_df, amt_user_choice)
            pc_curr_actual, amt_curr_actual = self.find_best_match_column(curr_df, pc_user_choice), self.find_best_match_column(curr_df, amt_user_choice)
            if not pc_past_actual: raise ValueError(f"集計基準列 '{pc_user_choice}' が過去Fで見つかりません.")
            if not amt_past_actual: raise ValueError(f"金額列 '{amt_user_choice}' が過去Fで見つかりません.")
            if not pc_curr_actual: raise ValueError(f"集計基準列 '{pc_user_choice}' が現在Fで見つかりません.")
            if not amt_curr_actual: raise ValueError(f"金額列 '{amt_user_choice}' が現在Fで見つかりません.")
            # (列名修正のshowinfoは省略)
            for df, col_actual in [(past_df, amt_past_actual), (curr_df, amt_curr_actual)]: # 金額列のみ処理対象で十分
                if col_actual not in df.columns or df[col_actual].empty: df[col_actual] = 0
                elif df[col_actual].dtype == object or isinstance(df[col_actual].iloc[0], str) :
                    df[col_actual] = df[col_actual].astype(str).str.replace(',', '', regex=False).str.replace(' ', '', regex=False).str.replace(r'[¥円]', '', regex=True)
                    df[col_actual] = pd.to_numeric(df[col_actual], errors='coerce').fillna(0)
                elif not pd.api.types.is_numeric_dtype(df[col_actual]): df[col_actual] = pd.to_numeric(df[col_actual], errors='coerce').fillna(0)
                else: df[col_actual] = df[col_actual].fillna(0)
            self.past_summary = past_df.groupby(pc_past_actual, dropna=False)[amt_past_actual].sum().reset_index().rename(columns={pc_past_actual: pc_user_choice, amt_past_actual: amt_user_choice})
            past_total = self.past_summary[amt_user_choice].sum(); self.past_summary['構成比'] = (self.past_summary[amt_user_choice] / past_total * 100) if past_total != 0 else 0; self.past_summary['構成比'] = self.past_summary['構成比'].fillna(0)
            self.current_summary = curr_df.groupby(pc_curr_actual, dropna=False)[amt_curr_actual].sum().reset_index().rename(columns={pc_curr_actual: pc_user_choice, amt_curr_actual: amt_user_choice})
            current_total = self.current_summary[amt_user_choice].sum(); self.current_summary['構成比'] = (self.current_summary[amt_user_choice] / current_total * 100) if current_total != 0 else 0; self.current_summary['構成比'] = self.current_summary['構成比'].fillna(0)
            self.prepare_merged_data(); self.display_results(); self.plot_results(); self.analyze_and_display_sales_trends()
            messagebox.showinfo("完了", "分析が完了しました")
        except Exception as e: import traceback; traceback.print_exc(); messagebox.showerror("エラー", f"分析中にエラー: {e}")

    def prepare_merged_data(self):
        pc_key, amt_key = self.product_code_column.get(), self.amount_column.get() 
        past_t, current_t = self.past_summary.copy(), self.current_summary.copy()
        if pc_key not in past_t.columns or pc_key not in current_t.columns: raise ValueError(f"マージキー'{pc_key}'エラー")
        past_t[pc_key], current_t[pc_key] = past_t[pc_key].astype(str), current_t[pc_key].astype(str)
        m = pd.merge(past_t, current_t, on=pc_key, how='outer', suffixes=('_過去', '_現在'))
        m.rename(columns={f'{amt_key}_過去': '過去売上金額', f'{amt_key}_現在': '現在売上金額','構成比_過去': '過去構成比', '構成比_現在': '現在構成比'}, inplace=True)
        for col in ['過去売上金額', '現在売上金額', '過去構成比', '現在構成比']: m[col] = m[col].fillna(0) if col in m.columns else 0
        m['増減額'] = m['現在売上金額'] - m['過去売上金額']; m['増減率'] = m.apply(lambda r: (r['増減額']/r['過去売上金額']*100) if r['過去売上金額']!=0 else (np.inf if r['増減額'] > 0 else (-np.inf if r['増減額'] < 0 else 0)), axis=1)
        self.merged_data = m.sort_values('現在売上金額', ascending=False)
        if pc_key not in self.merged_data.columns: messagebox.showwarning("警告", f"最終データに集計キー '{pc_key}' が見つかりません。")

    def display_results(self):
        for i in self.tree.get_children(): self.tree.delete(i)
        key_col = self.product_code_column.get(); self.tree.heading("集計キー", text=key_col) 
        if self.merged_data is None or key_col not in self.merged_data.columns: messagebox.showerror("エラー", f"表示データまたは列 '{key_col}' が無効です。"); return
        for _, r in self.merged_data.iterrows():
            self.tree.insert('', 'end', values=(r[key_col],f"{int(r['過去売上金額']):,}", f"{r['過去構成比']:.2f}%",f"{int(r['現在売上金額']):,}", f"{r['現在構成比']:.2f}%",f"{int(r['増減額']):,}", f"{r['増減率']:.2f}%" if pd.notnull(r['増減率']) and np.isfinite(r['増減率']) else ("-" if r['増減率']==0 else ("増加" if r['増減率']==np.inf else "減少")) ))
            
    def analyze_and_display_sales_trends(self):
        for tree in [self.lost_sales_tree, self.decrease_sales_tree]:
            for item in tree.get_children(): tree.delete(item)
        if self.merged_data is None or self.merged_data.empty:
            self.lost_sales_tree.insert('', 'end', values=("データ無", "")); self.decrease_sales_tree.insert('', 'end', values=("データ無", "", "", "", "")); return
        key_col = self.product_code_column.get()
        self.lost_sales_data = self.merged_data[(self.merged_data['過去売上金額'] > 0) & (self.merged_data['現在売上金額'] == 0)].sort_values(by='過去売上金額', ascending=False)
        for _, r in self.lost_sales_data.iterrows(): self.lost_sales_tree.insert('', 'end', values=(r[key_col], f"{int(r['過去売上金額']):,}"))
        self.significant_decrease_data = self.merged_data[(self.merged_data['過去売上金額'] > 0) & (self.merged_data['現在売上金額'] > 0) & (self.merged_data['増減率'] <= self.DECREASE_THRESHOLD_PERCENT)].sort_values(by='増減額', ascending=True)
        for _, r in self.significant_decrease_data.iterrows(): self.decrease_sales_tree.insert('', 'end', values=(r[key_col], f"{int(r['過去売上金額']):,}", f"{int(r['現在売上金額']):,}", f"{int(r['増減額']):,}", f"{r['増減率']:.2f}%"))
        if self.lost_sales_data.empty: self.lost_sales_tree.insert('', 'end', values=("該当なし", ""))
        if self.significant_decrease_data.empty: self.decrease_sales_tree.insert('', 'end', values=("該当なし", "", "", "", ""))

    def plot_results(self):
        for w in self.graph_frame.winfo_children(): w.destroy()
        if self.merged_data is None or self.merged_data.empty: ttk.Label(self.graph_frame, text="表示データ無").pack(padx=10, pady=10); return
        top_n, top = 10, self.merged_data.head(10); key_col = self.product_code_column.get()
        if key_col not in top.columns: messagebox.showerror("エラー", f"グラフ用データに列 '{key_col}' が無効です。"); return
        try: plt.rcParams['font.family'] = 'Yu Gothic'; plt.rcParams['font.family'] = 'MS Gothic' if not any(f.name == 'Yu Gothic' for f in plt.matplotlib.font_manager.fontManager.ttflist) else 'Yu Gothic'
        except: plt.rcParams['font.family'] = 'sans-serif'
        plt.rcParams['axes.unicode_minus'] = False; fig = plt.Figure(figsize=(14, 15), dpi=100); fig.subplots_adjust(hspace=0.6, wspace=0.35, bottom=0.15, top=0.95, left=0.08, right=0.95) 
        ax1 = fig.add_subplot(221); nz_past = top[top['過去構成比'] > 0.01]
        if not nz_past.empty: wedges, texts, autotexts = ax1.pie(nz_past['過去構成比'], labels=nz_past[key_col], autopct='%1.1f%%', startangle=90, pctdistance=0.85, wedgeprops={'edgecolor': 'white', 'linewidth': 0.5}, textprops={'fontsize': 7}); plt.setp(autotexts, size=6, weight="bold", color="white")
        ax1.set_title(f'過去売上構成比 (上位{len(nz_past)}項目)', fontsize=9); ax1.axis('equal')
        ax2 = fig.add_subplot(222); nz_curr = top[top['現在構成比'] > 0.01]
        if not nz_curr.empty: wedges, texts, autotexts = ax2.pie(nz_curr['現在構成比'], labels=nz_curr[key_col], autopct='%1.1f%%', startangle=90, pctdistance=0.85, wedgeprops={'edgecolor': 'white', 'linewidth': 0.5}, textprops={'fontsize': 7}); plt.setp(autotexts, size=6, weight="bold", color="white")
        ax2.set_title(f'現在売上構成比 (上位{len(nz_curr)}項目)', fontsize=9); ax2.axis('equal')
        ax3 = fig.add_subplot(223); x_idx, width = np.arange(len(top)), 0.35; r1 = ax3.bar(x_idx - width/2, top['過去売上金額'], width, label='過去売上', color='skyblue'); r2 = ax3.bar(x_idx + width/2, top['現在売上金額'], width, label='現在売上', color='lightcoral')
        ax3.set_ylabel('売上金額', fontsize=8); ax3.set_title(f'過去・現在 売上比較 (上位{top_n}項目)', fontsize=9); ax3.set_xticks(x_idx); ax3.set_xticklabels(top[key_col], rotation=45, ha='right', fontsize=7); ax3.legend(fontsize=7); ax3.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda val, loc: "{:,.0f}".format(val))); ax3.tick_params(axis='y', labelsize=7)
        def autolabel(rects, ax_target):
            for r_ in rects: ax_target.annotate('{:,.0f}'.format(r_.get_height()), xy=(r_.get_x() + r_.get_width() / 2, r_.get_height()), xytext=(0, 3), textcoords="offset points", ha='center', va='bottom', fontsize=6)
        autolabel(r1, ax3); autolabel(r2, ax3)
        ax4 = fig.add_subplot(224); sorted_by_change = top.sort_values('増減額', ascending=True); colors = ['red' if v < 0 else 'green' for v in sorted_by_change['増減額']]; bars = ax4.barh(sorted_by_change[key_col], sorted_by_change['増減額'], color=colors, edgecolor='grey')
        ax4.set_xlabel('増減額', fontsize=8); ax4.set_title(f'売上増減額 (現在売上上位{top_n}項目内)', fontsize=9); ax4.get_xaxis().set_major_formatter(plt.FuncFormatter(lambda val, loc: "{:,.0f}".format(val))); ax4.tick_params(axis='y', labelsize=7); ax4.tick_params(axis='x', labelsize=7)
        for bar in bars: bar_w = bar.get_width(); ha, off = ('left', 3) if bar_w > 0 else ('right', -3); ax4.text(bar.get_width() + off , bar.get_y() + bar.get_height()/2., '{:,.0f}'.format(bar_w), ha=ha, va='center', fontsize=6) if bar_w != 0 else None
        canvas = FigureCanvasTkAgg(fig, master=self.graph_frame); canvas.draw(); canvas.get_tk_widget().pack(fill="both", expand=True); toolbar = NavigationToolbar2Tk(canvas, self.graph_frame); toolbar.update()

    def export_to_excel(self):
        if self.merged_data is None and self.lost_sales_data is None and self.significant_decrease_data is None:
            messagebox.showerror("エラー", "先に分析を実行してください")
            return

        path = self.output_file_path.get()
        if not path:
            path = filedialog.asksaveasfilename(title="保存先を選択", defaultextension=".xlsx", filetypes=[("Excel files","*.xlsx"),("All","*.*")])
            if not path: return
        self.output_file_path.set(path)

        try:
            wb = openpyxl.Workbook()
            # 既存のメインシートを最初に作成 (アクティブシート)
            ws_main = wb.active
            ws_main.title = "全体比較" # シート名変更

            export_key_col = self.product_code_column.get() # どの分析でも共通のキー列名

            # --- 1. 全体比較シートへの出力 ---
            if self.merged_data is not None:
                ws_main.merge_cells('A1:G1'); ws_main['A1'] = f"{export_key_col}別 全体売上比較"; ws_main['A1'].font = Font(size=14, bold=True, color="000080"); ws_main['A1'].alignment = Alignment(horizontal='center')
                ws_main.merge_cells('A2:C2'); ws_main['A2'] = f"作成日: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}"; ws_main['A2'].font = Font(italic=True)
                headers_main = [export_key_col,"過去売上金額","過去構成比","現在売上金額","現在構成比","増減額","増減率"]
                for i, h in enumerate(headers_main, start=1):
                    c = ws_main.cell(row=4, column=i, value=h); c.font = Font(bold=True, color="FFFFFF"); c.alignment = Alignment(horizontal='center', vertical='center'); c.fill = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")
                if export_key_col not in self.merged_data.columns: messagebox.showerror("エラー", f"Excel出力エラー(全体): 列 '{export_key_col}' がデータにありません。"); return
                for idx, r_tuple in enumerate(self.merged_data.itertuples(index=False), start=5):
                    # ... (既存のデータ書き込みと書式設定) ...
                    key_value = getattr(r_tuple, export_key_col)
                    ws_main.cell(row=idx, column=1, value=key_value)
                    ws_main.cell(row=idx, column=2, value=r_tuple.過去売上金額); ws_main.cell(row=idx, column=3, value=r_tuple.過去構成比/100 if pd.notnull(r_tuple.過去構成比) else 0)
                    ws_main.cell(row=idx, column=4, value=r_tuple.現在売上金額); ws_main.cell(row=idx, column=5, value=r_tuple.現在構成比/100 if pd.notnull(r_tuple.現在構成比) else 0)
                    ws_main.cell(row=idx, column=6, value=r_tuple.増減額); ws_main.cell(row=idx, column=7, value=r_tuple.増減率/100 if pd.notnull(r_tuple.増減率) and np.isfinite(r_tuple.増減率) else 0)
                    ws_main.cell(row=idx, column=1).alignment = Alignment(horizontal='left')
                    for col_num in [2, 4, 6]: ws_main.cell(row=idx, column=col_num).number_format = '#,##0'; ws_main.cell(row=idx, column=col_num).alignment = Alignment(horizontal='right')
                    for col_num in [3, 5, 7]: ws_main.cell(row=idx, column=col_num).number_format = '0.00%'; ws_main.cell(row=idx, column=col_num).alignment = Alignment(horizontal='right')
                    if r_tuple.増減額 < 0: ws_main.cell(row=idx, column=6).font = Font(color="FF0000"); ws_main.cell(row=idx, column=7).font = Font(color="FF0000")
                    elif r_tuple.増減額 > 0 : ws_main.cell(row=idx, column=6).font = Font(color="008000"); ws_main.cell(row=idx, column=7).font = Font(color="008000")

                # 列幅調整 (ws_main)
                for i, column_cells in enumerate(ws_main.columns):
                    max_length = 0; column_letter_val = get_column_letter(i + 1) 
                    if ws_main.cell(row=4, column=i+1).value: max_length = max(max_length, len(str(ws_main.cell(row=4, column=i+1).value)))
                    for cell in column_cells[4:]: 
                        try:
                            if cell.value is not None: 
                                cell_str_len = len(str(cell.value))
                                if isinstance(cell.value, (int, float)):
                                    if cell.number_format and '%' in cell.number_format: cell_str_len = len(f"{cell.value:.2%}")
                                    elif cell.number_format and ',' in cell.number_format: cell_str_len = len(f"{cell.value:,}")
                                max_length = max(max_length, cell_str_len)
                        except: pass 
                    adjusted_width = (max_length + 2); ws_main.column_dimensions[column_letter_val].width = adjusted_width if adjusted_width < 50 else 50
                # 合計行 (ws_main)
                total_row = 5 + len(self.merged_data)
                ws_main.cell(row=total_row, column=1, value="合計").font = Font(bold=True); ws_main.cell(row=total_row, column=1).alignment = Alignment(horizontal='center'); ws_main.cell(row=total_row, column=1).fill = PatternFill(start_color="DDDDDD", end_color="DDDDDD", fill_type="solid")
                for col_idx, key in [(2, '過去売上金額'), (4, '現在売上金額'), (6, '増減額')]:
                    val = self.merged_data[key].sum(); cell = ws_main.cell(row=total_row, column=col_idx, value=val); cell.font = Font(bold=True); cell.number_format = '#,##0'; cell.alignment = Alignment(horizontal='right'); cell.fill = PatternFill(start_color="DDDDDD", end_color="DDDDDD", fill_type="solid")
            else:
                 ws_main['A1'] = "全体比較データがありません。"


            # --- 2. 売上消失商品シートへの出力 ---
            if self.lost_sales_data is not None and not self.lost_sales_data.empty:
                ws_lost = wb.create_sheet(title="売上消失商品")
                ws_lost.merge_cells('A1:B1'); ws_lost['A1'] = "売上消失商品リスト"; ws_lost['A1'].font = Font(size=14, bold=True, color="000080"); ws_lost['A1'].alignment = Alignment(horizontal='center')
                headers_lost = [export_key_col, "過去売上金額"]
                for i, h in enumerate(headers_lost, start=1):
                    c = ws_lost.cell(row=3, column=i, value=h); c.font = Font(bold=True, color="FFFFFF"); c.alignment = Alignment(horizontal='center', vertical='center'); c.fill = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")
                
                for idx, row in self.lost_sales_data.iterrows(): # iterrows を使用
                    ws_lost.cell(row=idx+4, column=1, value=row[export_key_col]) # row インデックスを使用
                    ws_lost.cell(row=idx+4, column=2, value=row['過去売上金額']).number_format = '#,##0'
                    ws_lost.cell(row=idx+4, column=1).alignment = Alignment(horizontal='left')
                    ws_lost.cell(row=idx+4, column=2).alignment = Alignment(horizontal='right')
                
                # 列幅調整 (ws_lost)
                for i, col_data in enumerate(self.lost_sales_data.reindex(columns=headers_lost).values.T.tolist() + [headers_lost], start=0): # ヘッダーも含めて調整
                    max_l = max(len(str(x)) for x in col_data) if col_data else 0
                    ws_lost.column_dimensions[get_column_letter(i+1)].width = max_l + 2 if max_l < 50 else 50

            # --- 3. 売上激減商品シートへの出力 ---
            if self.significant_decrease_data is not None and not self.significant_decrease_data.empty:
                ws_decrease = wb.create_sheet(title="売上激減商品")
                ws_decrease.merge_cells('A1:E1'); ws_decrease['A1'] = f"売上激減商品リスト (増減率 {self.DECREASE_THRESHOLD_PERCENT}%以下)"; ws_decrease['A1'].font = Font(size=14, bold=True, color="000080"); ws_decrease['A1'].alignment = Alignment(horizontal='center')
                headers_decrease = [export_key_col, "過去売上金額", "現在売上金額", "増減額", "増減率"]
                for i, h in enumerate(headers_decrease, start=1):
                    c = ws_decrease.cell(row=3, column=i, value=h); c.font = Font(bold=True, color="FFFFFF"); c.alignment = Alignment(horizontal='center', vertical='center'); c.fill = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")

                for idx, row in self.significant_decrease_data.iterrows(): # iterrows を使用
                    ws_decrease.cell(row=idx+4, column=1, value=row[export_key_col])
                    ws_decrease.cell(row=idx+4, column=2, value=row['過去売上金額']).number_format = '#,##0'
                    ws_decrease.cell(row=idx+4, column=3, value=row['現在売上金額']).number_format = '#,##0'
                    ws_decrease.cell(row=idx+4, column=4, value=row['増減額']).number_format = '#,##0'
                    ws_decrease.cell(row=idx+4, column=5, value=row['増減率']/100 if pd.notnull(row['増減率']) else 0).number_format = '0.00%'
                    
                    ws_decrease.cell(row=idx+4, column=1).alignment = Alignment(horizontal='left')
                    for col_num in [2,3,4,5]: ws_decrease.cell(row=idx+4, column=col_num).alignment = Alignment(horizontal='right')
                    if row['増減額'] < 0: ws_decrease.cell(row=idx+4, column=4).font = Font(color="FF0000"); ws_decrease.cell(row=idx+4, column=5).font = Font(color="FF0000")
                
                # 列幅調整 (ws_decrease)
                for i, col_data in enumerate(self.significant_decrease_data.reindex(columns=headers_decrease).values.T.tolist() + [headers_decrease], start=0):
                    max_l = max(len(str(x)) for x in col_data) if col_data else 0
                    ws_decrease.column_dimensions[get_column_letter(i+1)].width = max_l + 2 if max_l < 50 else 50


            # グラフシートは情報のみ
            chart_sheet = wb.create_sheet(title="グラフについて")
            chart_sheet['A1'] = "グラフの確認は、分析アプリ本体の「全体比較グラフ」タブで行ってください。"
            chart_sheet['A1'].font = Font(size=12, bold=True); chart_sheet.merge_cells('A1:E1')

            wb.save(path)
            messagebox.showinfo("完了", f"Excelファイルを保存しました: {path}")
        except PermissionError: messagebox.showerror("エラー", f"ファイル '{path}' への書き込み権限がありません。\nファイルが開かれていないか確認してください。")
        except Exception as e: import traceback; traceback.print_exc(); messagebox.showerror("エラー", f"Excelファイル出力中にエラーが発生しました: {e}")

    def clear_results(self):
        for tree in [self.tree, self.lost_sales_tree, self.decrease_sales_tree]:
            for item in tree.get_children(): tree.delete(item)
        for w in self.graph_frame.winfo_children(): w.destroy()
        ttk.Label(self.graph_frame, text="分析を実行するとここにグラフが表示されます。").pack(padx=10,pady=10)
        if hasattr(self, 'lost_sales_tree'): self.lost_sales_tree.insert('', 'end', values=("分析を実行してください", ""))
        if hasattr(self, 'decrease_sales_tree'): self.decrease_sales_tree.insert('', 'end', values=("分析を実行してください", "", "", "", ""))
        self.past_summary, self.current_summary, self.merged_data, self.lost_sales_data, self.significant_decrease_data = None, None, None, None, None
        self.product_code_column.set(self.default_product_code_options[0]); self.amount_column.set("金額")
        messagebox.showinfo("クリア", "分析結果をクリアしました")

def main():
    root = tk.Tk()
    app = SalesAnalysisApp(root)
    ttk.Label(app.graph_frame, text="分析を実行するとここにグラフが表示されます。").pack(padx=10,pady=10)
    if hasattr(app, 'lost_sales_tree'): app.lost_sales_tree.insert('', 'end', values=("分析を実行してください", ""))
    if hasattr(app, 'decrease_sales_tree'): app.decrease_sales_tree.insert('', 'end', values=("分析を実行してください", "", "", "", ""))
    root.mainloop()

if __name__ == "__main__":
    main()

In [ ]:
#2025 05 31 売上分析 エクセル出力できず
import pandas as pd
import tkinter as tk
from tkinter import filedialog, ttk, messagebox
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
import openpyxl
from openpyxl.styles import Font, Alignment, PatternFill
from openpyxl.utils import get_column_letter
import re
import numpy as np

class SalesAnalysisApp:
    def __init__(self, root):
        self.root = root
        self.root.title("売上照会分析アプリ")
        self.root.geometry("1200x850") # ウィンドウの高さを少し広げました

        self.past_file_path = tk.StringVar()
        self.current_file_path = tk.StringVar()
        self.output_file_path = tk.StringVar()

        self.default_product_code_options = ["旧ｺｰﾄﾞ連結", "中分類１", "取引先", "取引先名"]
        self.product_code_column = tk.StringVar(value=self.default_product_code_options[0])
        self.amount_column = tk.StringVar(value="金額")

        self.past_summary = None
        self.current_summary = None
        self.merged_data = None
        self.lost_sales_data = None # 売上消失商品データ用
        self.significant_decrease_data = None # 売上激減商品データ用

        # 売上激減の閾値（固定）
        self.DECREASE_THRESHOLD_PERCENT = -80.0

        self.create_widgets()

    def create_widgets(self):
        main_frame = ttk.Frame(self.root)
        main_frame.pack(fill="both", expand=True, padx=10, pady=10)

        input_frame = ttk.LabelFrame(main_frame, text="ファイル選択")
        input_frame.pack(fill="x", pady=5)
        # ... (ファイル選択部分は変更なし) ...
        ttk.Label(input_frame, text="過去の売上照会ファイル:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        ttk.Entry(input_frame, textvariable=self.past_file_path, width=50).grid(row=0, column=1, padx=5, pady=5)
        ttk.Button(input_frame, text="参照", command=lambda: self.browse_file(self.past_file_path)).grid(row=0, column=2, padx=5)
        ttk.Label(input_frame, text="現在の売上照会ファイル:").grid(row=1, column=0, padx=5, pady=5, sticky="w")
        ttk.Entry(input_frame, textvariable=self.current_file_path, width=50).grid(row=1, column=1, padx=5, pady=5)
        ttk.Button(input_frame, text="参照", command=lambda: self.browse_file(self.current_file_path)).grid(row=1, column=2, padx=5)
        ttk.Label(input_frame, text="分析結果の出力先:").grid(row=2, column=0, padx=5, pady=5, sticky="w")
        ttk.Entry(input_frame, textvariable=self.output_file_path, width=50).grid(row=2, column=1, padx=5, pady=5)
        ttk.Button(input_frame, text="参照", command=self.browse_output_file).grid(row=2, column=2, padx=5)

        column_frame = ttk.LabelFrame(main_frame, text="列名設定")
        column_frame.pack(fill="x", pady=5)
        # ... (列名設定部分は変更なし) ...
        ttk.Label(column_frame, text="集計基準列名:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        self.product_code_combo_main = ttk.Combobox(column_frame, textvariable=self.product_code_column, width=18, values=self.default_product_code_options, state="readonly")
        self.product_code_combo_main.grid(row=0, column=1, padx=5, pady=5)
        self.product_code_column.set(self.default_product_code_options[0])
        ttk.Label(column_frame, text="金額列名:").grid(row=0, column=2, padx=5, pady=5, sticky="w")
        self.amount_entry_main = ttk.Entry(column_frame, textvariable=self.amount_column, width=20)
        self.amount_entry_main.grid(row=0, column=3, padx=5, pady=5)
        ttk.Label(column_frame, text="※「分析実行」でシート/列名詳細設定").grid(row=1, column=0, columnspan=4, padx=5, pady=5)


        buttons_frame = ttk.Frame(main_frame)
        buttons_frame.pack(fill="x", pady=5)
        # ... (実行ボタン部分は変更なし) ...
        ttk.Button(buttons_frame, text="分析実行", command=self.analyze_sales, width=20).pack(side="left", padx=5)
        ttk.Button(buttons_frame, text="Excelに出力", command=self.export_to_excel, width=20).pack(side="left", padx=5)
        ttk.Button(buttons_frame, text="クリア", command=self.clear_results, width=20).pack(side="left", padx=5)

        tab_control = ttk.Notebook(main_frame)
        table_tab = ttk.Frame(tab_control)
        graph_tab = ttk.Frame(tab_control)
        # 新しいタブ「売上傾向分析」を追加
        self.sales_trend_tab = ttk.Frame(tab_control) # self にして後でアクセス可能に
        tab_control.add(table_tab, text='全体比較テーブル') # 名前を少し変更
        tab_control.add(graph_tab, text='全体比較グラフ') # 名前を少し変更
        tab_control.add(self.sales_trend_tab, text='売上傾向分析')
        tab_control.pack(expand=1, fill="both")

        # --- 全体比較テーブル ---
        cols_main = ("集計キー", "過去売上金額", "過去構成比", "現在売上金額", "現在構成比", "増減額", "増減率")
        self.tree = ttk.Treeview(table_tab, columns=cols_main, show="headings")
        for c in cols_main:
            self.tree.heading(c, text=c)
            self.tree.column(c, width=180 if c=="集計キー" else 130, anchor="center")
        ysb = ttk.Scrollbar(table_tab, orient="vertical", command=self.tree.yview)
        xsb = ttk.Scrollbar(table_tab, orient="horizontal", command=self.tree.xview)
        self.tree.configure(yscrollcommand=ysb.set, xscrollcommand=xsb.set)
        self.tree.pack(side="left", fill="both", expand=True)
        ysb.pack(side="right", fill="y")
        xsb.pack(side="bottom", fill="x")

        # --- 全体比較グラフフレーム ---
        self.graph_frame = graph_tab

        # --- 売上傾向分析タブのコンテンツ ---
        trend_frame = ttk.Frame(self.sales_trend_tab)
        trend_frame.pack(fill="both", expand=True, padx=5, pady=5)

        # 売上消失商品フレーム
        lost_sales_frame = ttk.LabelFrame(trend_frame, text="売上消失商品 (過去売上あり / 現在売上なし)")
        lost_sales_frame.pack(fill="x", pady=5, expand=True) # expand=True を追加
        cols_lost = ("集計キー", "過去売上金額")
        self.lost_sales_tree = ttk.Treeview(lost_sales_frame, columns=cols_lost, show="headings", height=7) # height指定
        for c in cols_lost:
            self.lost_sales_tree.heading(c, text=c)
            self.lost_sales_tree.column(c, width=250 if c=="集計キー" else 150, anchor="w" if c=="集計キー" else "e")
        ls_ysb = ttk.Scrollbar(lost_sales_frame, orient="vertical", command=self.lost_sales_tree.yview)
        self.lost_sales_tree.configure(yscrollcommand=ls_ysb.set)
        self.lost_sales_tree.pack(side="left", fill="both", expand=True)
        ls_ysb.pack(side="right", fill="y")


        # 売上激減商品フレーム
        decrease_sales_frame = ttk.LabelFrame(trend_frame, text=f"売上激減商品 (増減率 {self.DECREASE_THRESHOLD_PERCENT}%以下)")
        decrease_sales_frame.pack(fill="x", pady=5, expand=True) # expand=True を追加
        cols_decrease = ("集計キー", "過去売上金額", "現在売上金額", "増減額", "増減率")
        self.decrease_sales_tree = ttk.Treeview(decrease_sales_frame, columns=cols_decrease, show="headings", height=7) # height指定
        for c in cols_decrease:
            self.decrease_sales_tree.heading(c, text=c)
            self.decrease_sales_tree.column(c, width=200 if c=="集計キー" else 120, anchor="w" if c=="集計キー" else "e")
        ds_ysb = ttk.Scrollbar(decrease_sales_frame, orient="vertical", command=self.decrease_sales_tree.yview)
        self.decrease_sales_tree.configure(yscrollcommand=ds_ysb.set)
        self.decrease_sales_tree.pack(side="left", fill="both", expand=True)
        ds_ysb.pack(side="right", fill="y")

    # ... (browse_file, update_main_product_code_combo, browse_output_file, get_excel_sheets, normalize_column_name, find_best_match_column, show_sheet_selection_dialog は変更なし) ...
    def browse_file(self, var):
        f = filedialog.askopenfilename(title="Excelファイルを選択", filetypes=[("Excel files","*.xlsx *.xls"),("All","*.*")])
        if f:
            var.set(f)
            if var == self.past_file_path:
                self.update_main_product_code_combo()

    def update_main_product_code_combo(self):
        try:
            self.product_code_combo_main['values'] = self.default_product_code_options
            current_selection = self.product_code_column.get()
            if current_selection not in self.default_product_code_options or not current_selection:
                 self.product_code_column.set(self.default_product_code_options[0])
            elif current_selection in self.default_product_code_options:
                 self.product_code_combo_main.set(current_selection)
        except Exception as e:
            print(f"メイン画面のCombobox更新エラー（固定値設定時）: {e}")
            self.product_code_combo_main['values'] = self.default_product_code_options
            if self.product_code_column.get() not in self.default_product_code_options:
                self.product_code_column.set(self.default_product_code_options[0])

    def browse_output_file(self):
        f = filedialog.asksaveasfilename(title="保存先を選択", defaultextension=".xlsx", filetypes=[("Excel files","*.xlsx"),("All","*.*")])
        if f:
            self.output_file_path.set(f)

    def get_excel_sheets(self, path):
        try: return pd.ExcelFile(path).sheet_names
        except Exception as e: messagebox.showerror("エラー", f"シート取得失敗: {e}"); return []

    def normalize_column_name(self, name):
        if not isinstance(name, str): return str(name)
        return name.replace('　', ' ').strip()

    def find_best_match_column(self, df, target_name):
        if not target_name: return None
        if target_name in df.columns: return target_name
        norm_target = self.normalize_column_name(target_name)
        for col in df.columns:
            if self.normalize_column_name(col) == norm_target: return col
        if target_name in self.default_product_code_options:
            if target_name in df.columns: return target_name
            norm_default_opt = self.normalize_column_name(target_name)
            for col in df.columns:
                if self.normalize_column_name(col) == norm_default_opt: return col
        best_match, best_score = None, 0
        for col in df.columns:
            norm_col = self.normalize_column_name(col)
            if norm_target in norm_col or norm_col in norm_target:
                match_len = len(set(norm_target) & set(norm_col))
                if match_len > best_score: best_score, best_match = match_len, col
        if best_match: print(f"部分一致: '{target_name}' の代わりに '{best_match}' (スコア: {best_score}) を使用します。")
        return best_match

    def show_sheet_selection_dialog(self, past_file, current_file):
        dialog = tk.Toplevel(self.root)
        dialog.title("シート選択と列設定"); dialog.geometry("500x470"); dialog.transient(self.root); dialog.grab_set()
        past_sheets, current_sheets = self.get_excel_sheets(past_file), self.get_excel_sheets(current_file)
        ttk.Label(dialog, text="過去ファイルのシート:").pack(anchor="w", padx=10, pady=5)
        past_sheet_var = tk.StringVar(value=past_sheets[0] if past_sheets else "")
        past_combo = ttk.Combobox(dialog, textvariable=past_sheet_var, values=past_sheets, state="readonly", width=30)
        past_combo.pack(anchor="w", padx=10)
        ttk.Label(dialog, text="現在ファイルのシート:").pack(anchor="w", padx=10, pady=5)
        current_sheet_var = tk.StringVar(value=current_sheets[0] if current_sheets else "")
        current_combo = ttk.Combobox(dialog, textvariable=current_sheet_var, values=current_sheets, state="readonly", width=30)
        current_combo.pack(anchor="w", padx=10)
        preview_frame = ttk.LabelFrame(dialog, text="列名設定とプレビュー"); preview_frame.pack(fill="both", expand=True, padx=10, pady=10)
        product_code_var_dialog, amount_var_dialog = tk.StringVar(value=self.product_code_column.get()), tk.StringVar(value=self.amount_column.get())
        ttk.Label(preview_frame, text="集計基準列:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        product_code_combo_dialog = ttk.Combobox(preview_frame, textvariable=product_code_var_dialog, width=30)
        product_code_combo_dialog.grid(row=0, column=1, padx=5, pady=5)
        ttk.Label(preview_frame, text="金額列:").grid(row=1, column=0, padx=5, pady=5, sticky="w")
        amount_combo_dialog = ttk.Combobox(preview_frame, textvariable=amount_var_dialog, width=30)
        amount_combo_dialog.grid(row=1, column=1, padx=5, pady=5)
        preview_text = tk.Text(preview_frame, height=8, width=45); preview_text.grid(row=3, column=0, columnspan=2, padx=5, pady=5)
        def preview_columns_and_set_defaults():
            try:
                selected_past_sheet = past_sheet_var.get()
                if not selected_past_sheet: messagebox.showwarning("注意", "過去ファイルのシートを選択してください."); return
                df = pd.read_excel(past_file, sheet_name=selected_past_sheet, nrows=5); file_cols = df.columns.tolist()
                dialog_pc_options = sorted(list(set(self.default_product_code_options + file_cols)))
                product_code_combo_dialog['values'], amount_combo_dialog['values'] = dialog_pc_options, file_cols
                current_pc_selection_in_dialog = product_code_var_dialog.get()
                if current_pc_selection_in_dialog in dialog_pc_options: product_code_var_dialog.set(current_pc_selection_in_dialog)
                else:
                    main_pc_selection = self.product_code_column.get()
                    if main_pc_selection in file_cols and main_pc_selection in dialog_pc_options: product_code_var_dialog.set(main_pc_selection)
                    else:
                        found_default_in_file = False
                        for opt in self.default_product_code_options:
                            if opt in file_cols and opt in dialog_pc_options: product_code_var_dialog.set(opt); found_default_in_file = True; break
                        if not found_default_in_file and dialog_pc_options: product_code_var_dialog.set(dialog_pc_options[0])
                        elif not found_default_in_file and not dialog_pc_options : product_code_var_dialog.set("")
                current_amount_selection_in_dialog = amount_var_dialog.get()
                if current_amount_selection_in_dialog in file_cols: amount_var_dialog.set(current_amount_selection_in_dialog)
                else:
                    target_amount_col_for_find = self.amount_column.get(); best_amount_match = self.find_best_match_column(df, target_amount_col_for_find)
                    if best_amount_match and best_amount_match in file_cols: amount_var_dialog.set(best_amount_match)
                    elif file_cols: amount_var_dialog.set(file_cols[0])
                    else: amount_var_dialog.set("")
                preview_text.delete("1.0", tk.END); preview_text.insert(tk.END, "使用可能な列一覧 (過去ファイルより):\n" + "\n".join(f"{i+1}. {c}" for i, c in enumerate(file_cols)))
                if len(df) > 0:
                    preview_text.insert(tk.END, "\n\n最初の行のデータ例:\n")
                    for col_idx, col_name in enumerate(file_cols[:5]): preview_text.insert(tk.END, f"{col_name}: {df.iloc[0, col_idx]}\n")
            except Exception as e: messagebox.showerror("エラー", f"列プレビュー失敗: {e}"); import traceback; traceback.print_exc()
        past_combo.bind("<<ComboboxSelected>>", lambda event: preview_columns_and_set_defaults()); dialog.after(100, preview_columns_and_set_defaults)
        ttk.Button(preview_frame, text="列名プレビュー更新", command=preview_columns_and_set_defaults).grid(row=2, column=0, columnspan=2, pady=5)
        def on_ok():
            selected_pc_in_dialog, selected_amount_in_dialog = product_code_var_dialog.get(), amount_var_dialog.get()
            if not past_sheet_var.get() or not current_sheet_var.get(): messagebox.showerror("エラー", "過去と現在の両方のシートを選択してください."); return
            if not selected_pc_in_dialog or not selected_amount_in_dialog: messagebox.showerror("エラー", "集計基準列と金額列を選択または入力してください."); return
            self.past_sheet, self.current_sheet = past_sheet_var.get(), current_sheet_var.get()
            self.product_code_column.set(selected_pc_in_dialog); self.amount_column.set(selected_amount_in_dialog)
            self.product_code_combo_main.set(selected_pc_in_dialog); self.amount_entry_main.delete(0, tk.END); self.amount_entry_main.insert(0, selected_amount_in_dialog)
            dialog.destroy(); self.perform_analysis()
        ttk.Button(dialog, text="OK", command=on_ok).pack(pady=10); self.root.wait_window(dialog)

    def analyze_sales(self):
        if not self.past_file_path.get() or not self.current_file_path.get():
            messagebox.showerror("エラー", "過去と現在の両方のファイルを選択してください")
            return
        self.product_code_column.set(self.product_code_combo_main.get())
        self.amount_column.set(self.amount_entry_main.get())
        self.show_sheet_selection_dialog(self.past_file_path.get(), self.current_file_path.get())

    def perform_analysis(self):
        try:
            pc_user_choice = self.product_code_column.get()
            amt_user_choice = self.amount_column.get()
            if not pc_user_choice or not amt_user_choice:
                messagebox.showerror("エラー", "集計基準列または金額列が指定されていません。")
                return

            past_df = pd.read_excel(self.past_file_path.get(), sheet_name=self.past_sheet)
            curr_df = pd.read_excel(self.current_file_path.get(), sheet_name=self.current_sheet)

            pc_past_actual = self.find_best_match_column(past_df, pc_user_choice)
            if not pc_past_actual: raise ValueError(f"集計基準列 '{pc_user_choice}' が過去ファイル({self.past_sheet})で見つかりません。")
            if pc_past_actual != pc_user_choice: messagebox.showinfo("列名修正(過去/基準)", f"'{pc_user_choice}'の代わりに'{pc_past_actual}'を使用")
            amt_past_actual = self.find_best_match_column(past_df, amt_user_choice)
            if not amt_past_actual: raise ValueError(f"金額列 '{amt_user_choice}' が過去ファイル({self.past_sheet})で見つかりません。")
            if amt_past_actual != amt_user_choice: messagebox.showinfo("列名修正(過去/金額)", f"'{amt_user_choice}'の代わりに'{amt_past_actual}'を使用")
            pc_curr_actual = self.find_best_match_column(curr_df, pc_user_choice)
            if not pc_curr_actual: raise ValueError(f"集計基準列 '{pc_user_choice}' が現在ファイル({self.current_sheet})で見つかりません。")
            if pc_curr_actual != pc_user_choice: messagebox.showinfo("列名修正(現在/基準)", f"'{pc_user_choice}'の代わりに'{pc_curr_actual}'を使用")
            amt_curr_actual = self.find_best_match_column(curr_df, amt_user_choice)
            if not amt_curr_actual: raise ValueError(f"金額列 '{amt_user_choice}' が現在ファイル({self.current_sheet})で見つかりません。")
            if amt_curr_actual != amt_user_choice: messagebox.showinfo("列名修正(現在/金額)", f"'{amt_user_choice}'の代わりに'{amt_curr_actual}'を使用")

            for df, col_actual, df_name in [(past_df, amt_past_actual, "過去"), (curr_df, amt_curr_actual, "現在")]:
                if col_actual not in df.columns or df[col_actual].empty: df[col_actual] = 0
                elif df[col_actual].dtype == object or isinstance(df[col_actual].iloc[0], str) :
                    df[col_actual] = df[col_actual].astype(str).str.replace(',', '', regex=False).str.replace(' ', '', regex=False)
                    df[col_actual] = df[col_actual].str.replace(r'[¥円]', '', regex=True)
                    df[col_actual] = pd.to_numeric(df[col_actual], errors='coerce').fillna(0)
                elif not pd.api.types.is_numeric_dtype(df[col_actual]): df[col_actual] = pd.to_numeric(df[col_actual], errors='coerce').fillna(0)
                else: df[col_actual] = df[col_actual].fillna(0)
            
            self.past_summary = past_df.groupby(pc_past_actual, dropna=False)[amt_past_actual].sum().reset_index()
            self.past_summary.rename(columns={pc_past_actual: pc_user_choice, amt_past_actual: amt_user_choice}, inplace=True)
            past_total_sales = self.past_summary[amt_user_choice].sum()
            self.past_summary['構成比'] = (self.past_summary[amt_user_choice] / past_total_sales * 100) if past_total_sales != 0 else 0
            self.past_summary['構成比'] = self.past_summary['構成比'].fillna(0)

            self.current_summary = curr_df.groupby(pc_curr_actual, dropna=False)[amt_curr_actual].sum().reset_index()
            self.current_summary.rename(columns={pc_curr_actual: pc_user_choice, amt_curr_actual: amt_user_choice}, inplace=True)
            current_total_sales = self.current_summary[amt_user_choice].sum()
            self.current_summary['構成比'] = (self.current_summary[amt_user_choice] / current_total_sales * 100) if current_total_sales != 0 else 0
            self.current_summary['構成比'] = self.current_summary['構成比'].fillna(0)
            
            self.prepare_merged_data() # メインの比較データ作成
            self.display_results() # メインのテーブル表示
            self.plot_results() # メインのグラフ表示
            self.analyze_and_display_sales_trends() # 売上傾向分析の実行と表示
            messagebox.showinfo("完了", "分析が完了しました")
        except Exception as e:
            import traceback
            traceback.print_exc()
            messagebox.showerror("エラー", f"分析中にエラー: {e}")

    def prepare_merged_data(self):
        pc_key_for_merge = self.product_code_column.get()
        amt_key_for_merge = self.amount_column.get() 
        past_temp, current_temp = self.past_summary.copy(), self.current_summary.copy()
        if pc_key_for_merge not in past_temp.columns: raise ValueError(f"マージ用データ(過去)に列 '{pc_key_for_merge}' がありません。")
        if pc_key_for_merge not in current_temp.columns: raise ValueError(f"マージ用データ(現在)に列 '{pc_key_for_merge}' がありません。")
        past_temp[pc_key_for_merge], current_temp[pc_key_for_merge] = past_temp[pc_key_for_merge].astype(str), current_temp[pc_key_for_merge].astype(str)
        m = pd.merge(past_temp, current_temp, on=pc_key_for_merge, how='outer', suffixes=('_過去', '_現在'))
        m.rename(columns={f'{amt_key_for_merge}_過去': '過去売上金額', f'{amt_key_for_merge}_現在': '現在売上金額',
                          '構成比_過去': '過去構成比', '構成比_現在': '現在構成比'}, inplace=True)
        final_cols_to_fill = ['過去売上金額', '現在売上金額', '過去構成比', '現在構成比']
        for col in final_cols_to_fill:
            if col not in m.columns: m[col] = 0 
            else: m[col] = m[col].fillna(0)
        m['増減額'] = m['現在売上金額'] - m['過去売上金額']
        m['増減率'] = m.apply(lambda r: (r['増減額']/r['過去売上金額']*100) if r['過去売上金額']!=0 else (np.inf if r['増減額'] > 0 else (-np.inf if r['増減額'] < 0 else 0)), axis=1)
        self.merged_data = m.sort_values('現在売上金額', ascending=False)
        if pc_key_for_merge not in self.merged_data.columns: messagebox.showwarning("警告", f"最終データに集計キー '{pc_key_for_merge}' が見つかりません。")

    def display_results(self):
        for i in self.tree.get_children(): self.tree.delete(i)
        display_key_column = self.product_code_column.get()
        self.tree.heading("集計キー", text=display_key_column) 
        if self.merged_data is None or display_key_column not in self.merged_data.columns:
            messagebox.showerror("エラー", f"表示するデータ、または列 '{display_key_column}' が見つかりません。")
            return
        for _, r in self.merged_data.iterrows():
            self.tree.insert('', 'end', values=(
                r[display_key_column],
                f"{int(r['過去売上金額']):,}", f"{r['過去構成比']:.2f}%",
                f"{int(r['現在売上金額']):,}", f"{r['現在構成比']:.2f}%",
                f"{int(r['増減額']):,}",
                f"{r['増減率']:.2f}%" if pd.notnull(r['増減率']) and np.isfinite(r['増減率']) else ("-" if r['増減率']==0 else ("増加" if r['増減率']==np.inf else "減少"))
            ))
            
    def analyze_and_display_sales_trends(self):
        """売上消失商品と売上激減商品を分析し、専用タブに表示する"""
        # Treeviewをクリア
        for tree_widget in [self.lost_sales_tree, self.decrease_sales_tree]:
            for item in tree_widget.get_children():
                tree_widget.delete(item)

        if self.merged_data is None or self.merged_data.empty:
            self.lost_sales_tree.insert('', 'end', values=("データがありません", ""))
            self.decrease_sales_tree.insert('', 'end', values=("データがありません", "", "", "", ""))
            return

        key_col = self.product_code_column.get()

        # 売上消失商品の抽出
        self.lost_sales_data = self.merged_data[
            (self.merged_data['過去売上金額'] > 0) & (self.merged_data['現在売上金額'] == 0)
        ].sort_values(by='過去売上金額', ascending=False)

        for _, row in self.lost_sales_data.iterrows():
            self.lost_sales_tree.insert('', 'end', values=(
                row[key_col],
                f"{int(row['過去売上金額']):,}"
            ))

        # 売上激減商品の抽出
        self.significant_decrease_data = self.merged_data[
            (self.merged_data['過去売上金額'] > 0) &
            (self.merged_data['現在売上金額'] > 0) & # 現在も売上はある
            (self.merged_data['増減率'] <= self.DECREASE_THRESHOLD_PERCENT)
        ].sort_values(by='増減額', ascending=True) # 増減額の小さい順 (マイナスが大きい順)

        for _, row in self.significant_decrease_data.iterrows():
            self.decrease_sales_tree.insert('', 'end', values=(
                row[key_col],
                f"{int(row['過去売上金額']):,}",
                f"{int(row['現在売上金額']):,}",
                f"{int(row['増減額']):,}",
                f"{row['増減率']:.2f}%"
            ))
        
        if self.lost_sales_data.empty:
             self.lost_sales_tree.insert('', 'end', values=("該当なし", ""))
        if self.significant_decrease_data.empty:
             self.decrease_sales_tree.insert('', 'end', values=("該当なし", "", "", "", ""))


    def plot_results(self):
        # (前回のグラフサイズ調整済みのコード)
        for w in self.graph_frame.winfo_children(): w.destroy()
        if self.merged_data is None or self.merged_data.empty:
            ttk.Label(self.graph_frame, text="表示するデータがありません。").pack(padx=10, pady=10); return
        top_n = 10; top = self.merged_data.head(top_n)
        plot_label_column = self.product_code_column.get()
        if plot_label_column not in top.columns:
             messagebox.showerror("エラー", f"グラフ用データに列 '{plot_label_column}' が見つかりません。"); return
        try:
            plt.rcParams['font.family'] = 'Yu Gothic'
            if not any(f.name == 'Yu Gothic' for f in plt.matplotlib.font_manager.fontManager.ttflist): plt.rcParams['font.family'] = 'MS Gothic'
        except: plt.rcParams['font.family'] = 'sans-serif'
        plt.rcParams['axes.unicode_minus'] = False
        fig = plt.Figure(figsize=(14, 15), dpi=100)
        fig.subplots_adjust(hspace=0.6, wspace=0.35, bottom=0.15, top=0.95, left=0.08, right=0.95) 
        ax1 = fig.add_subplot(221) # 過去売上構成比
        nonzero_past = top[top['過去構成比'] > 0.01]
        if not nonzero_past.empty:
            wedges, texts, autotexts = ax1.pie( nonzero_past['過去構成比'], labels=nonzero_past[plot_label_column], autopct='%1.1f%%', startangle=90, pctdistance=0.85, wedgeprops={'edgecolor': 'white', 'linewidth': 0.5}, textprops={'fontsize': 7} )
            plt.setp(autotexts, size=6, weight="bold", color="white")
        ax1.set_title(f'過去売上構成比 (上位{len(nonzero_past)}項目)', fontsize=9); ax1.axis('equal')
        ax2 = fig.add_subplot(222) # 現在売上構成比
        nonzero_current = top[top['現在構成比'] > 0.01]
        if not nonzero_current.empty:
            wedges, texts, autotexts = ax2.pie( nonzero_current['現在構成比'], labels=nonzero_current[plot_label_column], autopct='%1.1f%%', startangle=90, pctdistance=0.85, wedgeprops={'edgecolor': 'white', 'linewidth': 0.5}, textprops={'fontsize': 7} )
            plt.setp(autotexts, size=6, weight="bold", color="white")
        ax2.set_title(f'現在売上構成比 (上位{len(nonzero_current)}項目)', fontsize=9); ax2.axis('equal')
        ax3 = fig.add_subplot(223) # 過去・現在 売上比較
        x_indices = np.arange(len(top)); width = 0.35
        rects1 = ax3.bar(x_indices - width/2, top['過去売上金額'], width, label='過去売上', color='skyblue')
        rects2 = ax3.bar(x_indices + width/2, top['現在売上金額'], width, label='現在売上', color='lightcoral')
        ax3.set_ylabel('売上金額', fontsize=8); ax3.set_title(f'過去・現在 売上比較 (上位{top_n}項目)', fontsize=9)
        ax3.set_xticks(x_indices); ax3.set_xticklabels(top[plot_label_column], rotation=45, ha='right', fontsize=7) 
        ax3.legend(fontsize=7); ax3.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda val, loc: "{:,.0f}".format(val))); ax3.tick_params(axis='y', labelsize=7)
        def autolabel(rects, ax_target):
            for rect in rects: ax_target.annotate('{:,.0f}'.format(rect.get_height()), xy=(rect.get_x() + rect.get_width() / 2, rect.get_height()), xytext=(0, 3), textcoords="offset points", ha='center', va='bottom', fontsize=6)
        autolabel(rects1, ax3); autolabel(rects2, ax3)
        ax4 = fig.add_subplot(224) # 売上増減額
        sorted_by_change_in_top = top.sort_values('増減額', ascending=True)
        plot_colors = ['red' if val < 0 else 'green' for val in sorted_by_change_in_top['増減額']]
        bars = ax4.barh(sorted_by_change_in_top[plot_label_column], sorted_by_change_in_top['増減額'], color=plot_colors, edgecolor='grey')
        ax4.set_xlabel('増減額', fontsize=8); ax4.set_title(f'売上増減額 (現在売上上位{top_n}項目内)', fontsize=9)
        ax4.get_xaxis().set_major_formatter(plt.FuncFormatter(lambda val, loc: "{:,.0f}".format(val))); ax4.tick_params(axis='y', labelsize=7); ax4.tick_params(axis='x', labelsize=7)
        for bar in bars:
            bar_width_val = bar.get_width();
            if bar_width_val == 0: continue
            ha_align, offset_val = ('left', 3) if bar_width_val > 0 else ('right', -3)
            ax4.text(bar.get_width() + offset_val , bar.get_y() + bar.get_height()/2., '{:,.0f}'.format(bar_width_val), ha=ha_align, va='center', fontsize=6)
        canvas = FigureCanvasTkAgg(fig, master=self.graph_frame); canvas.draw(); canvas.get_tk_widget().pack(fill="both", expand=True)
        toolbar = NavigationToolbar2Tk(canvas, self.graph_frame); toolbar.update()

    def export_to_excel(self):
        # (変更なし)
        if self.merged_data is None: messagebox.showerror("エラー", "先に分析を実行してください"); return
        path = self.output_file_path.get()
        if not path: path = filedialog.asksaveasfilename(title="保存先を選択", defaultextension=".xlsx", filetypes=[("Excel files","*.xlsx"),("All","*.*")]);
        if not path: return
        self.output_file_path.set(path)
        try:
            wb = openpyxl.Workbook(); ws = wb.active; ws.title = "売上分析結果"
            export_key_column_name = self.product_code_column.get()
            ws.merge_cells('A1:G1'); ws['A1'] = f"{export_key_column_name}別売上分析"; ws['A1'].font = Font(size=14, bold=True, color="000080"); ws['A1'].alignment = Alignment(horizontal='center')
            ws.merge_cells('A2:C2'); ws['A2'] = f"作成日: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}"; ws['A2'].font = Font(italic=True)
            headers = [export_key_column_name,"過去売上金額","過去構成比","現在売上金額","現在構成比","増減額","増減率"]
            for i, h in enumerate(headers, start=1):
                c = ws.cell(row=4, column=i, value=h); c.font = Font(bold=True, color="FFFFFF"); c.alignment = Alignment(horizontal='center', vertical='center'); c.fill = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")
            if export_key_column_name not in self.merged_data.columns: messagebox.showerror("エラー", f"Excelエクスポートエラー: 列 '{export_key_column_name}' がデータに存在しません。"); return
            for idx, r_tuple in enumerate(self.merged_data.itertuples(index=False), start=5):
                key_value = getattr(r_tuple, export_key_column_name)
                ws.cell(row=idx, column=1, value=key_value)
                ws.cell(row=idx, column=2, value=r_tuple.過去売上金額); ws.cell(row=idx, column=3, value=r_tuple.過去構成比/100 if pd.notnull(r_tuple.過去構成比) else 0)
                ws.cell(row=idx, column=4, value=r_tuple.現在売上金額); ws.cell(row=idx, column=5, value=r_tuple.現在構成比/100 if pd.notnull(r_tuple.現在構成比) else 0)
                ws.cell(row=idx, column=6, value=r_tuple.増減額); ws.cell(row=idx, column=7, value=r_tuple.増減率/100 if pd.notnull(r_tuple.増減率) and np.isfinite(r_tuple.増減率) else 0)
                ws.cell(row=idx, column=1).alignment = Alignment(horizontal='left')
                for col_num in [2, 4, 6]: ws.cell(row=idx, column=col_num).number_format = '#,##0'; ws.cell(row=idx, column=col_num).alignment = Alignment(horizontal='right')
                for col_num in [3, 5, 7]: ws.cell(row=idx, column=col_num).number_format = '0.00%'; ws.cell(row=idx, column=col_num).alignment = Alignment(horizontal='right')
                if r_tuple.増減額 < 0: ws.cell(row=idx, column=6).font = Font(color="FF0000"); ws.cell(row=idx, column=7).font = Font(color="FF0000")
                elif r_tuple.増減額 > 0 : ws.cell(row=idx, column=6).font = Font(color="008000"); ws.cell(row=idx, column=7).font = Font(color="008000")
            for i, column_cells in enumerate(ws.columns):
                max_length = 0; column_letter_val = get_column_letter(i + 1) 
                if ws.cell(row=4, column=i+1).value: max_length = max(max_length, len(str(ws.cell(row=4, column=i+1).value)))
                for cell in column_cells[4:]: 
                    try:
                        if cell.value is not None: 
                            cell_str_len = len(str(cell.value))
                            if isinstance(cell.value, (int, float)):
                                if cell.number_format and '%' in cell.number_format: cell_str_len = len(f"{cell.value:.2%}")
                                elif cell.number_format and ',' in cell.number_format: cell_str_len = len(f"{cell.value:,}")
                            max_length = max(max_length, cell_str_len)
                    except: pass 
                adjusted_width = (max_length + 2); ws.column_dimensions[column_letter_val].width = adjusted_width if adjusted_width < 50 else 50
            total_row = 5 + len(self.merged_data)
            ws.cell(row=total_row, column=1, value="合計").font = Font(bold=True); ws.cell(row=total_row, column=1).alignment = Alignment(horizontal='center'); ws.cell(row=total_row, column=1).fill = PatternFill(start_color="DDDDDD", end_color="DDDDDD", fill_type="solid")
            for col_idx, key in [(2, '過去売上金額'), (4, '現在売上金額'), (6, '増減額')]:
                val = self.merged_data[key].sum(); cell = ws.cell(row=total_row, column=col_idx, value=val); cell.font = Font(bold=True); cell.number_format = '#,##0'; cell.alignment = Alignment(horizontal='right'); cell.fill = PatternFill(start_color="DDDDDD", end_color="DDDDDD", fill_type="solid")
            chart_sheet = wb.create_sheet(title="グラフについて"); chart_sheet['A1'] = "グラフの確認は、分析アプリ本体の「グラフ」タブで行ってください。"; chart_sheet['A1'].font = Font(size=12, bold=True); chart_sheet.merge_cells('A1:E1')
            wb.save(path); messagebox.showinfo("完了", f"Excelファイルを保存しました: {path}")
        except PermissionError: messagebox.showerror("エラー", f"ファイル '{path}' への書き込み権限がありません。\nファイルが開かれていないか確認してください。")
        except Exception as e: import traceback; traceback.print_exc(); messagebox.showerror("エラー", f"Excelファイル出力中にエラーが発生しました: {e}")

    def clear_results(self):
        for tree_widget in [self.tree, self.lost_sales_tree, self.decrease_sales_tree]:
            for item in tree_widget.get_children():
                tree_widget.delete(item)
        for w in self.graph_frame.winfo_children(): w.destroy()
        ttk.Label(self.graph_frame, text="分析を実行するとここにグラフが表示されます。").pack(padx=10,pady=10) # 初期メッセージ再表示
        
        # 該当なしの場合のメッセージもクリア後に再挿入するならここで
        if hasattr(self, 'lost_sales_tree'): # Treeが初期化されていれば
             self.lost_sales_tree.insert('', 'end', values=("分析を実行してください", ""))
        if hasattr(self, 'decrease_sales_tree'):
             self.decrease_sales_tree.insert('', 'end', values=("分析を実行してください", "", "", "", ""))

        self.past_summary = None
        self.current_summary = None
        self.merged_data = None
        self.lost_sales_data = None
        self.significant_decrease_data = None
        self.product_code_column.set(self.default_product_code_options[0])
        self.amount_column.set("金額")
        messagebox.showinfo("クリア", "分析結果をクリアしました")

def main():
    root = tk.Tk()
    app = SalesAnalysisApp(root)
    ttk.Label(app.graph_frame, text="分析を実行するとここにグラフが表示されます。").pack(padx=10,pady=10)
    # 初期状態で新しいタブにもメッセージ表示
    if hasattr(app, 'lost_sales_tree'):
        app.lost_sales_tree.insert('', 'end', values=("分析を実行してください", ""))
    if hasattr(app, 'decrease_sales_tree'):
        app.decrease_sales_tree.insert('', 'end', values=("分析を実行してください", "", "", "", ""))
    root.mainloop()

if __name__ == "__main__":
    main()

In [5]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog, ttk, messagebox
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
import openpyxl
from openpyxl.styles import Font, Alignment, PatternFill
from openpyxl.utils import get_column_letter
import re
import numpy as np

class SalesAnalysisApp:
    def __init__(self, root):
        self.root = root
        self.root.title("売上照会分析アプリ")
        self.root.geometry("1200x800") # ウィンドウサイズ

        self.past_file_path = tk.StringVar()
        self.current_file_path = tk.StringVar()
        self.output_file_path = tk.StringVar()

        # 固定の集計基準列名の選択肢
        self.default_product_code_options = ["旧ｺｰﾄﾞ連結", "中分類１", "取引先", "取引先名"]
        self.product_code_column = tk.StringVar(value=self.default_product_code_options[0]) # 初期値をリストの先頭に
        self.amount_column = tk.StringVar(value="金額")

        self.past_summary = None
        self.current_summary = None
        self.merged_data = None

        self.create_widgets()

    def create_widgets(self):
        main_frame = ttk.Frame(self.root)
        main_frame.pack(fill="both", expand=True, padx=10, pady=10)

        # --- ファイル選択フレーム ---
        input_frame = ttk.LabelFrame(main_frame, text="ファイル選択")
        input_frame.pack(fill="x", pady=5)
        ttk.Label(input_frame, text="過去の売上照会ファイル:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        ttk.Entry(input_frame, textvariable=self.past_file_path, width=50).grid(row=0, column=1, padx=5, pady=5)
        ttk.Button(input_frame, text="参照", command=lambda: self.browse_file(self.past_file_path)).grid(row=0, column=2, padx=5)

        ttk.Label(input_frame, text="現在の売上照会ファイル:").grid(row=1, column=0, padx=5, pady=5, sticky="w")
        ttk.Entry(input_frame, textvariable=self.current_file_path, width=50).grid(row=1, column=1, padx=5, pady=5)
        ttk.Button(input_frame, text="参照", command=lambda: self.browse_file(self.current_file_path)).grid(row=1, column=2, padx=5)

        ttk.Label(input_frame, text="分析結果の出力先:").grid(row=2, column=0, padx=5, pady=5, sticky="w")
        ttk.Entry(input_frame, textvariable=self.output_file_path, width=50).grid(row=2, column=1, padx=5, pady=5)
        ttk.Button(input_frame, text="参照", command=self.browse_output_file).grid(row=2, column=2, padx=5)

        # --- 列名設定フレーム ---
        column_frame = ttk.LabelFrame(main_frame, text="列名設定")
        column_frame.pack(fill="x", pady=5)
        ttk.Label(column_frame, text="集計基準列名:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        # 集計基準列のCombobox (メイン画面)
        self.product_code_combo_main = ttk.Combobox(column_frame, textvariable=self.product_code_column, width=18, values=self.default_product_code_options, state="readonly")
        self.product_code_combo_main.grid(row=0, column=1, padx=5, pady=5)
        self.product_code_column.set(self.default_product_code_options[0]) # 初期値設定

        ttk.Label(column_frame, text="金額列名:").grid(row=0, column=2, padx=5, pady=5, sticky="w")
        self.amount_entry_main = ttk.Entry(column_frame, textvariable=self.amount_column, width=20)
        self.amount_entry_main.grid(row=0, column=3, padx=5, pady=5)
        ttk.Label(column_frame, text="※「分析実行」でシート/列名詳細設定").grid(row=1, column=0, columnspan=4, padx=5, pady=5)


        # --- 実行ボタンフレーム ---
        buttons_frame = ttk.Frame(main_frame)
        buttons_frame.pack(fill="x", pady=5)
        ttk.Button(buttons_frame, text="分析実行", command=self.analyze_sales, width=20).pack(side="left", padx=5)
        ttk.Button(buttons_frame, text="Excelに出力", command=self.export_to_excel, width=20).pack(side="left", padx=5)
        ttk.Button(buttons_frame, text="クリア", command=self.clear_results, width=20).pack(side="left", padx=5)

        # --- タブコントロール ---
        tab_control = ttk.Notebook(main_frame)
        table_tab = ttk.Frame(tab_control)
        graph_tab = ttk.Frame(tab_control)
        tab_control.add(table_tab, text='テーブル')
        tab_control.add(graph_tab, text='グラフ')
        tab_control.pack(expand=1, fill="both")

        # --- テーブル表示 ---
        cols = ("集計キー", "過去売上金額", "過去構成比", "現在売上金額", "現在構成比", "増減額", "増減率")
        self.tree = ttk.Treeview(table_tab, columns=cols, show="headings")
        for c in cols:
            self.tree.heading(c, text=c)
            self.tree.column(c, width=180 if c=="集計キー" else 130, anchor="center")
        ysb = ttk.Scrollbar(table_tab, orient="vertical", command=self.tree.yview)
        xsb = ttk.Scrollbar(table_tab, orient="horizontal", command=self.tree.xview)
        self.tree.configure(yscrollcommand=ysb.set, xscrollcommand=xsb.set)
        self.tree.pack(side="left", fill="both", expand=True)
        ysb.pack(side="right", fill="y")
        xsb.pack(side="bottom", fill="x")

        # --- グラフフレーム ---
        self.graph_frame = graph_tab

    def browse_file(self, var):
        f = filedialog.askopenfilename(title="Excelファイルを選択", filetypes=[("Excel files","*.xlsx *.xls"),("All","*.*")])
        if f:
            var.set(f)
            if var == self.past_file_path:
                self.update_main_product_code_combo()

    def update_main_product_code_combo(self):
        try:
            self.product_code_combo_main['values'] = self.default_product_code_options
            current_selection = self.product_code_column.get()
            if current_selection not in self.default_product_code_options or not current_selection:
                 self.product_code_column.set(self.default_product_code_options[0])
            elif current_selection in self.default_product_code_options:
                 self.product_code_combo_main.set(current_selection)
        except Exception as e:
            print(f"メイン画面のCombobox更新エラー（固定値設定時）: {e}")
            self.product_code_combo_main['values'] = self.default_product_code_options
            if self.product_code_column.get() not in self.default_product_code_options:
                self.product_code_column.set(self.default_product_code_options[0])

    def browse_output_file(self):
        f = filedialog.asksaveasfilename(title="保存先を選択", defaultextension=".xlsx", filetypes=[("Excel files","*.xlsx"),("All","*.*")])
        if f:
            self.output_file_path.set(f)

    def get_excel_sheets(self, path):
        try:
            return pd.ExcelFile(path).sheet_names
        except Exception as e:
            messagebox.showerror("エラー", f"シート取得失敗: {e}")
            return []

    def normalize_column_name(self, name):
        if not isinstance(name, str):
            return str(name)
        name = name.replace('　', ' ').strip()
        return name

    def find_best_match_column(self, df, target_name):
        if not target_name:
            return None
        if target_name in df.columns:
            return target_name
        norm_target = self.normalize_column_name(target_name)
        for col in df.columns:
            if self.normalize_column_name(col) == norm_target:
                return col
        if target_name in self.default_product_code_options:
            if target_name in df.columns:
                return target_name
            norm_default_opt = self.normalize_column_name(target_name)
            for col in df.columns:
                if self.normalize_column_name(col) == norm_default_opt:
                    return col
        best_match = None
        best_score = 0
        for col in df.columns:
            norm_col = self.normalize_column_name(col)
            if norm_target in norm_col or norm_col in norm_target:
                match_len = len(set(norm_target) & set(norm_col))
                if match_len > best_score:
                    best_score = match_len
                    best_match = col
        if best_match:
            print(f"部分一致: '{target_name}' の代わりに '{best_match}' (スコア: {best_score}) を使用します。")
        return best_match

    def show_sheet_selection_dialog(self, past_file, current_file):
        dialog = tk.Toplevel(self.root)
        dialog.title("シート選択と列設定")
        dialog.geometry("500x470")
        dialog.transient(self.root)
        dialog.grab_set()

        past_sheets = self.get_excel_sheets(past_file)
        current_sheets = self.get_excel_sheets(current_file)

        ttk.Label(dialog, text="過去ファイルのシート:").pack(anchor="w", padx=10, pady=5)
        past_sheet_var = tk.StringVar(value=past_sheets[0] if past_sheets else "")
        past_combo = ttk.Combobox(dialog, textvariable=past_sheet_var, values=past_sheets, state="readonly", width=30)
        past_combo.pack(anchor="w", padx=10)

        ttk.Label(dialog, text="現在ファイルのシート:").pack(anchor="w", padx=10, pady=5)
        current_sheet_var = tk.StringVar(value=current_sheets[0] if current_sheets else "")
        current_combo = ttk.Combobox(dialog, textvariable=current_sheet_var, values=current_sheets, state="readonly", width=30)
        current_combo.pack(anchor="w", padx=10)

        preview_frame = ttk.LabelFrame(dialog, text="列名設定とプレビュー")
        preview_frame.pack(fill="both", expand=True, padx=10, pady=10)
        
        product_code_var_dialog = tk.StringVar(value=self.product_code_column.get())
        amount_var_dialog = tk.StringVar(value=self.amount_column.get())

        ttk.Label(preview_frame, text="集計基準列:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        product_code_combo_dialog = ttk.Combobox(preview_frame, textvariable=product_code_var_dialog, width=30)
        product_code_combo_dialog.grid(row=0, column=1, padx=5, pady=5)

        ttk.Label(preview_frame, text="金額列:").grid(row=1, column=0, padx=5, pady=5, sticky="w")
        amount_combo_dialog = ttk.Combobox(preview_frame, textvariable=amount_var_dialog, width=30)
        amount_combo_dialog.grid(row=1, column=1, padx=5, pady=5)

        preview_text = tk.Text(preview_frame, height=8, width=45)
        preview_text.grid(row=3, column=0, columnspan=2, padx=5, pady=5)

        def preview_columns_and_set_defaults():
            try:
                selected_past_sheet = past_sheet_var.get()
                if not selected_past_sheet:
                    messagebox.showwarning("注意", "過去ファイルのシートを選択してください。")
                    return

                df = pd.read_excel(past_file, sheet_name=selected_past_sheet, nrows=5)
                file_cols = df.columns.tolist()
                
                dialog_pc_options = sorted(list(set(self.default_product_code_options + file_cols)))
                product_code_combo_dialog['values'] = dialog_pc_options
                amount_combo_dialog['values'] = file_cols

                current_pc_selection_in_dialog = product_code_var_dialog.get()
                if current_pc_selection_in_dialog in dialog_pc_options:
                    product_code_var_dialog.set(current_pc_selection_in_dialog)
                else:
                    main_pc_selection = self.product_code_column.get()
                    if main_pc_selection in file_cols and main_pc_selection in dialog_pc_options:
                        product_code_var_dialog.set(main_pc_selection)
                    else:
                        found_default_in_file = False
                        for opt in self.default_product_code_options:
                            if opt in file_cols and opt in dialog_pc_options:
                                product_code_var_dialog.set(opt)
                                found_default_in_file = True
                                break
                        if not found_default_in_file and dialog_pc_options:
                            product_code_var_dialog.set(dialog_pc_options[0])
                        elif not found_default_in_file and not dialog_pc_options :
                             product_code_var_dialog.set("")

                current_amount_selection_in_dialog = amount_var_dialog.get()
                if current_amount_selection_in_dialog in file_cols:
                    amount_var_dialog.set(current_amount_selection_in_dialog)
                else:
                    target_amount_col_for_find = self.amount_column.get()
                    best_amount_match = self.find_best_match_column(df, target_amount_col_for_find)
                    if best_amount_match and best_amount_match in file_cols:
                        amount_var_dialog.set(best_amount_match)
                    elif file_cols:
                        amount_var_dialog.set(file_cols[0])
                    else:
                        amount_var_dialog.set("")

                preview_text.delete("1.0", tk.END)
                preview_text.insert(tk.END, "使用可能な列一覧 (過去ファイルより):\n" + "\n".join(f"{i+1}. {c}" for i, c in enumerate(file_cols)))
                if len(df) > 0:
                    preview_text.insert(tk.END, "\n\n最初の行のデータ例:\n")
                    for col_idx, col_name in enumerate(file_cols[:5]):
                        val = df.iloc[0, col_idx]
                        preview_text.insert(tk.END, f"{col_name}: {val}\n")
            except Exception as e:
                messagebox.showerror("エラー", f"列プレビュー失敗: {e}")
                import traceback
                traceback.print_exc()

        past_combo.bind("<<ComboboxSelected>>", lambda event: preview_columns_and_set_defaults())
        dialog.after(100, preview_columns_and_set_defaults)
        ttk.Button(preview_frame, text="列名プレビュー更新", command=preview_columns_and_set_defaults).grid(row=2, column=0, columnspan=2, pady=5)

        def on_ok():
            selected_pc_in_dialog = product_code_var_dialog.get()
            selected_amount_in_dialog = amount_var_dialog.get()
            if not past_sheet_var.get() or not current_sheet_var.get():
                messagebox.showerror("エラー", "過去と現在の両方のシートを選択してください。")
                return
            if not selected_pc_in_dialog or not selected_amount_in_dialog:
                messagebox.showerror("エラー", "集計基準列と金額列を選択または入力してください。")
                return

            self.past_sheet = past_sheet_var.get()
            self.current_sheet = current_sheet_var.get()
            self.product_code_column.set(selected_pc_in_dialog)
            self.amount_column.set(selected_amount_in_dialog)
            self.product_code_combo_main.set(selected_pc_in_dialog)
            self.amount_entry_main.delete(0, tk.END)
            self.amount_entry_main.insert(0, selected_amount_in_dialog)
            dialog.destroy()
            self.perform_analysis()

        ttk.Button(dialog, text="OK", command=on_ok).pack(pady=10)
        self.root.wait_window(dialog)

    def analyze_sales(self):
        if not self.past_file_path.get() or not self.current_file_path.get():
            messagebox.showerror("エラー", "過去と現在の両方のファイルを選択してください")
            return
        self.product_code_column.set(self.product_code_combo_main.get())
        self.amount_column.set(self.amount_entry_main.get())
        self.show_sheet_selection_dialog(self.past_file_path.get(), self.current_file_path.get())

    def perform_analysis(self):
        try:
            pc_user_choice = self.product_code_column.get()
            amt_user_choice = self.amount_column.get()
            if not pc_user_choice or not amt_user_choice:
                messagebox.showerror("エラー", "集計基準列または金額列が指定されていません。")
                return

            print(f"過去ファイルを読み込み中: {self.past_file_path.get()}, シート: {self.past_sheet}")
            past_df = pd.read_excel(self.past_file_path.get(), sheet_name=self.past_sheet)
            print(f"現在ファイルを読み込み中: {self.current_file_path.get()}, シート: {self.current_sheet}")
            curr_df = pd.read_excel(self.current_file_path.get(), sheet_name=self.current_sheet)

            pc_past_actual = self.find_best_match_column(past_df, pc_user_choice)
            if not pc_past_actual: raise ValueError(f"集計基準列 '{pc_user_choice}' が過去ファイル({self.past_sheet})で見つかりません。利用可能な列: {past_df.columns.tolist()}")
            if pc_past_actual != pc_user_choice: messagebox.showinfo("列名修正(過去/基準)", f"'{pc_user_choice}'の代わりに'{pc_past_actual}'を使用")
            amt_past_actual = self.find_best_match_column(past_df, amt_user_choice)
            if not amt_past_actual: raise ValueError(f"金額列 '{amt_user_choice}' が過去ファイル({self.past_sheet})で見つかりません。利用可能な列: {past_df.columns.tolist()}")
            if amt_past_actual != amt_user_choice: messagebox.showinfo("列名修正(過去/金額)", f"'{amt_user_choice}'の代わりに'{amt_past_actual}'を使用")
            pc_curr_actual = self.find_best_match_column(curr_df, pc_user_choice)
            if not pc_curr_actual: raise ValueError(f"集計基準列 '{pc_user_choice}' が現在ファイル({self.current_sheet})で見つかりません。利用可能な列: {curr_df.columns.tolist()}")
            if pc_curr_actual != pc_user_choice: messagebox.showinfo("列名修正(現在/基準)", f"'{pc_user_choice}'の代わりに'{pc_curr_actual}'を使用")
            amt_curr_actual = self.find_best_match_column(curr_df, amt_user_choice)
            if not amt_curr_actual: raise ValueError(f"金額列 '{amt_user_choice}' が現在ファイル({self.current_sheet})で見つかりません。利用可能な列: {curr_df.columns.tolist()}")
            if amt_curr_actual != amt_user_choice: messagebox.showinfo("列名修正(現在/金額)", f"'{amt_user_choice}'の代わりに'{amt_curr_actual}'を使用")

            for df, col_actual, df_name in [(past_df, amt_past_actual, "過去"), (curr_df, amt_curr_actual, "現在")]:
                if col_actual not in df.columns or df[col_actual].empty:
                    df[col_actual] = 0
                elif df[col_actual].dtype == object or isinstance(df[col_actual].iloc[0], str) :
                    df[col_actual] = df[col_actual].astype(str).str.replace(',', '', regex=False).str.replace(' ', '', regex=False)
                    df[col_actual] = df[col_actual].str.replace(r'[¥円]', '', regex=True)
                    df[col_actual] = pd.to_numeric(df[col_actual], errors='coerce').fillna(0)
                elif not pd.api.types.is_numeric_dtype(df[col_actual]):
                    df[col_actual] = pd.to_numeric(df[col_actual], errors='coerce').fillna(0)
                else:
                    df[col_actual] = df[col_actual].fillna(0)
            
            self.past_summary = past_df.groupby(pc_past_actual, dropna=False)[amt_past_actual].sum().reset_index()
            self.past_summary.rename(columns={pc_past_actual: pc_user_choice, amt_past_actual: amt_user_choice}, inplace=True)
            past_total_sales = self.past_summary[amt_user_choice].sum()
            self.past_summary['構成比'] = (self.past_summary[amt_user_choice] / past_total_sales * 100) if past_total_sales != 0 else 0
            self.past_summary['構成比'] = self.past_summary['構成比'].fillna(0)

            self.current_summary = curr_df.groupby(pc_curr_actual, dropna=False)[amt_curr_actual].sum().reset_index()
            self.current_summary.rename(columns={pc_curr_actual: pc_user_choice, amt_curr_actual: amt_user_choice}, inplace=True)
            current_total_sales = self.current_summary[amt_user_choice].sum()
            self.current_summary['構成比'] = (self.current_summary[amt_user_choice] / current_total_sales * 100) if current_total_sales != 0 else 0
            self.current_summary['構成比'] = self.current_summary['構成比'].fillna(0)
            
            self.prepare_merged_data()
            self.display_results()
            self.plot_results()
            messagebox.showinfo("完了", "分析が完了しました")
        except Exception as e:
            import traceback
            traceback.print_exc()
            messagebox.showerror("エラー", f"分析中にエラー: {e}")

    def prepare_merged_data(self):
        pc_key_for_merge = self.product_code_column.get()
        amt_key_for_merge = self.amount_column.get() 

        past_temp = self.past_summary.copy()
        current_temp = self.current_summary.copy()
        
        if pc_key_for_merge not in past_temp.columns: raise ValueError(f"マージ用データ(過去)に列 '{pc_key_for_merge}' がありません。列: {past_temp.columns}")
        if pc_key_for_merge not in current_temp.columns: raise ValueError(f"マージ用データ(現在)に列 '{pc_key_for_merge}' がありません。列: {current_temp.columns}")

        past_temp[pc_key_for_merge] = past_temp[pc_key_for_merge].astype(str)
        current_temp[pc_key_for_merge] = current_temp[pc_key_for_merge].astype(str)
        
        # suffixesを指定して列名の衝突を明確に管理
        m = pd.merge(past_temp, current_temp, on=pc_key_for_merge, how='outer', suffixes=('_過去', '_現在'))
        
        # 金額列と構成比列のリネーム (suffixesで生成された列名から変更)
        # amt_key_for_merge は過去・現在ともに同じ名前なので、suffixesが付与される
        m.rename(columns={
            f'{amt_key_for_merge}_過去': '過去売上金額',
            f'{amt_key_for_merge}_現在': '現在売上金額',
            '構成比_過去': '過去構成比',
            '構成比_現在': '現在構成比',
        }, inplace=True)

        final_cols_to_fill = ['過去売上金額', '現在売上金額', '過去構成比', '現在構成比']
        for col in final_cols_to_fill:
            if col not in m.columns:
                m[col] = 0 
            else:
                m[col] = m[col].fillna(0) # fillna はここでまとめて行う

        m['増減額'] = m['現在売上金額'] - m['過去売上金額']
        m['増減率'] = m.apply(lambda r: (r['増減額']/r['過去売上金額']*100) if r['過去売上金額']!=0 else (np.inf if r['増減額'] > 0 else (-np.inf if r['増減額'] < 0 else 0)), axis=1)
        
        self.merged_data = m.sort_values('現在売上金額', ascending=False)
        
        if pc_key_for_merge not in self.merged_data.columns:
             messagebox.showwarning("警告", f"最終データに集計キー '{pc_key_for_merge}' が見つかりません。利用可能な列: {self.merged_data.columns}")

    def display_results(self):
        for i in self.tree.get_children():
            self.tree.delete(i)
        display_key_column = self.product_code_column.get()
        self.tree.heading("集計キー", text=display_key_column) 
        if self.merged_data is None or display_key_column not in self.merged_data.columns:
            print(f"表示エラー: merged_data is None: {self.merged_data is None}")
            if self.merged_data is not None: print(f"表示エラー: 列 '{display_key_column}' not in merged_data.columns: {self.merged_data.columns.tolist()}")
            messagebox.showerror("エラー", f"表示するデータ、または列 '{display_key_column}' が見つかりません。")
            return
        for _, r in self.merged_data.iterrows():
            self.tree.insert('', 'end', values=(
                r[display_key_column],
                f"{int(r['過去売上金額']):,}", f"{r['過去構成比']:.2f}%",
                f"{int(r['現在売上金額']):,}", f"{r['現在構成比']:.2f}%",
                f"{int(r['増減額']):,}",
                f"{r['増減率']:.2f}%" if pd.notnull(r['増減率']) and np.isfinite(r['増減率']) else ("-" if r['増減率']==0 else ("増加" if r['増減率']==np.inf else "減少"))
            ))

    def plot_results(self):
        for w in self.graph_frame.winfo_children():
            w.destroy()

        if self.merged_data is None or self.merged_data.empty:
            ttk.Label(self.graph_frame, text="表示するデータがありません。").pack(padx=10, pady=10)
            return

        top_n = 10
        top = self.merged_data.head(top_n)
        
        plot_label_column = self.product_code_column.get()
        if plot_label_column not in top.columns:
             messagebox.showerror("エラー", f"グラフ用データに列 '{plot_label_column}' が見つかりません。利用可能な列: {top.columns}")
             return

        try:
            plt.rcParams['font.family'] = 'Yu Gothic'
            if not any(f.name == 'Yu Gothic' for f in plt.matplotlib.font_manager.fontManager.ttflist):
                 plt.rcParams['font.family'] = 'MS Gothic'
        except:
             plt.rcParams['font.family'] = 'sans-serif'
        plt.rcParams['axes.unicode_minus'] = False

        fig = plt.Figure(figsize=(14, 15), dpi=100)
        fig.subplots_adjust(hspace=0.6, wspace=0.35, bottom=0.15, top=0.95, left=0.08, right=0.95) 

        ax1 = fig.add_subplot(221)
        nonzero_past = top[top['過去構成比'] > 0.01]
        if not nonzero_past.empty:
            wedges, texts, autotexts = ax1.pie(
                nonzero_past['過去構成比'], labels=nonzero_past[plot_label_column], autopct='%1.1f%%',
                startangle=90, pctdistance=0.85, wedgeprops={'edgecolor': 'white', 'linewidth': 0.5},
                textprops={'fontsize': 7} )
            plt.setp(autotexts, size=6, weight="bold", color="white")
        ax1.set_title(f'過去売上構成比 (上位{len(nonzero_past)}項目)', fontsize=9)
        ax1.axis('equal')

        ax2 = fig.add_subplot(222)
        nonzero_current = top[top['現在構成比'] > 0.01]
        if not nonzero_current.empty:
            wedges, texts, autotexts = ax2.pie(
                nonzero_current['現在構成比'], labels=nonzero_current[plot_label_column], autopct='%1.1f%%',
                startangle=90, pctdistance=0.85, wedgeprops={'edgecolor': 'white', 'linewidth': 0.5},
                textprops={'fontsize': 7} )
            plt.setp(autotexts, size=6, weight="bold", color="white")
        ax2.set_title(f'現在売上構成比 (上位{len(nonzero_current)}項目)', fontsize=9)
        ax2.axis('equal')

        ax3 = fig.add_subplot(223)
        x_indices = np.arange(len(top)) 
        width = 0.35
        rects1 = ax3.bar(x_indices - width/2, top['過去売上金額'], width, label='過去売上', color='skyblue')
        rects2 = ax3.bar(x_indices + width/2, top['現在売上金額'], width, label='現在売上', color='lightcoral')
        ax3.set_ylabel('売上金額', fontsize=8)
        ax3.set_title(f'過去・現在 売上比較 (上位{top_n}項目)', fontsize=9)
        ax3.set_xticks(x_indices)
        ax3.set_xticklabels(top[plot_label_column], rotation=45, ha='right', fontsize=7) 
        ax3.legend(fontsize=7)
        ax3.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda val, loc: "{:,.0f}".format(val)))
        ax3.tick_params(axis='y', labelsize=7)

        def autolabel(rects, ax_target):
            for rect in rects:
                height = rect.get_height()
                ax_target.annotate('{:,.0f}'.format(height),
                                xy=(rect.get_x() + rect.get_width() / 2, height),
                                xytext=(0, 3), textcoords="offset points",
                                ha='center', va='bottom', fontsize=6)
        autolabel(rects1, ax3)
        autolabel(rects2, ax3)
        
        ax4 = fig.add_subplot(224)
        sorted_by_change_in_top = top.sort_values('増減額', ascending=True)
        plot_colors = ['red' if val < 0 else 'green' for val in sorted_by_change_in_top['増減額']]
        bars = ax4.barh(sorted_by_change_in_top[plot_label_column], sorted_by_change_in_top['増減額'], color=plot_colors, edgecolor='grey')
        ax4.set_xlabel('増減額', fontsize=8)
        ax4.set_title(f'売上増減額 (現在売上上位{top_n}項目内)', fontsize=9)
        ax4.get_xaxis().set_major_formatter(plt.FuncFormatter(lambda val, loc: "{:,.0f}".format(val)))
        ax4.tick_params(axis='y', labelsize=7)
        ax4.tick_params(axis='x', labelsize=7)

        for bar in bars:
            bar_width_val = bar.get_width()
            if bar_width_val == 0: continue
            ha_align = 'left' if bar_width_val > 0 else 'right'
            offset_val = 3 if bar_width_val > 0 else -3
            ax4.text(bar.get_width() + offset_val , bar.get_y() + bar.get_height()/2.,
                     '{:,.0f}'.format(bar_width_val), ha=ha_align, va='center', fontsize=6)

        canvas = FigureCanvasTkAgg(fig, master=self.graph_frame)
        canvas.draw()
        canvas.get_tk_widget().pack(fill="both", expand=True)
        toolbar = NavigationToolbar2Tk(canvas, self.graph_frame)
        toolbar.update()

    def export_to_excel(self):
        if self.merged_data is None:
            messagebox.showerror("エラー", "先に分析を実行してください")
            return
        path = self.output_file_path.get()
        if not path:
            path = filedialog.asksaveasfilename(title="保存先を選択", defaultextension=".xlsx", filetypes=[("Excel files","*.xlsx"),("All","*.*")])
            if not path: return
        self.output_file_path.set(path)

        try:
            wb = openpyxl.Workbook()
            ws = wb.active
            ws.title = "売上分析結果"
            export_key_column_name = self.product_code_column.get()
            ws.merge_cells('A1:G1')
            ws['A1'] = f"{export_key_column_name}別売上分析"
            ws['A1'].font = Font(size=14, bold=True, color="000080")
            ws['A1'].alignment = Alignment(horizontal='center')
            ws.merge_cells('A2:C2')
            ws['A2'] = f"作成日: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}"
            ws['A2'].font = Font(italic=True)
            headers = [export_key_column_name,"過去売上金額","過去構成比","現在売上金額","現在構成比","増減額","増減率"]
            for i, h in enumerate(headers, start=1):
                c = ws.cell(row=4, column=i, value=h)
                c.font = Font(bold=True, color="FFFFFF")
                c.alignment = Alignment(horizontal='center', vertical='center')
                c.fill = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")
            if export_key_column_name not in self.merged_data.columns:
                messagebox.showerror("エラー", f"Excelエクスポートエラー: 列 '{export_key_column_name}' がデータに存在しません。")
                return
            for idx, r_tuple in enumerate(self.merged_data.itertuples(index=False), start=5):
                key_value = getattr(r_tuple, export_key_column_name)
                ws.cell(row=idx, column=1, value=key_value)
                ws.cell(row=idx, column=2, value=r_tuple.過去売上金額)
                ws.cell(row=idx, column=3, value=r_tuple.過去構成比/100 if pd.notnull(r_tuple.過去構成比) else 0)
                ws.cell(row=idx, column=4, value=r_tuple.現在売上金額)
                ws.cell(row=idx, column=5, value=r_tuple.現在構成比/100 if pd.notnull(r_tuple.現在構成比) else 0)
                ws.cell(row=idx, column=6, value=r_tuple.増減額)
                ws.cell(row=idx, column=7, value=r_tuple.増減率/100 if pd.notnull(r_tuple.増減率) and np.isfinite(r_tuple.増減率) else 0)
                ws.cell(row=idx, column=1).alignment = Alignment(horizontal='left')
                for col_num in [2, 4, 6]:
                    ws.cell(row=idx, column=col_num).number_format = '#,##0'
                    ws.cell(row=idx, column=col_num).alignment = Alignment(horizontal='right')
                for col_num in [3, 5, 7]:
                    ws.cell(row=idx, column=col_num).number_format = '0.00%'
                    ws.cell(row=idx, column=col_num).alignment = Alignment(horizontal='right')
                if r_tuple.増減額 < 0:
                    ws.cell(row=idx, column=6).font = Font(color="FF0000")
                    ws.cell(row=idx, column=7).font = Font(color="FF0000")
                elif r_tuple.増減額 > 0 :
                    ws.cell(row=idx, column=6).font = Font(color="008000")
                    ws.cell(row=idx, column=7).font = Font(color="008000")
            for i, column_cells in enumerate(ws.columns):
                max_length = 0
                column_letter_val = get_column_letter(i + 1) 
                if ws.cell(row=4, column=i+1).value: max_length = max(max_length, len(str(ws.cell(row=4, column=i+1).value)))
                for cell in column_cells[4:]: 
                    try:
                        if cell.value is not None: 
                            cell_str_len = len(str(cell.value))
                            if isinstance(cell.value, (int, float)):
                                if cell.number_format and '%' in cell.number_format: cell_str_len = len(f"{cell.value:.2%}")
                                elif cell.number_format and ',' in cell.number_format: cell_str_len = len(f"{cell.value:,}")
                            max_length = max(max_length, cell_str_len)
                    except: pass 
                adjusted_width = (max_length + 2)
                ws.column_dimensions[column_letter_val].width = adjusted_width if adjusted_width < 50 else 50
            total_row = 5 + len(self.merged_data)
            ws.cell(row=total_row, column=1, value="合計").font = Font(bold=True)
            ws.cell(row=total_row, column=1).alignment = Alignment(horizontal='center')
            ws.cell(row=total_row, column=1).fill = PatternFill(start_color="DDDDDD", end_color="DDDDDD", fill_type="solid")
            for col_idx, key in [(2, '過去売上金額'), (4, '現在売上金額'), (6, '増減額')]:
                val = self.merged_data[key].sum()
                cell = ws.cell(row=total_row, column=col_idx, value=val)
                cell.font = Font(bold=True)
                cell.number_format = '#,##0'
                cell.alignment = Alignment(horizontal='right')
                cell.fill = PatternFill(start_color="DDDDDD", end_color="DDDDDD", fill_type="solid")
            chart_sheet = wb.create_sheet(title="グラフについて")
            chart_sheet['A1'] = "グラフの確認は、分析アプリ本体の「グラフ」タブで行ってください。"
            chart_sheet['A1'].font = Font(size=12, bold=True)
            chart_sheet.merge_cells('A1:E1')
            wb.save(path)
            messagebox.showinfo("完了", f"Excelファイルを保存しました: {path}")
        except PermissionError: messagebox.showerror("エラー", f"ファイル '{path}' への書き込み権限がありません。\nファイルが開かれていないか確認してください。")
        except Exception as e:
            import traceback
            traceback.print_exc()
            messagebox.showerror("エラー", f"Excelファイル出力中にエラーが発生しました: {e}")

    def clear_results(self):
        for i in self.tree.get_children(): self.tree.delete(i)
        for w in self.graph_frame.winfo_children(): w.destroy()
        ttk.Label(self.graph_frame, text="分析を実行するとここにグラフが表示されます。").pack(padx=10,pady=10)
        self.past_summary = None
        self.current_summary = None
        self.merged_data = None
        self.product_code_column.set(self.default_product_code_options[0])
        self.amount_column.set("金額")
        messagebox.showinfo("クリア", "分析結果をクリアしました")

def main():
    root = tk.Tk()
    app = SalesAnalysisApp(root)
    ttk.Label(app.graph_frame, text="分析を実行するとここにグラフが表示されます。").pack(padx=10,pady=10)
    root.mainloop()

if __name__ == "__main__":
    main()

過去ファイルを読み込み中: C:/Users/kkt040/Downloads/2025 04 売上照会.xlsx, シート: 2025 04 売上照会.
現在ファイルを読み込み中: C:/Users/kkt040/Downloads/売上照会.xlsx, シート: 売上照会.


In [4]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog, ttk, messagebox
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
import openpyxl
from openpyxl.styles import Font, Alignment, PatternFill
from openpyxl.utils import get_column_letter
import re
import numpy as np

class SalesAnalysisApp:
    def __init__(self, root):
        self.root = root
        self.root.title("売上照会分析アプリ")
        self.root.geometry("1200x800")

        self.past_file_path = tk.StringVar()
        self.current_file_path = tk.StringVar()
        self.output_file_path = tk.StringVar()

        self.default_product_code_options = ["旧ｺｰﾄﾞ連結", "中分類１", "取引先", "取引先名"]
        self.product_code_column = tk.StringVar(value=self.default_product_code_options[0])
        self.amount_column = tk.StringVar(value="金額")

        self.past_summary = None
        self.current_summary = None
        self.merged_data = None

        self.create_widgets()

    def create_widgets(self):
        main_frame = ttk.Frame(self.root)
        main_frame.pack(fill="both", expand=True, padx=10, pady=10)

        input_frame = ttk.LabelFrame(main_frame, text="ファイル選択")
        input_frame.pack(fill="x", pady=5)
        ttk.Label(input_frame, text="過去の売上照会ファイル:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        ttk.Entry(input_frame, textvariable=self.past_file_path, width=50).grid(row=0, column=1, padx=5, pady=5)
        ttk.Button(input_frame, text="参照", command=lambda: self.browse_file(self.past_file_path)).grid(row=0, column=2, padx=5)

        ttk.Label(input_frame, text="現在の売上照会ファイル:").grid(row=1, column=0, padx=5, pady=5, sticky="w")
        ttk.Entry(input_frame, textvariable=self.current_file_path, width=50).grid(row=1, column=1, padx=5, pady=5)
        ttk.Button(input_frame, text="参照", command=lambda: self.browse_file(self.current_file_path)).grid(row=1, column=2, padx=5)

        ttk.Label(input_frame, text="分析結果の出力先:").grid(row=2, column=0, padx=5, pady=5, sticky="w")
        ttk.Entry(input_frame, textvariable=self.output_file_path, width=50).grid(row=2, column=1, padx=5, pady=5)
        ttk.Button(input_frame, text="参照", command=self.browse_output_file).grid(row=2, column=2, padx=5)

        column_frame = ttk.LabelFrame(main_frame, text="列名設定")
        column_frame.pack(fill="x", pady=5)
        ttk.Label(column_frame, text="集計基準列名:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        self.product_code_combo_main = ttk.Combobox(column_frame, textvariable=self.product_code_column, width=18, values=self.default_product_code_options, state="readonly") # state="readonly" を追加しても良いかもしれません
        self.product_code_combo_main.grid(row=0, column=1, padx=5, pady=5)
        self.product_code_column.set(self.default_product_code_options[0])


        ttk.Label(column_frame, text="金額列名:").grid(row=0, column=2, padx=5, pady=5, sticky="w")
        # 金額列もComboboxにするか、Entryのままにするか検討の余地あり。今回はEntryのまま。
        self.amount_entry_main = ttk.Entry(column_frame, textvariable=self.amount_column, width=20)
        self.amount_entry_main.grid(row=0, column=3, padx=5, pady=5)

        ttk.Label(column_frame, text="※「分析実行」でシート/列名詳細設定").grid(row=1, column=0, columnspan=4, padx=5, pady=5)


        buttons_frame = ttk.Frame(main_frame)
        buttons_frame.pack(fill="x", pady=5)
        ttk.Button(buttons_frame, text="分析実行", command=self.analyze_sales, width=20).pack(side="left", padx=5)
        ttk.Button(buttons_frame, text="Excelに出力", command=self.export_to_excel, width=20).pack(side="left", padx=5)
        ttk.Button(buttons_frame, text="クリア", command=self.clear_results, width=20).pack(side="left", padx=5)

        tab_control = ttk.Notebook(main_frame)
        table_tab = ttk.Frame(tab_control)
        graph_tab = ttk.Frame(tab_control)
        tab_control.add(table_tab, text='テーブル')
        tab_control.add(graph_tab, text='グラフ')
        tab_control.pack(expand=1, fill="both")

        cols = ("集計キー", "過去売上金額", "過去構成比", "現在売上金額", "現在構成比", "増減額", "増減率")
        self.tree = ttk.Treeview(table_tab, columns=cols, show="headings")
        for c in cols:
            self.tree.heading(c, text=c)
            self.tree.column(c, width=180 if c=="集計キー" else 130, anchor="center")
        ysb = ttk.Scrollbar(table_tab, orient="vertical", command=self.tree.yview)
        xsb = ttk.Scrollbar(table_tab, orient="horizontal", command=self.tree.xview)
        self.tree.configure(yscrollcommand=ysb.set, xscrollcommand=xsb.set)
        self.tree.pack(side="left", fill="both", expand=True)
        ysb.pack(side="right", fill="y")
        xsb.pack(side="bottom", fill="x")

        self.graph_frame = graph_tab

    def browse_file(self, var):
        f = filedialog.askopenfilename(title="Excelファイルを選択", filetypes=[("Excel files","*.xlsx *.xls"),("All","*.*")])
        if f:
            var.set(f)
            # 過去ファイル選択時のみメイン画面のComboboxを更新（固定値にリセット）
            if var == self.past_file_path:
                self.update_main_product_code_combo() # 引数なしで呼び出し


    def update_main_product_code_combo(self):
        """メインウィンドウの集計基準列Comboboxの選択肢を固定値に設定し、
           現在の選択値が固定値にない場合はデフォルト値にリセットする。"""
        try:
            self.product_code_combo_main['values'] = self.default_product_code_options
            current_selection = self.product_code_column.get()
            if current_selection not in self.default_product_code_options or not current_selection:
                 self.product_code_column.set(self.default_product_code_options[0])
            # 既にStringVarが正しい値を持っていても、Comboboxの表示と確実に同期させるために再設定
            elif current_selection in self.default_product_code_options:
                 self.product_code_combo_main.set(current_selection)

        except Exception as e:
            print(f"メイン画面のCombobox更新エラー（固定値設定時）: {e}")
            self.product_code_combo_main['values'] = self.default_product_code_options
            if self.product_code_column.get() not in self.default_product_code_options:
                self.product_code_column.set(self.default_product_code_options[0])


    def browse_output_file(self):
        f = filedialog.asksaveasfilename(title="保存先を選択", defaultextension=".xlsx", filetypes=[("Excel files","*.xlsx"),("All","*.*")])
        if f:
            self.output_file_path.set(f)

    def get_excel_sheets(self, path):
        try:
            return pd.ExcelFile(path).sheet_names
        except Exception as e:
            messagebox.showerror("エラー", f"シート取得失敗: {e}")
            return []

    def normalize_column_name(self, name):
        if not isinstance(name, str):
            return str(name)
        name = name.replace('　', ' ').strip()
        return name

    def find_best_match_column(self, df, target_name):
        # target_nameがNoneや空文字列の場合、Noneを返す（エラーや意図しないマッチを防ぐ）
        if not target_name:
            return None
            
        # 1. target_name がそのままDataFrameの列に存在するか（大文字・小文字区別）
        if target_name in df.columns:
            return target_name

        # 2. target_name を正規化したものが、DataFrameの列を正規化したものと一致するか
        norm_target = self.normalize_column_name(target_name)
        for col in df.columns:
            if self.normalize_column_name(col) == norm_target:
                return col
        
        # 3. 固定選択肢の中に target_name があり、かつそれがDataFrameの列に存在するか
        # (これは主に、ユーザーが固定選択肢から選んだ場合、その名前でファイル内を探すため)
        if target_name in self.default_product_code_options:
            # 固定選択肢の名前でファイル内に列があるか (正規化なしで)
            if target_name in df.columns:
                return target_name
            # 固定選択肢の名前を正規化し、ファイル内の列を正規化したものと一致するか
            norm_default_opt = self.normalize_column_name(target_name)
            for col in df.columns:
                if self.normalize_column_name(col) == norm_default_opt:
                    return col
        
        # 4. 部分一致のロジック (最後の手段)
        best_match = None
        best_score = 0
        for col in df.columns:
            norm_col = self.normalize_column_name(col)
            # target_name が col に含まれる、またはその逆 (より厳密な部分一致を求めるなら調整)
            if norm_target in norm_col or norm_col in norm_target:
                match_len = len(set(norm_target) & set(norm_col)) # 共通文字の数
                # スコアリング: 完全一致に近いものを優先するため、長さの差も考慮できる
                # ここでは単純な共通文字数だが、より洗練された類似度スコアも検討可能
                # 例えば、Jaro-Winkler距離など
                if match_len > best_score:
                    best_score = match_len
                    best_match = col
        
        if best_match:
            print(f"部分一致: '{target_name}' の代わりに '{best_match}' (スコア: {best_score}) を使用します。")
        return best_match


    def show_sheet_selection_dialog(self, past_file, current_file):
        dialog = tk.Toplevel(self.root)
        dialog.title("シート選択と列設定")
        dialog.geometry("500x470")
        dialog.transient(self.root)
        dialog.grab_set()

        past_sheets = self.get_excel_sheets(past_file)
        current_sheets = self.get_excel_sheets(current_file)

        ttk.Label(dialog, text="過去ファイルのシート:").pack(anchor="w", padx=10, pady=5)
        past_sheet_var = tk.StringVar(value=past_sheets[0] if past_sheets else "")
        past_combo = ttk.Combobox(dialog, textvariable=past_sheet_var, values=past_sheets, state="readonly", width=30)
        past_combo.pack(anchor="w", padx=10)

        ttk.Label(dialog, text="現在ファイルのシート:").pack(anchor="w", padx=10, pady=5)
        current_sheet_var = tk.StringVar(value=current_sheets[0] if current_sheets else "")
        current_combo = ttk.Combobox(dialog, textvariable=current_sheet_var, values=current_sheets, state="readonly", width=30)
        current_combo.pack(anchor="w", padx=10)

        preview_frame = ttk.LabelFrame(dialog, text="列名設定とプレビュー")
        preview_frame.pack(fill="both", expand=True, padx=10, pady=10)
        
        # ダイアログ内の変数は、メイン画面のStringVarの現在の値で初期化
        product_code_var_dialog = tk.StringVar(value=self.product_code_column.get())
        amount_var_dialog = tk.StringVar(value=self.amount_column.get())

        ttk.Label(preview_frame, text="集計基準列:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        product_code_combo_dialog = ttk.Combobox(preview_frame, textvariable=product_code_var_dialog, width=30)
        product_code_combo_dialog.grid(row=0, column=1, padx=5, pady=5)

        ttk.Label(preview_frame, text="金額列:").grid(row=1, column=0, padx=5, pady=5, sticky="w")
        amount_combo_dialog = ttk.Combobox(preview_frame, textvariable=amount_var_dialog, width=30)
        amount_combo_dialog.grid(row=1, column=1, padx=5, pady=5)

        preview_text = tk.Text(preview_frame, height=8, width=45)
        preview_text.grid(row=3, column=0, columnspan=2, padx=5, pady=5)

        def preview_columns_and_set_defaults():
            try:
                selected_past_sheet = past_sheet_var.get()
                if not selected_past_sheet:
                    messagebox.showwarning("注意", "過去ファイルのシートを選択してください。")
                    return

                df = pd.read_excel(past_file, sheet_name=selected_past_sheet, nrows=5)
                file_cols = df.columns.tolist()
                
                # ダイアログの集計基準列の選択肢: 固定選択肢 + ファイルの列名 (重複除去、ソート)
                dialog_pc_options = sorted(list(set(self.default_product_code_options + file_cols)))
                product_code_combo_dialog['values'] = dialog_pc_options
                
                # 金額列の選択肢はファイル内の列のみ
                amount_combo_dialog['values'] = file_cols

                # --- 集計基準列のデフォルト値設定 ---
                current_pc_selection_in_dialog = product_code_var_dialog.get()
                # 1. 現在ダイアログで選択されている値が、新しい選択肢にあればそれを維持
                if current_pc_selection_in_dialog in dialog_pc_options:
                    product_code_var_dialog.set(current_pc_selection_in_dialog)
                else:
                    # 2. メイン画面で選択されている値が、ファイル内にあればそれを優先
                    main_pc_selection = self.product_code_column.get()
                    if main_pc_selection in file_cols and main_pc_selection in dialog_pc_options:
                        product_code_var_dialog.set(main_pc_selection)
                    else:
                        # 3. 固定選択肢の中でファイル内に存在するものを優先
                        found_default_in_file = False
                        for opt in self.default_product_code_options:
                            if opt in file_cols and opt in dialog_pc_options: # ファイル内にあり、かつダイアログの選択肢にもある
                                product_code_var_dialog.set(opt)
                                found_default_in_file = True
                                break
                        if not found_default_in_file and dialog_pc_options:
                            # 4. それ以外ならダイアログの選択肢の最初のもの
                            product_code_var_dialog.set(dialog_pc_options[0])
                        elif not found_default_in_file and not dialog_pc_options : # 選択肢が空の場合
                             product_code_var_dialog.set("")


                # --- 金額列のデフォルト値設定 ---
                current_amount_selection_in_dialog = amount_var_dialog.get()
                # 1. 現在ダイアログで選択されている値が、ファイル列にあればそれを維持
                if current_amount_selection_in_dialog in file_cols:
                    amount_var_dialog.set(current_amount_selection_in_dialog)
                else:
                    # 2. メイン画面で設定されている金額列名を元に、ファイル内で最適なものを探す
                    target_amount_col_for_find = self.amount_column.get() # メイン画面の値
                    best_amount_match = self.find_best_match_column(df, target_amount_col_for_find)
                    if best_amount_match and best_amount_match in file_cols:
                        amount_var_dialog.set(best_amount_match)
                    elif file_cols: # マッチがなくてもファイルに列があれば最初の列
                        amount_var_dialog.set(file_cols[0])
                    else: # ファイルに列がなければ空
                        amount_var_dialog.set("")


                preview_text.delete("1.0", tk.END)
                preview_text.insert(tk.END, "使用可能な列一覧 (過去ファイルより):\n" + "\n".join(f"{i+1}. {c}" for i, c in enumerate(file_cols)))

                if len(df) > 0:
                    preview_text.insert(tk.END, "\n\n最初の行のデータ例:\n")
                    for col_idx, col_name in enumerate(file_cols[:5]):
                        val = df.iloc[0, col_idx] # df.iloc[0][col_name] より効率的
                        preview_text.insert(tk.END, f"{col_name}: {val}\n")

            except Exception as e:
                messagebox.showerror("エラー", f"列プレビュー失敗: {e}")
                import traceback
                traceback.print_exc()


        past_combo.bind("<<ComboboxSelected>>", lambda event: preview_columns_and_set_defaults())
        dialog.after(100, preview_columns_and_set_defaults) # Toplevel描画後に実行
        ttk.Button(preview_frame, text="列名プレビュー更新", command=preview_columns_and_set_defaults).grid(row=2, column=0, columnspan=2, pady=5)

        def on_ok():
            if not past_sheet_var.get() or not current_sheet_var.get():
                messagebox.showerror("エラー", "過去と現在の両方のシートを選択してください。")
                return
            # ダイアログで選択された値を取得
            selected_pc_in_dialog = product_code_var_dialog.get()
            selected_amount_in_dialog = amount_var_dialog.get()

            if not selected_pc_in_dialog or not selected_amount_in_dialog:
                messagebox.showerror("エラー", "集計基準列と金額列を選択または入力してください。")
                return

            self.past_sheet = past_sheet_var.get()
            self.current_sheet = current_sheet_var.get()
            # メインのStringVarとメイン画面のComboboxにダイアログでの選択を反映
            self.product_code_column.set(selected_pc_in_dialog)
            self.amount_column.set(selected_amount_in_dialog) # 金額もダイアログでの選択を反映
            
            self.product_code_combo_main.set(selected_pc_in_dialog)
            self.amount_entry_main.delete(0, tk.END) # 金額Entryの場合
            self.amount_entry_main.insert(0, selected_amount_in_dialog)


            dialog.destroy()
            self.perform_analysis()

        ttk.Button(dialog, text="OK", command=on_ok).pack(pady=10)
        self.root.wait_window(dialog)


    def analyze_sales(self):
        if not self.past_file_path.get() or not self.current_file_path.get():
            messagebox.showerror("エラー", "過去と現在の両方のファイルを選択してください")
            return
        # 分析実行前にメイン画面の選択値をproduct_code_columnとamount_columnに確実にセット
        self.product_code_column.set(self.product_code_combo_main.get())
        self.amount_column.set(self.amount_entry_main.get()) # 金額もEntryから取得
        self.show_sheet_selection_dialog(self.past_file_path.get(), self.current_file_path.get())

    def perform_analysis(self):
        try:
            # ユーザーがUIで最終的に選択/入力した列名
            pc_user_choice = self.product_code_column.get()
            amt_user_choice = self.amount_column.get()

            if not pc_user_choice or not amt_user_choice:
                messagebox.showerror("エラー", "集計基準列または金額列が指定されていません。")
                return

            print(f"過去ファイルを読み込み中: {self.past_file_path.get()}, シート: {self.past_sheet}")
            past_df = pd.read_excel(self.past_file_path.get(), sheet_name=self.past_sheet)
            print(f"過去ファイルの列名: {past_df.columns.tolist()}")

            print(f"現在ファイルを読み込み中: {self.current_file_path.get()}, シート: {self.current_sheet}")
            curr_df = pd.read_excel(self.current_file_path.get(), sheet_name=self.current_sheet)
            print(f"現在ファイルの列名: {curr_df.columns.tolist()}")

            print(f"ユーザー指定の集計基準列: {pc_user_choice}")
            print(f"ユーザー指定の金額列: {amt_user_choice}")

            # 実際にDataFrame内に存在する列名を探す
            pc_past_actual = self.find_best_match_column(past_df, pc_user_choice)
            if not pc_past_actual: raise ValueError(f"集計基準列 '{pc_user_choice}' が過去ファイル({self.past_sheet})で見つかりません。利用可能な列: {past_df.columns.tolist()}")
            if pc_past_actual != pc_user_choice: messagebox.showinfo("列名修正(過去/基準)", f"'{pc_user_choice}'の代わりに'{pc_past_actual}'を使用")

            amt_past_actual = self.find_best_match_column(past_df, amt_user_choice)
            if not amt_past_actual: raise ValueError(f"金額列 '{amt_user_choice}' が過去ファイル({self.past_sheet})で見つかりません。利用可能な列: {past_df.columns.tolist()}")
            if amt_past_actual != amt_user_choice: messagebox.showinfo("列名修正(過去/金額)", f"'{amt_user_choice}'の代わりに'{amt_past_actual}'を使用")

            pc_curr_actual = self.find_best_match_column(curr_df, pc_user_choice)
            if not pc_curr_actual: raise ValueError(f"集計基準列 '{pc_user_choice}' が現在ファイル({self.current_sheet})で見つかりません。利用可能な列: {curr_df.columns.tolist()}")
            if pc_curr_actual != pc_user_choice: messagebox.showinfo("列名修正(現在/基準)", f"'{pc_user_choice}'の代わりに'{pc_curr_actual}'を使用")

            amt_curr_actual = self.find_best_match_column(curr_df, amt_user_choice)
            if not amt_curr_actual: raise ValueError(f"金額列 '{amt_user_choice}' が現在ファイル({self.current_sheet})で見つかりません。利用可能な列: {curr_df.columns.tolist()}")
            if amt_curr_actual != amt_user_choice: messagebox.showinfo("列名修正(現在/金額)", f"'{amt_user_choice}'の代わりに'{amt_curr_actual}'を使用")


            for df, col_actual, df_name in [(past_df, amt_past_actual, "過去"), (curr_df, amt_curr_actual, "現在")]:
                # 空のDataFrameや列の場合のチェックを追加
                if col_actual not in df.columns or df[col_actual].empty:
                    print(f"{df_name}ファイルの金額列 '{col_actual}' が存在しないか空です。0で埋めます。")
                    df[col_actual] = 0 # 列が存在しない場合は作成して0で埋める (またはエラー処理)
                elif df[col_actual].dtype == object or isinstance(df[col_actual].iloc[0], str) :
                    print(f"{df_name}ファイルの金額列 '{col_actual}' は文字列型です。数値に変換します。")
                    df[col_actual] = df[col_actual].astype(str).str.replace(',', '', regex=False).str.replace(' ', '', regex=False)
                    df[col_actual] = df[col_actual].str.replace(r'[¥円]', '', regex=True)
                    df[col_actual] = pd.to_numeric(df[col_actual], errors='coerce').fillna(0)
                elif not pd.api.types.is_numeric_dtype(df[col_actual]):
                    print(f"{df_name}ファイルの金額列 '{col_actual}' は数値型ではありません。数値に変換を試みます。")
                    df[col_actual] = pd.to_numeric(df[col_actual], errors='coerce').fillna(0)
                else:
                    df[col_actual] = df[col_actual].fillna(0)
            
            # 集計は実際の列名で行い、結果の列名はユーザーが選択した名前にする
            self.past_summary = past_df.groupby(pc_past_actual, dropna=False)[amt_past_actual].sum().reset_index() # dropna=False を追加
            self.past_summary.rename(columns={pc_past_actual: pc_user_choice, amt_past_actual: amt_user_choice}, inplace=True)
            if not self.past_summary[amt_user_choice].empty and self.past_summary[amt_user_choice].sum() != 0:
                self.past_summary['構成比'] = (self.past_summary[amt_user_choice] / self.past_summary[amt_user_choice].sum() * 100)
            else:
                self.past_summary['構成比'] = 0
            self.past_summary['構成比'] = self.past_summary['構成比'].fillna(0)


            self.current_summary = curr_df.groupby(pc_curr_actual, dropna=False)[amt_curr_actual].sum().reset_index() # dropna=False を追加
            self.current_summary.rename(columns={pc_curr_actual: pc_user_choice, amt_curr_actual: amt_user_choice}, inplace=True)
            if not self.current_summary[amt_user_choice].empty and self.current_summary[amt_user_choice].sum() != 0:
                self.current_summary['構成比'] = (self.current_summary[amt_user_choice] / self.current_summary[amt_user_choice].sum() * 100)
            else:
                self.current_summary['構成比'] = 0
            self.current_summary['構成比'] = self.current_summary['構成比'].fillna(0)
            
            self.prepare_merged_data()
            self.display_results()
            self.plot_results()
            messagebox.showinfo("完了", "分析が完了しました")
        except Exception as e:
            import traceback
            traceback.print_exc()
            messagebox.showerror("エラー", f"分析中にエラー: {e}")

    def prepare_merged_data(self):
        # マージに使用するキー名は、ユーザーがUIで選択/入力したもので、
        # perform_analysisでサマリーの列名がこれにリネームされているはず。
        pc_key_for_merge = self.product_code_column.get()
        amt_key_for_merge = self.amount_column.get() # 金額列名も同様

        past_temp = self.past_summary.copy()
        current_temp = self.current_summary.copy()
        
        if pc_key_for_merge not in past_temp.columns:
             raise ValueError(f"マージ用データ(過去)に列 '{pc_key_for_merge}' がありません。列: {past_temp.columns}")
        if pc_key_for_merge not in current_temp.columns:
             raise ValueError(f"マージ用データ(現在)に列 '{pc_key_for_merge}' がありません。列: {current_temp.columns}")

        # 結合キーのデータ型を文字列に統一 (NaNや欠損値も 'nan' のような文字列になる)
        past_temp[pc_key_for_merge] = past_temp[pc_key_for_merge].astype(str)
        current_temp[pc_key_for_merge] = current_temp[pc_key_for_merge].astype(str)
        
        m = pd.merge(past_temp, current_temp, on=pc_key_for_merge, how='outer').fillna(0)
        
        # suffixes は merge 時に amt_key_for_merge 列が両方のDFにある場合に自動で付与される
        # ここでは、rename で明確に指定する
        m.rename(columns={
            f'{amt_key_for_merge}_x': '過去売上金額', # _x が過去側, _y が現在側と仮定 (suffixes指定なしの場合)
            f'{amt_key_for_merge}_y': '現在売上金額',
            f'{amt_key_for_merge}': '現在売上金額', #片方にしかない場合
            '構成比_x': '過去構成比',
            '構成比_y': '現在構成比',
        }, inplace=True)

        # もし suffixes が付与されなかった場合（列名が衝突しなかった場合）の対応
        if amt_key_for_merge in m.columns and '過去売上金額' not in m.columns and past_temp[amt_key_for_merge].sum() !=0 :
             # このケースは通常 merge(suffixes=...) で対応するが、手動で対応する場合
             # 過去データのみ存在し、現在データに金額列がなかった場合など
             # このロジックは複雑なので、merge時にsuffixesを指定する方が堅牢
             # 今回はsuffixesなしで進めて、問題あれば修正
             pass #一旦そのまま

        # '過去売上金額' や '現在売上金額' が存在しない場合のフォールバック
        if '過去売上金額' not in m.columns and f'{amt_key_for_merge}_x' in m.columns: m.rename(columns={f'{amt_key_for_merge}_x': '過去売上金額'}, inplace=True)
        if '現在売上金額' not in m.columns and f'{amt_key_for_merge}_y' in m.columns: m.rename(columns={f'{amt_key_for_merge}_y': '現在売上金額'}, inplace=True)
        if '過去構成比' not in m.columns and '構成比_x' in m.columns: m.rename(columns={'構成比_x': '過去構成比'}, inplace=True)
        if '現在構成比' not in m.columns and '構成比_y' in m.columns: m.rename(columns={'構成比_y': '現在構成比'}, inplace=True)

        # fillna(0) は merge 後に再度適用した方が確実
        final_cols_to_fill = ['過去売上金額', '現在売上金額', '過去構成比', '現在構成比']
        for col in final_cols_to_fill:
            if col not in m.columns:
                m[col] = 0 # 列がなければ作成して0で埋める
            else:
                m[col] = m[col].fillna(0)


        m['増減額'] = m['現在売上金額'] - m['過去売上金額']
        m['増減率'] = m.apply(lambda r: (r['増減額']/r['過去売上金額']*100) if r['過去売上金額']!=0 else (np.inf if r['増減額'] > 0 else (-np.inf if r['増減額'] < 0 else 0)), axis=1)
        
        self.merged_data = m.sort_values('現在売上金額', ascending=False)
        
        if pc_key_for_merge not in self.merged_data.columns:
             messagebox.showwarning("警告", f"最終データに集計キー '{pc_key_for_merge}' が見つかりません。利用可能な列: {self.merged_data.columns}")


    def display_results(self):
        # (変更なし)
        for i in self.tree.get_children():
            self.tree.delete(i)
        
        display_key_column = self.product_code_column.get()
        self.tree.heading("集計キー", text=display_key_column) 

        if self.merged_data is None or display_key_column not in self.merged_data.columns:
            # ユーザーにエラーを通知する前にコンソールにも出力
            print(f"表示エラー: merged_data is None: {self.merged_data is None}")
            if self.merged_data is not None:
                print(f"表示エラー: 列 '{display_key_column}' not in merged_data.columns: {self.merged_data.columns.tolist()}")
            messagebox.showerror("エラー", f"表示するデータ、または列 '{display_key_column}' が見つかりません。")
            return

        for _, r in self.merged_data.iterrows():
            self.tree.insert('', 'end', values=(
                r[display_key_column],
                f"{int(r['過去売上金額']):,}",
                f"{r['過去構成比']:.2f}%",
                f"{int(r['現在売上金額']):,}",
                f"{r['現在構成比']:.2f}%",
                f"{int(r['増減額']):,}",
                f"{r['増減率']:.2f}%" if pd.notnull(r['増減率']) and np.isfinite(r['増減率']) else ("-" if r['増減率']==0 else ("増加" if r['増減率']==np.inf else "減少"))
            ))


    def plot_results(self):
        # (変更なし、ただし変数名の微修正は前のステップで実施済み)
        for w in self.graph_frame.winfo_children():
            w.destroy()

        if self.merged_data is None or self.merged_data.empty:
            ttk.Label(self.graph_frame, text="表示するデータがありません。").pack(padx=10, pady=10)
            return

        top_n = 10
        top = self.merged_data.head(top_n)
        
        plot_label_column = self.product_code_column.get()
        if plot_label_column not in top.columns:
             messagebox.showerror("エラー", f"グラフ用データに列 '{plot_label_column}' が見つかりません。利用可能な列: {top.columns}")
             return

        try:
            plt.rcParams['font.family'] = 'Yu Gothic'
            if not any(f.name == 'Yu Gothic' for f in plt.matplotlib.font_manager.fontManager.ttflist):
                 plt.rcParams['font.family'] = 'MS Gothic'
        except:
             plt.rcParams['font.family'] = 'sans-serif'
        plt.rcParams['axes.unicode_minus'] = False

        fig = plt.Figure(figsize=(14, 12), dpi=100)
        fig.subplots_adjust(hspace=0.5, wspace=0.3)

        ax1 = fig.add_subplot(221)
        nonzero_past = top[top['過去構成比'] > 0.01]
        if not nonzero_past.empty:
            wedges, texts, autotexts = ax1.pie(
                nonzero_past['過去構成比'],
                labels=nonzero_past[plot_label_column],
                autopct='%1.1f%%',
                startangle=90,
                pctdistance=0.85,
                wedgeprops={'edgecolor': 'white', 'linewidth': 0.5},
                textprops={'fontsize': 8}
            )
            plt.setp(autotexts, size=7, weight="bold", color="white")
        ax1.set_title(f'過去売上構成比 (上位{len(nonzero_past)}項目)', fontsize=10)
        ax1.axis('equal')

        ax2 = fig.add_subplot(222)
        nonzero_current = top[top['現在構成比'] > 0.01]
        if not nonzero_current.empty:
            wedges, texts, autotexts = ax2.pie(
                nonzero_current['現在構成比'],
                labels=nonzero_current[plot_label_column],
                autopct='%1.1f%%',
                startangle=90,
                pctdistance=0.85,
                wedgeprops={'edgecolor': 'white', 'linewidth': 0.5},
                textprops={'fontsize': 8}
            )
            plt.setp(autotexts, size=7, weight="bold", color="white")
        ax2.set_title(f'現在売上構成比 (上位{len(nonzero_current)}項目)', fontsize=10)
        ax2.axis('equal')

        ax3 = fig.add_subplot(223)
        x_indices = np.arange(len(top)) 
        width = 0.35
        rects1 = ax3.bar(x_indices - width/2, top['過去売上金額'], width, label='過去売上', color='skyblue')
        rects2 = ax3.bar(x_indices + width/2, top['現在売上金額'], width, label='現在売上', color='lightcoral')

        ax3.set_ylabel('売上金額', fontsize=9)
        ax3.set_title(f'過去・現在 売上比較 (上位{top_n}項目)', fontsize=10)
        ax3.set_xticks(x_indices)
        ax3.set_xticklabels(top[plot_label_column], rotation=45, ha='right', fontsize=8)
        ax3.legend(fontsize=8)
        ax3.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda val, loc: "{:,.0f}".format(val)))

        def autolabel(rects, ax_target):
            for rect in rects:
                height = rect.get_height()
                ax_target.annotate('{:,.0f}'.format(height),
                                xy=(rect.get_x() + rect.get_width() / 2, height),
                                xytext=(0, 3),
                                textcoords="offset points",
                                ha='center', va='bottom', fontsize=7)
        autolabel(rects1, ax3)
        autolabel(rects2, ax3)
        ax3.tick_params(axis='y', labelsize=8)

        ax4 = fig.add_subplot(224)
        sorted_by_change = top.sort_values('増減額', ascending=False)
        plot_colors = ['green' if val > 0 else 'red' for val in sorted_by_change['増減額']]
        bars = ax4.barh(sorted_by_change[plot_label_column], sorted_by_change['増減額'], color=plot_colors, edgecolor='grey')
        ax4.set_xlabel('増減額', fontsize=9)
        ax4.set_title(f'売上増減額 (上位{top_n}項目の現在売上ベース)', fontsize=10)
        ax4.invert_yaxis()
        ax4.get_xaxis().set_major_formatter(plt.FuncFormatter(lambda val, loc: "{:,.0f}".format(val)))
        ax4.tick_params(axis='y', labelsize=8)
        ax4.tick_params(axis='x', labelsize=8)

        for bar in bars:
            bar_width_val = bar.get_width()
            if bar_width_val > 0:
                ha_align = 'left'
                offset_val = 5
            else:
                ha_align = 'right'
                offset_val = -5

            ax4.text(bar.get_width() + offset_val, bar.get_y() + bar.get_height()/2.,
                     '{:,.0f}'.format(bar_width_val),
                     ha=ha_align, va='center', fontsize=7)

        fig.tight_layout(pad=2.0)

        canvas = FigureCanvasTkAgg(fig, master=self.graph_frame)
        canvas.draw()
        canvas.get_tk_widget().pack(fill="both", expand=True)
        toolbar = NavigationToolbar2Tk(canvas, self.graph_frame)
        toolbar.update()


    def export_to_excel(self):
        # (変更なし、ただし変数名の微修正は前のステップで実施済み)
        if self.merged_data is None:
            messagebox.showerror("エラー", "先に分析を実行してください")
            return

        path = self.output_file_path.get()
        if not path:
            path = filedialog.asksaveasfilename(
                title="保存先を選択", defaultextension=".xlsx", filetypes=[("Excel files","*.xlsx"),("All","*.*")])
            if not path:
                return
        self.output_file_path.set(path)

        try:
            wb = openpyxl.Workbook()
            ws = wb.active
            ws.title = "売上分析結果"

            export_key_column_name = self.product_code_column.get()

            ws.merge_cells('A1:G1')
            ws['A1'] = f"{export_key_column_name}別売上分析"
            ws['A1'].font = Font(size=14, bold=True, color="000080")
            ws['A1'].alignment = Alignment(horizontal='center')
            ws.merge_cells('A2:C2')
            ws['A2'] = f"作成日: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}"
            ws['A2'].font = Font(italic=True)

            headers = [export_key_column_name,"過去売上金額","過去構成比","現在売上金額","現在構成比","増減額","増減率"]
            for i, h in enumerate(headers, start=1):
                c = ws.cell(row=4, column=i, value=h)
                c.font = Font(bold=True, color="FFFFFF")
                c.alignment = Alignment(horizontal='center', vertical='center')
                c.fill = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")
            
            if export_key_column_name not in self.merged_data.columns:
                messagebox.showerror("エラー", f"Excelエクスポートエラー: 列 '{export_key_column_name}' がデータに存在しません。")
                return


            for idx, r_tuple in enumerate(self.merged_data.itertuples(index=False), start=5):
                key_value = getattr(r_tuple, export_key_column_name)
                ws.cell(row=idx, column=1, value=key_value)
                ws.cell(row=idx, column=2, value=r_tuple.過去売上金額)
                ws.cell(row=idx, column=3, value=r_tuple.過去構成比/100 if pd.notnull(r_tuple.過去構成比) else 0)
                ws.cell(row=idx, column=4, value=r_tuple.現在売上金額)
                ws.cell(row=idx, column=5, value=r_tuple.現在構成比/100 if pd.notnull(r_tuple.現在構成比) else 0)
                ws.cell(row=idx, column=6, value=r_tuple.増減額)
                ws.cell(row=idx, column=7, value=r_tuple.増減率/100 if pd.notnull(r_tuple.増減率) and np.isfinite(r_tuple.増減率) else 0)

                ws.cell(row=idx, column=1).alignment = Alignment(horizontal='left')
                for col_num in [2, 4, 6]:
                    ws.cell(row=idx, column=col_num).number_format = '#,##0'
                    ws.cell(row=idx, column=col_num).alignment = Alignment(horizontal='right')
                for col_num in [3, 5, 7]:
                    ws.cell(row=idx, column=col_num).number_format = '0.00%'
                    ws.cell(row=idx, column=col_num).alignment = Alignment(horizontal='right')

                if r_tuple.増減額 < 0:
                    ws.cell(row=idx, column=6).font = Font(color="FF0000")
                    ws.cell(row=idx, column=7).font = Font(color="FF0000")
                elif r_tuple.増減額 > 0 :
                    ws.cell(row=idx, column=6).font = Font(color="008000")
                    ws.cell(row=idx, column=7).font = Font(color="008000")

            for i, column_cells in enumerate(ws.columns):
                max_length = 0
                column_letter_val = get_column_letter(i + 1) 
                if ws.cell(row=4, column=i+1).value:
                    max_length = max(max_length, len(str(ws.cell(row=4, column=i+1).value)))

                for cell in column_cells[4:]: 
                    try:
                        if cell.value is not None: 
                            cell_str_len = len(str(cell.value))
                            if isinstance(cell.value, (int, float)):
                                if cell.number_format and '%' in cell.number_format:
                                    cell_str_len = len(f"{cell.value:.2%}")
                                elif cell.number_format and ',' in cell.number_format:
                                    cell_str_len = len(f"{cell.value:,}")
                            max_length = max(max_length, cell_str_len)
                    except:
                        pass 
                adjusted_width = (max_length + 2)
                ws.column_dimensions[column_letter_val].width = adjusted_width if adjusted_width < 50 else 50


            total_row = 5 + len(self.merged_data)
            ws.cell(row=total_row, column=1, value="合計").font = Font(bold=True)
            ws.cell(row=total_row, column=1).alignment = Alignment(horizontal='center')
            ws.cell(row=total_row, column=1).fill = PatternFill(start_color="DDDDDD", end_color="DDDDDD", fill_type="solid")

            for col_idx, key in [(2, '過去売上金額'), (4, '現在売上金額'), (6, '増減額')]:
                val = self.merged_data[key].sum()
                cell = ws.cell(row=total_row, column=col_idx, value=val)
                cell.font = Font(bold=True)
                cell.number_format = '#,##0'
                cell.alignment = Alignment(horizontal='right')
                cell.fill = PatternFill(start_color="DDDDDD", end_color="DDDDDD", fill_type="solid")

            chart_sheet = wb.create_sheet(title="グラフについて")
            chart_sheet['A1'] = "グラフの確認は、分析アプリ本体の「グラフ」タブで行ってください。"
            chart_sheet['A1'].font = Font(size=12, bold=True)
            chart_sheet.merge_cells('A1:E1')

            wb.save(path)
            messagebox.showinfo("完了", f"Excelファイルを保存しました: {path}")
        except PermissionError:
            messagebox.showerror("エラー", f"ファイル '{path}' への書き込み権限がありません。\nファイルが開かれていないか確認してください。")
        except Exception as e:
            import traceback
            traceback.print_exc()
            messagebox.showerror("エラー", f"Excelファイル出力中にエラーが発生しました: {e}")


    def clear_results(self):
        for i in self.tree.get_children():
            self.tree.delete(i)
        for w in self.graph_frame.winfo_children():
            w.destroy()
        ttk.Label(self.graph_frame, text="分析を実行するとここにグラフが表示されます。").pack(padx=10,pady=10)
        self.past_summary = None
        self.current_summary = None
        self.merged_data = None
        # メイン画面のComboboxとEntryの値を初期値に戻すか検討
        self.product_code_column.set(self.default_product_code_options[0])
        self.amount_column.set("金額") # 金額も初期値に戻す
        messagebox.showinfo("クリア", "分析結果をクリアしました")


def main():
    root = tk.Tk()
    app = SalesAnalysisApp(root)
    ttk.Label(app.graph_frame, text="分析を実行するとここにグラフが表示されます。").pack(padx=10,pady=10)
    root.mainloop()

if __name__ == "__main__":
    main()

過去ファイルを読み込み中: C:/Users/kkt040/Downloads/2025 04 売上照会.xlsx, シート: 2025 04 売上照会.
過去ファイルの列名: ['取引先', '金額', '売上数', '原価', '手配区分', '単価', '受注者', '配送便名', '取引先名', '納入先名', '旧ｺｰﾄﾞ連結', '商品名', '売上№', 'サイズ', '仕入先名', '直送仕入先名', '中分類１', '売上日', '粗利率', '仕入先コード', '直送仕入先', '来勘区分名', '商品', '取引区分', '売上行№', '来勘区分', '受注区分', '摘要名', '取引形態', '客先発注№', '納入先', '事業部', '売上数(バラ)', '事業部名', '部', '部名', 'グループ', 'グループ名', '担当者', '担当者名', '倉庫', '倉庫名', '受注№', '受注行№', '受注日', '受注数', '売上数(ケース)', 'ケース入数', '売上数（小箱）', '小箱入数', '単位', '納品書メモ', '仮単価区分', '赤黒', '税抜金額', '消費税', '税区分', '摘要', '配送便', '前回訂正', '一連№', '受注者.1', 'ﾋﾟｯｷﾝｸﾞﾒﾓ', '住所１', '住所２', '住所３']
現在ファイルを読み込み中: C:/Users/kkt040/Downloads/売上照会.xlsx, シート: 売上照会.
現在ファイルの列名: ['取引先', '金額', '利益総額', '原価総額', '売上数', '原価', '手配区分', '単価', '受注者', '配送便名', '取引先名', '納入先名', '旧ｺｰﾄﾞ連結', '商品名', '売上№', 'サイズ', '仕入先名', '直送仕入先名', '中分類１', '売上日', '粗利率', '仕入先コード', '直送仕入先', '来勘区分名', '商品', '取引区分', '売上行№', '来勘区分', '受注区分', '摘要名', '取引形態', '客先発注№', '納入先', '事業部', '売上数(バラ)', '事業部名', '部', '部名', 'グループ', 'グループ名', '担当者', '担当者名

In [1]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog, ttk, messagebox
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
import openpyxl
from openpyxl.styles import Font, Alignment, PatternFill
from openpyxl.utils import get_column_letter
import re
import numpy as np

class SalesAnalysisApp:
    def __init__(self, root):
        self.root = root
        self.root.title("売上照会分析アプリ")
        self.root.geometry("1200x800")

        self.past_file_path = tk.StringVar()
        self.current_file_path = tk.StringVar()
        self.output_file_path = tk.StringVar()

        self.product_code_column = tk.StringVar(value="旧ｺｰﾄﾞ連結") # 初期値
        self.amount_column = tk.StringVar(value="金額")

        self.past_summary = None
        self.current_summary = None
        self.merged_data = None

        self.create_widgets()

    def create_widgets(self):
        main_frame = ttk.Frame(self.root)
        main_frame.pack(fill="both", expand=True, padx=10, pady=10)

        input_frame = ttk.LabelFrame(main_frame, text="ファイル選択")
        input_frame.pack(fill="x", pady=5)
        ttk.Label(input_frame, text="過去の売上照会ファイル:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        ttk.Entry(input_frame, textvariable=self.past_file_path, width=50).grid(row=0, column=1, padx=5, pady=5)
        ttk.Button(input_frame, text="参照", command=lambda: self.browse_file(self.past_file_path)).grid(row=0, column=2, padx=5)

        ttk.Label(input_frame, text="現在の売上照会ファイル:").grid(row=1, column=0, padx=5, pady=5, sticky="w")
        ttk.Entry(input_frame, textvariable=self.current_file_path, width=50).grid(row=1, column=1, padx=5, pady=5)
        ttk.Button(input_frame, text="参照", command=lambda: self.browse_file(self.current_file_path)).grid(row=1, column=2, padx=5)

        ttk.Label(input_frame, text="分析結果の出力先:").grid(row=2, column=0, padx=5, pady=5, sticky="w")
        ttk.Entry(input_frame, textvariable=self.output_file_path, width=50).grid(row=2, column=1, padx=5, pady=5)
        ttk.Button(input_frame, text="参照", command=self.browse_output_file).grid(row=2, column=2, padx=5)

        column_frame = ttk.LabelFrame(main_frame, text="列名設定")
        column_frame.pack(fill="x", pady=5)
        ttk.Label(column_frame, text="集計基準列名:").grid(row=0, column=0, padx=5, pady=5, sticky="w") # ラベル変更
        # 商品コード列名をComboboxに変更
        self.product_code_combo_main = ttk.Combobox(column_frame, textvariable=self.product_code_column, width=18, values=["旧ｺｰﾄﾞ連結", "中分類１"])
        self.product_code_combo_main.grid(row=0, column=1, padx=5, pady=5)
        self.product_code_column.set("旧ｺｰﾄﾞ連結") # 初期値を設定

        ttk.Label(column_frame, text="金額列名:").grid(row=0, column=2, padx=5, pady=5, sticky="w")
        ttk.Entry(column_frame, textvariable=self.amount_column, width=20).grid(row=0, column=3, padx=5, pady=5)
        ttk.Label(column_frame, text="※ファイル選択後に「分析実行」を押すとシート選択と列名詳細設定が可能").grid(row=1, column=0, columnspan=4, padx=5, pady=5)


        buttons_frame = ttk.Frame(main_frame)
        buttons_frame.pack(fill="x", pady=5)
        ttk.Button(buttons_frame, text="分析実行", command=self.analyze_sales, width=20).pack(side="left", padx=5)
        ttk.Button(buttons_frame, text="Excelに出力", command=self.export_to_excel, width=20).pack(side="left", padx=5)
        ttk.Button(buttons_frame, text="クリア", command=self.clear_results, width=20).pack(side="left", padx=5)

        tab_control = ttk.Notebook(main_frame)
        table_tab = ttk.Frame(tab_control)
        graph_tab = ttk.Frame(tab_control)
        tab_control.add(table_tab, text='テーブル')
        tab_control.add(graph_tab, text='グラフ')
        tab_control.pack(expand=1, fill="both")

        cols = ("集計キー", "過去売上金額", "過去構成比", "現在売上金額", "現在構成比", "増減額", "増減率") # Treeviewの列名変更
        self.tree = ttk.Treeview(table_tab, columns=cols, show="headings")
        for c in cols:
            self.tree.heading(c, text=c)
            self.tree.column(c, width=180 if c=="集計キー" else 130, anchor="center")
        ysb = ttk.Scrollbar(table_tab, orient="vertical", command=self.tree.yview)
        xsb = ttk.Scrollbar(table_tab, orient="horizontal", command=self.tree.xview)
        self.tree.configure(yscrollcommand=ysb.set, xscrollcommand=xsb.set)
        self.tree.pack(side="left", fill="both", expand=True)
        ysb.pack(side="right", fill="y")
        xsb.pack(side="bottom", fill="x")

        self.graph_frame = graph_tab

    def browse_file(self, var):
        f = filedialog.askopenfilename(title="Excelファイルを選択", filetypes=[("Excel files","*.xlsx *.xls"),("All","*.*")])
        if f:
            var.set(f)
            # ファイル選択時にメイン画面のComboboxも更新する（過去ファイル基準）
            if var == self.past_file_path:
                self.update_main_product_code_combo(f)


    def update_main_product_code_combo(self, file_path):
        try:
            # 最初のシートの最初の数行だけ読み込んで列名を取得
            temp_df = pd.read_excel(file_path, sheet_name=0, nrows=1)
            available_cols = ["旧ｺｰﾄﾞ連結", "中分類１"] + temp_df.columns.tolist()
            # 重複を除去してユニークなリストにする
            unique_cols = sorted(list(set(available_cols)))
            self.product_code_combo_main['values'] = unique_cols
            if self.product_code_column.get() not in unique_cols:
                 self.product_code_column.set(unique_cols[0] if unique_cols else "旧ｺｰﾄﾞ連結")

        except Exception as e:
            print(f"メイン画面のCombobox更新エラー: {e}")
            # エラー時もデフォルト値を設定
            self.product_code_combo_main['values'] = ["旧ｺｰﾄﾞ連結", "中分類１"]
            if self.product_code_column.get() not in ["旧ｺｰﾄﾞ連結", "中分類１"]:
                self.product_code_column.set("旧ｺｰﾄﾞ連結")


    def browse_output_file(self):
        f = filedialog.asksaveasfilename(title="保存先を選択", defaultextension=".xlsx", filetypes=[("Excel files","*.xlsx"),("All","*.*")])
        if f:
            self.output_file_path.set(f)

    def get_excel_sheets(self, path):
        try:
            return pd.ExcelFile(path).sheet_names
        except Exception as e:
            messagebox.showerror("エラー", f"シート取得失敗: {e}")
            return []

    def normalize_column_name(self, name):
        if not isinstance(name, str):
            return str(name)
        name = name.replace('　', ' ').strip()
        return name

    def find_best_match_column(self, df, target_name):
        if target_name in df.columns:
            return target_name
        norm_target = self.normalize_column_name(target_name)
        for col in df.columns:
            if self.normalize_column_name(col) == norm_target:
                return col
        best_match = None
        best_score = 0
        for col in df.columns:
            norm_col = self.normalize_column_name(col)
            if norm_target in norm_col or norm_col in norm_target:
                match_len = len(set(norm_target) & set(norm_col))
                if match_len > best_score:
                    best_score = match_len
                    best_match = col
        return best_match

    def show_sheet_selection_dialog(self, past_file, current_file):
        dialog = tk.Toplevel(self.root)
        dialog.title("シート選択と列設定")
        dialog.geometry("500x470") #ダイアログの高さを少し大きく
        dialog.transient(self.root)
        dialog.grab_set()

        past_sheets = self.get_excel_sheets(past_file)
        current_sheets = self.get_excel_sheets(current_file)

        ttk.Label(dialog, text="過去ファイルのシート:").pack(anchor="w", padx=10, pady=5)
        past_sheet_var = tk.StringVar(value=past_sheets[0] if past_sheets else "")
        past_combo = ttk.Combobox(dialog, textvariable=past_sheet_var, values=past_sheets, state="readonly", width=30)
        past_combo.pack(anchor="w", padx=10)

        ttk.Label(dialog, text="現在ファイルのシート:").pack(anchor="w", padx=10, pady=5)
        current_sheet_var = tk.StringVar(value=current_sheets[0] if current_sheets else "")
        current_combo = ttk.Combobox(dialog, textvariable=current_sheet_var, values=current_sheets, state="readonly", width=30)
        current_combo.pack(anchor="w", padx=10)

        preview_frame = ttk.LabelFrame(dialog, text="列名設定とプレビュー")
        preview_frame.pack(fill="both", expand=True, padx=10, pady=10)
        
        # product_code_varはStringVarとして初期化、現在のproduct_code_columnの値を使用
        product_code_var = tk.StringVar(value=self.product_code_column.get())
        amount_var = tk.StringVar(value=self.amount_column.get())

        ttk.Label(preview_frame, text="集計基準列:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        # product_code_combo は Combobox
        product_code_combo = ttk.Combobox(preview_frame, textvariable=product_code_var, width=30)
        product_code_combo.grid(row=0, column=1, padx=5, pady=5)

        ttk.Label(preview_frame, text="金額列:").grid(row=1, column=0, padx=5, pady=5, sticky="w")
        amount_combo = ttk.Combobox(preview_frame, textvariable=amount_var, width=30) # 金額列もComboboxが良いでしょう
        amount_combo.grid(row=1, column=1, padx=5, pady=5)

        preview_text = tk.Text(preview_frame, height=8, width=45)
        preview_text.grid(row=3, column=0, columnspan=2, padx=5, pady=5)

        def preview_columns_and_set_defaults():
            try:
                selected_past_sheet = past_sheet_var.get()
                if not selected_past_sheet:
                    messagebox.showwarning("注意", "過去ファイルのシートを選択してください。")
                    return

                df = pd.read_excel(past_file, sheet_name=selected_past_sheet, nrows=5)
                file_cols = df.columns.tolist()
                
                # 集計基準列の選択肢を設定 ("旧ｺｰﾄﾞ連結", "中分類１" を優先的に追加)
                product_code_options = sorted(list(set(["旧ｺｰﾄﾞ連結", "中分類１"] + file_cols)))
                product_code_combo['values'] = product_code_options
                
                amount_combo['values'] = file_cols

                current_product_col_val = product_code_var.get() # 現在のComboboxの値
                current_amount_col_val = amount_var.get()

                # 集計基準列のデフォルト値設定ロジック
                if current_product_col_val in product_code_options:
                    product_code_var.set(current_product_col_val) # 既存の値が選択肢にあればそれを維持
                elif "旧ｺｰﾄﾞ連結" in product_code_options:
                    product_code_var.set("旧ｺｰﾄﾞ連結")
                elif "中分類１" in product_code_options:
                    product_code_var.set("中分類１")
                elif product_code_options:
                    product_code_var.set(product_code_options[0])


                best_amount_match = self.find_best_match_column(df, current_amount_col_val if current_amount_col_val else self.amount_column.get())
                if best_amount_match:
                    amount_var.set(best_amount_match)
                elif file_cols:
                     amount_var.set(file_cols[0] if len(file_cols) == 1 else (file_cols[1] if len(file_cols) > 1 else ""))


                preview_text.delete("1.0", tk.END)
                preview_text.insert(tk.END, "使用可能な列一覧 (過去ファイルより):\n" + "\n".join(f"{i+1}. {c}" for i, c in enumerate(file_cols)))

                if len(df) > 0:
                    preview_text.insert(tk.END, "\n\n最初の行のデータ例:\n")
                    for col in file_cols[:5]:
                        val = df.iloc[0][col]
                        preview_text.insert(tk.END, f"{col}: {val}\n")

            except Exception as e:
                messagebox.showerror("エラー", f"列プレビュー失敗: {e}")

        past_combo.bind("<<ComboboxSelected>>", lambda event: preview_columns_and_set_defaults())
        dialog.after(100, preview_columns_and_set_defaults)
        ttk.Button(preview_frame, text="列名プレビュー更新", command=preview_columns_and_set_defaults).grid(row=2, column=0, columnspan=2, pady=5)

        def on_ok():
            if not past_sheet_var.get() or not current_sheet_var.get():
                messagebox.showerror("エラー", "過去と現在の両方のシートを選択してください。")
                return
            if not product_code_var.get() or not amount_var.get():
                messagebox.showerror("エラー", "集計基準列と金額列を選択または入力してください。")
                return

            self.past_sheet = past_sheet_var.get()
            self.current_sheet = current_sheet_var.get()
            self.product_code_column.set(product_code_var.get()) # ダイアログでの選択をメインの変数に反映
            self.amount_column.set(amount_var.get())
            
            # メイン画面のComboboxの値を更新
            self.product_code_combo_main.set(product_code_var.get())

            dialog.destroy()
            self.perform_analysis()

        ttk.Button(dialog, text="OK", command=on_ok).pack(pady=10)
        self.root.wait_window(dialog)


    def analyze_sales(self):
        if not self.past_file_path.get() or not self.current_file_path.get():
            messagebox.showerror("エラー", "過去と現在の両方のファイルを選択してください")
            return
        # 分析実行前にメイン画面のComboboxの値をproduct_code_columnに確実にセット
        self.product_code_column.set(self.product_code_combo_main.get())
        self.show_sheet_selection_dialog(self.past_file_path.get(), self.current_file_path.get())

    def perform_analysis(self):
        try:
            print(f"過去ファイルを読み込み中: {self.past_file_path.get()}, シート: {self.past_sheet}")
            past = pd.read_excel(self.past_file_path.get(), sheet_name=self.past_sheet)
            print(f"過去ファイルの列名: {past.columns.tolist()}")

            print(f"現在ファイルを読み込み中: {self.current_file_path.get()}, シート: {self.current_sheet}")
            curr = pd.read_excel(self.current_file_path.get(), sheet_name=self.current_sheet)
            print(f"現在ファイルの列名: {curr.columns.tolist()}")

            # product_code_columnはStringVarなので .get() で値を取得
            pc_user = self.product_code_column.get()
            amt_user = self.amount_column.get()

            print(f"ユーザー指定の集計基準列: {pc_user}")
            print(f"ユーザー指定の金額列: {amt_user}")

            pc_past = self.find_best_match_column(past, pc_user)
            if not pc_past: raise ValueError(f"集計基準列 '{pc_user}' が過去ファイルで見つかりません。")
            if pc_past != pc_user: messagebox.showinfo("列名の自動修正", f"過去ファイルで集計基準列 '{pc_user}' が見つからないため、最も近い '{pc_past}' を使用します。")

            amt_past = self.find_best_match_column(past, amt_user)
            if not amt_past: raise ValueError(f"金額列 '{amt_user}' が過去ファイルで見つかりません。")
            if amt_past != amt_user: messagebox.showinfo("列名の自動修正", f"過去ファイルで金額列 '{amt_user}' が見つからないため、最も近い '{amt_past}' を使用します。")

            pc_curr = self.find_best_match_column(curr, pc_user)
            if not pc_curr: raise ValueError(f"集計基準列 '{pc_user}' が現在ファイルで見つかりません。")
            if pc_curr != pc_user: messagebox.showinfo("列名の自動修正", f"現在ファイルで集計基準列 '{pc_user}' が見つからないため、最も近い '{pc_curr}' を使用します。")

            amt_curr = self.find_best_match_column(curr, amt_user)
            if not amt_curr: raise ValueError(f"金額列 '{amt_user}' が現在ファイルで見つかりません。")
            if amt_curr != amt_user: messagebox.showinfo("列名の自動修正", f"現在ファイルで金額列 '{amt_user}' が見つからないため、最も近い '{amt_curr}' を使用します。")

            for df, col, df_name in [(past, amt_past, "過去"), (curr, amt_curr, "現在")]:
                if df[col].dtype == object or isinstance(df[col].iloc[0], str) :
                    print(f"{df_name}ファイルの金額列 '{col}' は文字列型です。数値に変換します。")
                    df[col] = df[col].astype(str).str.replace(',', '', regex=False).str.replace(' ', '', regex=False)
                    df[col] = df[col].str.replace(r'[¥円]', '', regex=True)
                    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
                elif not pd.api.types.is_numeric_dtype(df[col]):
                    print(f"{df_name}ファイルの金額列 '{col}' は数値型ではありません。数値に変換を試みます。")
                    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
                else:
                    df[col] = df[col].fillna(0)
            
            # pc_user を集計キーとして使用
            self.past_summary = past.groupby(pc_past)[amt_past].sum().reset_index()
            self.past_summary.rename(columns={pc_past: pc_user, amt_past: amt_user}, inplace=True) # pc_userは集計後の列名
            self.past_summary['構成比'] = (self.past_summary[amt_user] / self.past_summary[amt_user].sum() * 100).fillna(0)

            self.current_summary = curr.groupby(pc_curr)[amt_curr].sum().reset_index()
            self.current_summary.rename(columns={pc_curr: pc_user, amt_curr: amt_user}, inplace=True) # pc_userは集計後の列名
            self.current_summary['構成比'] = (self.current_summary[amt_user] / self.current_summary[amt_user].sum() * 100).fillna(0)
            
            # 分析に使った列名を確定（find_best_match後の値ではなくユーザーが指定した/Comboboxで選択した値）
            self.product_code_column.set(pc_user)
            self.amount_column.set(amt_user)

            self.prepare_merged_data()
            self.display_results()
            self.plot_results()
            messagebox.showinfo("完了", "分析が完了しました")
        except Exception as e:
            import traceback
            traceback.print_exc()
            messagebox.showerror("エラー", f"分析中にエラー: {e}")

    def prepare_merged_data(self):
        # self.product_code_column.get() でユーザーが最終的に選択/入力した集計キー名を取得
        pc_key_name = self.product_code_column.get()
        amt_col_name = self.amount_column.get() # 金額列名も同様

        past_temp = self.past_summary.copy()
        current_temp = self.current_summary.copy()
        
        # 結合キーの列名が pc_key_name であることを確認
        # rename済みなので、past_tempとcurrent_tempの該当列は pc_key_name になっているはず
        if pc_key_name not in past_temp.columns:
             raise ValueError(f"過去サマリーに列 '{pc_key_name}' が存在しません。列: {past_temp.columns}")
        if pc_key_name not in current_temp.columns:
             raise ValueError(f"現在サマリーに列 '{pc_key_name}' が存在しません。列: {current_temp.columns}")


        past_temp[pc_key_name] = past_temp[pc_key_name].astype(str)
        current_temp[pc_key_name] = current_temp[pc_key_name].astype(str)
        
        # onパラメータには、実際にDataFrameに存在する列名 (pc_key_name) を使用
        m = pd.merge(past_temp, current_temp, on=pc_key_name, how='outer', suffixes=('_過去','_現在')).fillna(0)
        
        # renameする際、金額列名は amt_col_name をベースにする
        m.rename(columns={
            f'{amt_col_name}_過去': '過去売上金額',
            f'{amt_col_name}_現在': '現在売上金額',
            '構成比_過去': '過去構成比',
            '構成比_現在': '現在構成比'
        }, inplace=True)

        m['増減額'] = m['現在売上金額'] - m['過去売上金額']
        m['増減率'] = m.apply(lambda r: (r['増減額']/r['過去売上金額']*100) if r['過去売上金額']!=0 else (np.inf if r['増減額'] > 0 else (-np.inf if r['増減額'] < 0 else 0)), axis=1)
        
        # 並び替えの基準列も変更
        self.merged_data = m.sort_values('現在売上金額', ascending=False)
        # self.merged_data の中に pc_key_name 列があることを確認
        if pc_key_name not in self.merged_data.columns:
            # もし `on` で指定した列名が消えていたら、元の集計キーの列名を保持するようにする
            # しかし、mergeの仕様上、onで指定した列は残るはず。
            # renameで変わっている可能性も考慮するが、ここではpc_key_nameがそのままのはず。
            print(f"警告: マージ後のデータに集計キー '{pc_key_name}' が見つかりません。利用可能な列: {self.merged_data.columns}")


    def display_results(self):
        for i in self.tree.get_children():
            self.tree.delete(i)
        
        # Treeviewの列名を"集計キー"に変更
        self.tree.heading("集計キー", text=self.product_code_column.get()) # 動的にヘッダーテキストを設定

        # merged_dataから表示する際に使用するキー名
        display_key_column = self.product_code_column.get()

        for _, r in self.merged_data.iterrows():
            if display_key_column not in r:
                messagebox.showerror("エラー", f"結果データに列 '{display_key_column}' が見つかりません。")
                return
            self.tree.insert('', 'end', values=(
                r[display_key_column], # ここで正しい列名を使用
                f"{int(r['過去売上金額']):,}",
                f"{r['過去構成比']:.2f}%",
                f"{int(r['現在売上金額']):,}",
                f"{r['現在構成比']:.2f}%",
                f"{int(r['増減額']):,}",
                f"{r['増減率']:.2f}%" if pd.notnull(r['増減率']) and np.isfinite(r['増減率']) else ("-" if r['増減率']==0 else ("増加" if r['増減率']==np.inf else "減少"))
            ))

    def plot_results(self):
        for w in self.graph_frame.winfo_children():
            w.destroy()

        if self.merged_data is None or self.merged_data.empty:
            ttk.Label(self.graph_frame, text="表示するデータがありません。").pack(padx=10, pady=10)
            return

        top_n = 10
        top = self.merged_data.head(top_n)
        
        # グラフのラベルに使用する列名
        plot_label_column = self.product_code_column.get()
        if plot_label_column not in top.columns:
             messagebox.showerror("エラー", f"グラフ用データに列 '{plot_label_column}' が見つかりません。")
             return


        try:
            plt.rcParams['font.family'] = 'Yu Gothic'
            if not any(f.name == 'Yu Gothic' for f in plt.matplotlib.font_manager.fontManager.ttflist):
                 plt.rcParams['font.family'] = 'MS Gothic'
        except:
             plt.rcParams['font.family'] = 'sans-serif'
        plt.rcParams['axes.unicode_minus'] = False

        fig = plt.Figure(figsize=(14, 12), dpi=100)
        fig.subplots_adjust(hspace=0.5, wspace=0.3)

        ax1 = fig.add_subplot(221)
        nonzero_past = top[top['過去構成比'] > 0.01]
        if not nonzero_past.empty:
            wedges, texts, autotexts = ax1.pie(
                nonzero_past['過去構成比'],
                labels=nonzero_past[plot_label_column], # ここで使用
                autopct='%1.1f%%',
                startangle=90,
                pctdistance=0.85,
                wedgeprops={'edgecolor': 'white', 'linewidth': 0.5},
                textprops={'fontsize': 8}
            )
            plt.setp(autotexts, size=7, weight="bold", color="white")
        ax1.set_title(f'過去売上構成比 (上位{len(nonzero_past)}項目)', fontsize=10)
        ax1.axis('equal')

        ax2 = fig.add_subplot(222)
        nonzero_current = top[top['現在構成比'] > 0.01]
        if not nonzero_current.empty:
            wedges, texts, autotexts = ax2.pie(
                nonzero_current['現在構成比'],
                labels=nonzero_current[plot_label_column], # ここで使用
                autopct='%1.1f%%',
                startangle=90,
                pctdistance=0.85,
                wedgeprops={'edgecolor': 'white', 'linewidth': 0.5},
                textprops={'fontsize': 8}
            )
            plt.setp(autotexts, size=7, weight="bold", color="white")
        ax2.set_title(f'現在売上構成比 (上位{len(nonzero_current)}項目)', fontsize=10)
        ax2.axis('equal')

        ax3 = fig.add_subplot(223)
        x = np.arange(len(top))
        width = 0.35
        rects1 = ax3.bar(x - width/2, top['過去売上金額'], width, label='過去売上', color='skyblue')
        rects2 = ax3.bar(x + width/2, top['現在売上金額'], width, label='現在売上', color='lightcoral')

        ax3.set_ylabel('売上金額', fontsize=9)
        ax3.set_title(f'過去・現在 売上比較 (上位{top_n}項目)', fontsize=10)
        ax3.set_xticks(x)
        ax3.set_xticklabels(top[plot_label_column], rotation=45, ha='right', fontsize=8) # ここで使用
        ax3.legend(fontsize=8)
        ax3.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda val, loc: "{:,.0f}".format(val)))

        def autolabel(rects, ax_target): # ax -> ax_target
            for rect in rects:
                height = rect.get_height()
                ax_target.annotate('{:,.0f}'.format(height),
                                xy=(rect.get_x() + rect.get_width() / 2, height),
                                xytext=(0, 3),
                                textcoords="offset points",
                                ha='center', va='bottom', fontsize=7)
        autolabel(rects1, ax3)
        autolabel(rects2, ax3)
        ax3.tick_params(axis='y', labelsize=8)

        ax4 = fig.add_subplot(224)
        sorted_by_change = top.sort_values('増減額', ascending=False) # top_n件の現在売上ベースの項目で増減額をソート
        colors = ['green' if x_val > 0 else 'red' for x_val in sorted_by_change['増減額']] # x -> x_val
        bars = ax4.barh(sorted_by_change[plot_label_column], sorted_by_change['増減額'], color=colors, edgecolor='grey') # ここで使用
        ax4.set_xlabel('増減額', fontsize=9)
        ax4.set_title(f'売上増減額 (上位{top_n}項目の現在売上ベース)', fontsize=10)
        ax4.invert_yaxis()
        ax4.get_xaxis().set_major_formatter(plt.FuncFormatter(lambda val, loc: "{:,.0f}".format(val)))
        ax4.tick_params(axis='y', labelsize=8)
        ax4.tick_params(axis='x', labelsize=8)

        for bar in bars:
            bar_width = bar.get_width() # width -> bar_width (予約語衝突回避)
            # label_x_pos = bar_width + (50 if bar_width >=0 else -50) # この行は未使用なのでコメントアウトまたは削除
            if bar_width > 0:
                ha_val = 'left' # ha -> ha_val
                offset_val = 5 # offset -> offset_val
            else:
                ha_val = 'right' # ha -> ha_val
                offset_val = -5 # offset -> offset_val

            ax4.text(bar.get_width() + offset_val, bar.get_y() + bar.get_height()/2.,
                     '{:,.0f}'.format(bar_width),
                     ha=ha_val, va='center', fontsize=7)

        fig.tight_layout(pad=2.0)

        canvas = FigureCanvasTkAgg(fig, master=self.graph_frame)
        canvas.draw()
        canvas.get_tk_widget().pack(fill="both", expand=True)
        toolbar = NavigationToolbar2Tk(canvas, self.graph_frame)
        toolbar.update()


    def export_to_excel(self):
        if self.merged_data is None:
            messagebox.showerror("エラー", "先に分析を実行してください")
            return

        path = self.output_file_path.get()
        if not path:
            path = filedialog.asksaveasfilename(
                title="保存先を選択", defaultextension=".xlsx", filetypes=[("Excel files","*.xlsx"),("All","*.*")])
            if not path:
                return
        self.output_file_path.set(path)

        try:
            wb = openpyxl.Workbook()
            ws = wb.active
            ws.title = "売上分析結果"

            ws.merge_cells('A1:G1')
            ws['A1'] = f"{self.product_code_column.get()}別売上分析" # タイトルに集計キー名を使用
            ws['A1'].font = Font(size=14, bold=True, color="000080")
            ws['A1'].alignment = Alignment(horizontal='center')
            ws.merge_cells('A2:C2')
            ws['A2'] = f"作成日: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}"
            ws['A2'].font = Font(italic=True)

            headers = [self.product_code_column.get(),"過去売上金額","過去構成比","現在売上金額","現在構成比","増減額","増減率"] # ヘッダーにも使用
            for i, h in enumerate(headers, start=1):
                c = ws.cell(row=4, column=i, value=h)
                c.font = Font(bold=True, color="FFFFFF")
                c.alignment = Alignment(horizontal='center', vertical='center')
                c.fill = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")

            # Excel出力時に使用する集計キーの列名
            export_key_column = self.product_code_column.get()

            for idx, r_tuple in enumerate(self.merged_data.itertuples(index=False), start=5):
                # getattrで動的に値を取得
                key_value = getattr(r_tuple, export_key_column)
                ws.cell(row=idx, column=1, value=key_value)
                ws.cell(row=idx, column=2, value=r_tuple.過去売上金額)
                ws.cell(row=idx, column=3, value=r_tuple.過去構成比/100 if pd.notnull(r_tuple.過去構成比) else 0)
                ws.cell(row=idx, column=4, value=r_tuple.現在売上金額)
                ws.cell(row=idx, column=5, value=r_tuple.現在構成比/100 if pd.notnull(r_tuple.現在構成比) else 0)
                ws.cell(row=idx, column=6, value=r_tuple.増減額)
                ws.cell(row=idx, column=7, value=r_tuple.増減率/100 if pd.notnull(r_tuple.増減率) and np.isfinite(r_tuple.増減率) else 0)

                ws.cell(row=idx, column=1).alignment = Alignment(horizontal='left')
                for col_num in [2, 4, 6]:
                    ws.cell(row=idx, column=col_num).number_format = '#,##0'
                    ws.cell(row=idx, column=col_num).alignment = Alignment(horizontal='right')
                for col_num in [3, 5, 7]:
                    ws.cell(row=idx, column=col_num).number_format = '0.00%'
                    ws.cell(row=idx, column=col_num).alignment = Alignment(horizontal='right')

                if r_tuple.増減額 < 0:
                    ws.cell(row=idx, column=6).font = Font(color="FF0000")
                    ws.cell(row=idx, column=7).font = Font(color="FF0000")
                elif r_tuple.増減額 > 0 :
                    ws.cell(row=idx, column=6).font = Font(color="008000")
                    ws.cell(row=idx, column=7).font = Font(color="008000")

            for i, column_cells in enumerate(ws.columns):
                max_length = 0
                column = get_column_letter(i + 1)
                if ws.cell(row=4, column=i+1).value:
                    max_length = max(max_length, len(str(ws.cell(row=4, column=i+1).value)))

                for cell in column_cells[4:]:
                    try:
                        if cell.value:
                            cell_str_len = len(str(cell.value))
                            if isinstance(cell.value, (int, float)):
                                if '%' in cell.number_format:
                                    cell_str_len = len(f"{cell.value:.2%}")
                                elif ',' in cell.number_format:
                                    cell_str_len = len(f"{cell.value:,}")
                            max_length = max(max_length, cell_str_len)
                    except:
                        pass
                adjusted_width = (max_length + 2)
                ws.column_dimensions[column].width = adjusted_width if adjusted_width < 50 else 50

            total_row = 5 + len(self.merged_data)
            ws.cell(row=total_row, column=1, value="合計").font = Font(bold=True)
            ws.cell(row=total_row, column=1).alignment = Alignment(horizontal='center')
            ws.cell(row=total_row, column=1).fill = PatternFill(start_color="DDDDDD", end_color="DDDDDD", fill_type="solid")

            for col_idx, key in [(2, '過去売上金額'), (4, '現在売上金額'), (6, '増減額')]:
                val = self.merged_data[key].sum()
                cell = ws.cell(row=total_row, column=col_idx, value=val)
                cell.font = Font(bold=True)
                cell.number_format = '#,##0'
                cell.alignment = Alignment(horizontal='right')
                cell.fill = PatternFill(start_color="DDDDDD", end_color="DDDDDD", fill_type="solid")

            chart_sheet = wb.create_sheet(title="グラフについて")
            chart_sheet['A1'] = "グラフの確認は、分析アプリ本体の「グラフ」タブで行ってください。"
            chart_sheet['A1'].font = Font(size=12, bold=True)
            chart_sheet.merge_cells('A1:E1')

            wb.save(path)
            messagebox.showinfo("完了", f"Excelファイルを保存しました: {path}")
        except PermissionError:
            messagebox.showerror("エラー", f"ファイル '{path}' への書き込み権限がありません。\nファイルが開かれていないか確認してください。")
        except Exception as e:
            import traceback
            traceback.print_exc()
            messagebox.showerror("エラー", f"Excelファイル出力中にエラーが発生しました: {e}")


    def clear_results(self):
        for i in self.tree.get_children():
            self.tree.delete(i)
        for w in self.graph_frame.winfo_children():
            w.destroy()
        ttk.Label(self.graph_frame, text="分析を実行するとここにグラフが表示されます。").pack(padx=10,pady=10)
        self.past_summary = None
        self.current_summary = None
        self.merged_data = None
        # メイン画面のComboboxの値をリセットしないように変更（必要であればリセット処理追加）
        # self.product_code_column.set("旧ｺｰﾄﾞ連結") # 必要ならコメントアウト解除
        messagebox.showinfo("クリア", "分析結果をクリアしました")


def main():
    root = tk.Tk()
    app = SalesAnalysisApp(root)
    ttk.Label(app.graph_frame, text="分析を実行するとここにグラフが表示されます。").pack(padx=10,pady=10)
    root.mainloop()

if __name__ == "__main__":
    main()

過去ファイルを読み込み中: C:/Users/kkt040/Downloads/2025 04 売上照会.xlsx, シート: 2025 04 売上照会.
過去ファイルの列名: ['取引先', '金額', '売上数', '原価', '手配区分', '単価', '受注者', '配送便名', '取引先名', '納入先名', '旧ｺｰﾄﾞ連結', '商品名', '売上№', 'サイズ', '仕入先名', '直送仕入先名', '中分類１', '売上日', '粗利率', '仕入先コード', '直送仕入先', '来勘区分名', '商品', '取引区分', '売上行№', '来勘区分', '受注区分', '摘要名', '取引形態', '客先発注№', '納入先', '事業部', '売上数(バラ)', '事業部名', '部', '部名', 'グループ', 'グループ名', '担当者', '担当者名', '倉庫', '倉庫名', '受注№', '受注行№', '受注日', '受注数', '売上数(ケース)', 'ケース入数', '売上数（小箱）', '小箱入数', '単位', '納品書メモ', '仮単価区分', '赤黒', '税抜金額', '消費税', '税区分', '摘要', '配送便', '前回訂正', '一連№', '受注者.1', 'ﾋﾟｯｷﾝｸﾞﾒﾓ', '住所１', '住所２', '住所３']
現在ファイルを読み込み中: C:/Users/kkt040/Downloads/売上照会.xlsx, シート: 売上照会.
現在ファイルの列名: ['取引先', '金額', '利益総額', '原価総額', '売上数', '原価', '手配区分', '単価', '受注者', '配送便名', '取引先名', '納入先名', '旧ｺｰﾄﾞ連結', '商品名', '売上№', 'サイズ', '仕入先名', '直送仕入先名', '中分類１', '売上日', '粗利率', '仕入先コード', '直送仕入先', '来勘区分名', '商品', '取引区分', '売上行№', '来勘区分', '受注区分', '摘要名', '取引形態', '客先発注№', '納入先', '事業部', '売上数(バラ)', '事業部名', '部', '部名', 'グループ', 'グループ名', '担当者', '担当者名

In [4]:
#Gemini＆Claud作成 2025 0524 これが一番いい。
import pandas as pd
import tkinter as tk
from tkinter import filedialog, ttk, messagebox
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
import openpyxl
from openpyxl.styles import Font, Alignment, PatternFill
from openpyxl.utils import get_column_letter
import re
import numpy as np # numpyをインポート

class SalesAnalysisApp:
    def __init__(self, root):
        self.root = root
        self.root.title("売上照会分析アプリ")
        self.root.geometry("1200x800") # ウィンドウサイズを少し大きくしました

        # ファイルパス用変数
        self.past_file_path = tk.StringVar()
        self.current_file_path = tk.StringVar()
        self.output_file_path = tk.StringVar()

        # 列名設定用変数（デフォルト値）
        self.product_code_column = tk.StringVar(value="旧ｺｰﾄﾞ連結")
        self.amount_column = tk.StringVar(value="金額")

        # データ保存用
        self.past_summary = None
        self.current_summary = None
        self.merged_data = None

        self.create_widgets()

    def create_widgets(self):
        main_frame = ttk.Frame(self.root)
        main_frame.pack(fill="both", expand=True, padx=10, pady=10)

        # ファイル選択
        input_frame = ttk.LabelFrame(main_frame, text="ファイル選択")
        input_frame.pack(fill="x", pady=5)
        ttk.Label(input_frame, text="過去の売上照会ファイル:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        ttk.Entry(input_frame, textvariable=self.past_file_path, width=50).grid(row=0, column=1, padx=5, pady=5)
        ttk.Button(input_frame, text="参照", command=lambda: self.browse_file(self.past_file_path)).grid(row=0, column=2, padx=5)

        ttk.Label(input_frame, text="現在の売上照会ファイル:").grid(row=1, column=0, padx=5, pady=5, sticky="w")
        ttk.Entry(input_frame, textvariable=self.current_file_path, width=50).grid(row=1, column=1, padx=5, pady=5)
        ttk.Button(input_frame, text="参照", command=lambda: self.browse_file(self.current_file_path)).grid(row=1, column=2, padx=5)

        ttk.Label(input_frame, text="分析結果の出力先:").grid(row=2, column=0, padx=5, pady=5, sticky="w")
        ttk.Entry(input_frame, textvariable=self.output_file_path, width=50).grid(row=2, column=1, padx=5, pady=5)
        ttk.Button(input_frame, text="参照", command=self.browse_output_file).grid(row=2, column=2, padx=5)

        # 列名設定
        column_frame = ttk.LabelFrame(main_frame, text="列名設定")
        column_frame.pack(fill="x", pady=5)
        ttk.Label(column_frame, text="商品コード列名:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        ttk.Entry(column_frame, textvariable=self.product_code_column, width=20).grid(row=0, column=1, padx=5, pady=5)
        ttk.Label(column_frame, text="金額列名:").grid(row=0, column=2, padx=5, pady=5, sticky="w")
        ttk.Entry(column_frame, textvariable=self.amount_column, width=20).grid(row=0, column=3, padx=5, pady=5)
        ttk.Label(column_frame, text="※ファイル選択後に「分析実行」を押すとシート選択が可能").grid(row=1, column=0, columnspan=4, padx=5, pady=5)

        # 実行ボタン
        buttons_frame = ttk.Frame(main_frame)
        buttons_frame.pack(fill="x", pady=5)
        ttk.Button(buttons_frame, text="分析実行", command=self.analyze_sales, width=20).pack(side="left", padx=5)
        ttk.Button(buttons_frame, text="Excelに出力", command=self.export_to_excel, width=20).pack(side="left", padx=5)
        ttk.Button(buttons_frame, text="クリア", command=self.clear_results, width=20).pack(side="left", padx=5)

        # タブ
        tab_control = ttk.Notebook(main_frame)
        table_tab = ttk.Frame(tab_control)
        graph_tab = ttk.Frame(tab_control)
        tab_control.add(table_tab, text='テーブル')
        tab_control.add(graph_tab, text='グラフ')
        tab_control.pack(expand=1, fill="both")

        # テーブル表示
        cols = ("商品コード", "過去売上金額", "過去構成比", "現在売上金額", "現在構成比", "増減額", "増減率")
        self.tree = ttk.Treeview(table_tab, columns=cols, show="headings")
        for c in cols:
            self.tree.heading(c, text=c)
            self.tree.column(c, width=180 if c=="商品コード" else 130, anchor="center") # 商品コード列幅調整
        ysb = ttk.Scrollbar(table_tab, orient="vertical", command=self.tree.yview)
        xsb = ttk.Scrollbar(table_tab, orient="horizontal", command=self.tree.xview)
        self.tree.configure(yscrollcommand=ysb.set, xscrollcommand=xsb.set)
        self.tree.pack(side="left", fill="both", expand=True)
        ysb.pack(side="right", fill="y")
        xsb.pack(side="bottom", fill="x")

        # グラフフレーム
        self.graph_frame = graph_tab

    def browse_file(self, var):
        f = filedialog.askopenfilename(title="Excelファイルを選択", filetypes=[("Excel files","*.xlsx *.xls"),("All","*.*")])
        if f:
            var.set(f)

    def browse_output_file(self):
        f = filedialog.asksaveasfilename(title="保存先を選択", defaultextension=".xlsx", filetypes=[("Excel files","*.xlsx"),("All","*.*")])
        if f:
            self.output_file_path.set(f)

    def get_excel_sheets(self, path):
        try:
            return pd.ExcelFile(path).sheet_names
        except Exception as e:
            messagebox.showerror("エラー", f"シート取得失敗: {e}")
            return []

    def normalize_column_name(self, name):
        if not isinstance(name, str):
            return str(name)
        name = name.replace('　', ' ').strip()
        return name

    def find_best_match_column(self, df, target_name):
        if target_name in df.columns:
            return target_name
        norm_target = self.normalize_column_name(target_name)
        for col in df.columns:
            if self.normalize_column_name(col) == norm_target:
                return col
        best_match = None
        best_score = 0
        for col in df.columns:
            norm_col = self.normalize_column_name(col)
            if norm_target in norm_col or norm_col in norm_target:
                match_len = len(set(norm_target) & set(norm_col))
                if match_len > best_score:
                    best_score = match_len
                    best_match = col
        return best_match

    def show_sheet_selection_dialog(self, past_file, current_file):
        dialog = tk.Toplevel(self.root)
        dialog.title("シート選択と列設定")
        dialog.geometry("500x450") #ダイアログの高さを少し大きく
        dialog.transient(self.root)
        dialog.grab_set()

        past_sheets = self.get_excel_sheets(past_file)
        current_sheets = self.get_excel_sheets(current_file)

        ttk.Label(dialog, text="過去ファイルのシート:").pack(anchor="w", padx=10, pady=5)
        past_sheet_var = tk.StringVar(value=past_sheets[0] if past_sheets else "")
        past_combo = ttk.Combobox(dialog, textvariable=past_sheet_var, values=past_sheets, state="readonly", width=30)
        past_combo.pack(anchor="w", padx=10)

        ttk.Label(dialog, text="現在ファイルのシート:").pack(anchor="w", padx=10, pady=5)
        current_sheet_var = tk.StringVar(value=current_sheets[0] if current_sheets else "")
        current_combo = ttk.Combobox(dialog, textvariable=current_sheet_var, values=current_sheets, state="readonly", width=30)
        current_combo.pack(anchor="w", padx=10)

        preview_frame = ttk.LabelFrame(dialog, text="列名設定とプレビュー")
        preview_frame.pack(fill="both", expand=True, padx=10, pady=10)
        product_code_var = tk.StringVar(value=self.product_code_column.get())
        amount_var = tk.StringVar(value=self.amount_column.get())

        ttk.Label(preview_frame, text="商品コード列:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        product_code_combo = ttk.Combobox(preview_frame, textvariable=product_code_var, width=30)
        product_code_combo.grid(row=0, column=1, padx=5, pady=5)

        ttk.Label(preview_frame, text="金額列:").grid(row=1, column=0, padx=5, pady=5, sticky="w")
        amount_combo = ttk.Combobox(preview_frame, textvariable=amount_var, width=30)
        amount_combo.grid(row=1, column=1, padx=5, pady=5)

        preview_text = tk.Text(preview_frame, height=8, width=45) #幅を少し大きく
        preview_text.grid(row=3, column=0, columnspan=2, padx=5, pady=5)

        def preview_columns_and_set_defaults():
            try:
                selected_past_sheet = past_sheet_var.get()
                if not selected_past_sheet:
                    messagebox.showwarning("注意", "過去ファイルのシートを選択してください。")
                    return

                df = pd.read_excel(past_file, sheet_name=selected_past_sheet, nrows=5)
                cols = df.columns.tolist()
                product_code_combo['values'] = cols
                amount_combo['values'] = cols

                # 既存の値を保持しつつ、最適な列を推測
                current_product_col = product_code_var.get()
                current_amount_col = amount_var.get()

                best_product_match = self.find_best_match_column(df, current_product_col)
                best_amount_match = self.find_best_match_column(df, current_amount_col)

                if best_product_match:
                    product_code_var.set(best_product_match)
                elif cols: # 一致がない場合はリストの先頭
                    product_code_var.set(cols[0])


                if best_amount_match:
                    amount_var.set(best_amount_match)
                elif len(cols) > 1: # 一致がない場合はリストの2番目（もしあれば）
                    amount_var.set(cols[1])
                elif cols: # 2番目がなければ先頭
                     amount_var.set(cols[0])


                preview_text.delete("1.0", tk.END)
                preview_text.insert(tk.END, "使用可能な列一覧 (過去ファイルより):\n" + "\n".join(f"{i+1}. {c}" for i, c in enumerate(cols)))

                if len(df) > 0:
                    preview_text.insert(tk.END, "\n\n最初の行のデータ例:\n")
                    for col in cols[:5]: # Display first 5 columns
                        val = df.iloc[0][col]
                        preview_text.insert(tk.END, f"{col}: {val}\n")

            except Exception as e:
                messagebox.showerror("エラー", f"列プレビュー失敗: {e}")

        # シート選択変更時にプレビューを更新
        past_combo.bind("<<ComboboxSelected>>", lambda event: preview_columns_and_set_defaults())
        # 初回表示時にもプレビューを実行
        # Toplevelが表示されてから実行するために少し遅延させる
        dialog.after(100, preview_columns_and_set_defaults)


        ttk.Button(preview_frame, text="列名プレビュー更新", command=preview_columns_and_set_defaults).grid(row=2, column=0, columnspan=2, pady=5)


        def on_ok():
            if not past_sheet_var.get() or not current_sheet_var.get():
                messagebox.showerror("エラー", "過去と現在の両方のシートを選択してください。")
                return
            if not product_code_var.get() or not amount_var.get():
                messagebox.showerror("エラー", "商品コード列と金額列を選択または入力してください。")
                return

            self.past_sheet = past_sheet_var.get()
            self.current_sheet = current_sheet_var.get()
            self.product_code_column.set(product_code_var.get())
            self.amount_column.set(amount_var.get())
            dialog.destroy()
            self.perform_analysis()

        ttk.Button(dialog, text="OK", command=on_ok).pack(pady=10)
        self.root.wait_window(dialog)


    def analyze_sales(self):
        if not self.past_file_path.get() or not self.current_file_path.get():
            messagebox.showerror("エラー", "過去と現在の両方のファイルを選択してください")
            return
        self.show_sheet_selection_dialog(self.past_file_path.get(), self.current_file_path.get())

    def perform_analysis(self):
        try:
            print(f"過去ファイルを読み込み中: {self.past_file_path.get()}, シート: {self.past_sheet}")
            past = pd.read_excel(self.past_file_path.get(), sheet_name=self.past_sheet)
            print(f"過去ファイルの列名: {past.columns.tolist()}")

            print(f"現在ファイルを読み込み中: {self.current_file_path.get()}, シート: {self.current_sheet}")
            curr = pd.read_excel(self.current_file_path.get(), sheet_name=self.current_sheet)
            print(f"現在ファイルの列名: {curr.columns.tolist()}")

            pc_user = self.product_code_column.get()
            amt_user = self.amount_column.get()

            print(f"ユーザー指定の商品コード列: {pc_user}")
            print(f"ユーザー指定の金額列: {amt_user}")

            # 列名の存在確認と自動修正ロジック
            pc_past = self.find_best_match_column(past, pc_user)
            if not pc_past: raise ValueError(f"商品コード列 '{pc_user}' が過去ファイルで見つかりません。")
            if pc_past != pc_user: messagebox.showinfo("列名の自動修正", f"過去ファイルで商品コード列 '{pc_user}' が見つからないため、最も近い '{pc_past}' を使用します。")

            amt_past = self.find_best_match_column(past, amt_user)
            if not amt_past: raise ValueError(f"金額列 '{amt_user}' が過去ファイルで見つかりません。")
            if amt_past != amt_user: messagebox.showinfo("列名の自動修正", f"過去ファイルで金額列 '{amt_user}' が見つからないため、最も近い '{amt_past}' を使用します。")

            pc_curr = self.find_best_match_column(curr, pc_user)
            if not pc_curr: raise ValueError(f"商品コード列 '{pc_user}' が現在ファイルで見つかりません。")
            if pc_curr != pc_user: messagebox.showinfo("列名の自動修正", f"現在ファイルで商品コード列 '{pc_user}' が見つからないため、最も近い '{pc_curr}' を使用します。")

            amt_curr = self.find_best_match_column(curr, amt_user)
            if not amt_curr: raise ValueError(f"金額列 '{amt_user}' が現在ファイルで見つかりません。")
            if amt_curr != amt_user: messagebox.showinfo("列名の自動修正", f"現在ファイルで金額列 '{amt_user}' が見つからないため、最も近い '{amt_curr}' を使用します。")

            # 金額列のデータ型変換
            for df, col, df_name in [(past, amt_past, "過去"), (curr, amt_curr, "現在")]:
                if df[col].dtype == object or isinstance(df[col].iloc[0], str) : # 文字列型の場合の処理を強化
                    print(f"{df_name}ファイルの金額列 '{col}' は文字列型です。数値に変換します。")
                    df[col] = df[col].astype(str).str.replace(',', '', regex=False).str.replace(' ', '', regex=False)
                    # 円マークやその他の通貨記号も除去する可能性を考慮
                    df[col] = df[col].str.replace(r'[¥円]', '', regex=True)
                    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
                elif not pd.api.types.is_numeric_dtype(df[col]): # 数値型でない場合も変換を試みる
                    print(f"{df_name}ファイルの金額列 '{col}' は数値型ではありません。数値に変換を試みます。")
                    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
                else: #既に数値型の場合
                     df[col] = df[col].fillna(0)


            # グループ集計 (ユーザーが指定した元の列名をpc_userとして保持)
            self.past_summary = past.groupby(pc_past)[amt_past].sum().reset_index()
            self.past_summary.rename(columns={pc_past: pc_user, amt_past: amt_user}, inplace=True)
            self.past_summary['構成比'] = (self.past_summary[amt_user] / self.past_summary[amt_user].sum() * 100).fillna(0)

            self.current_summary = curr.groupby(pc_curr)[amt_curr].sum().reset_index()
            self.current_summary.rename(columns={pc_curr: pc_user, amt_curr: amt_user}, inplace=True)
            self.current_summary['構成比'] = (self.current_summary[amt_user] / self.current_summary[amt_user].sum() * 100).fillna(0)

            # グローバル変数にもユーザーが指定した列名を設定
            self.product_code_column.set(pc_user)
            self.amount_column.set(amt_user)


            self.prepare_merged_data()
            self.display_results()
            self.plot_results() # 修正後のplot_resultsが呼ばれる
            messagebox.showinfo("完了", "分析が完了しました")
        except Exception as e:
            import traceback
            traceback.print_exc()
            messagebox.showerror("エラー", f"分析中にエラー: {e}")

    def prepare_merged_data(self):
        pc = self.product_code_column.get() # find_best_match後の正しい列名
        amt = self.amount_column.get() # find_best_match後の正しい列名

        past_temp = self.past_summary.copy()
        current_temp = self.current_summary.copy()

        # 結合キーとなる商品コード列のデータ型を文字列に統一
        past_temp[pc] = past_temp[pc].astype(str)
        current_temp[pc] = current_temp[pc].astype(str)

        m = pd.merge(past_temp, current_temp, on=pc, how='outer', suffixes=('_過去','_現在')).fillna(0)
        m.rename(columns={
            f'{amt}_過去': '過去売上金額',
            f'{amt}_現在': '現在売上金額',
            '構成比_過去': '過去構成比',
            '構成比_現在': '現在構成比'
        }, inplace=True)
        m['増減額'] = m['現在売上金額'] - m['過去売上金額']
        # ゼロ除算を避ける
        m['増減率'] = m.apply(lambda r: (r['増減額']/r['過去売上金額']*100) if r['過去売上金額']!=0 else (np.inf if r['増減額'] > 0 else (-np.inf if r['増減額'] < 0 else 0)), axis=1)
        self.merged_data = m.sort_values('現在売上金額', ascending=False)

    def display_results(self):
        for i in self.tree.get_children():
            self.tree.delete(i)
        pc_col_name = self.product_code_column.get() # 正しい商品コード列名を使用
        for _, r in self.merged_data.iterrows():
            self.tree.insert('', 'end', values=(
                r[pc_col_name], # ここを修正
                f"{int(r['過去売上金額']):,}",
                f"{r['過去構成比']:.2f}%",
                f"{int(r['現在売上金額']):,}",
                f"{r['現在構成比']:.2f}%",
                f"{int(r['増減額']):,}",
                f"{r['増減率']:.2f}%" if pd.notnull(r['増減率']) and np.isfinite(r['増減率']) else ("-" if r['増減率']==0 else ("増加" if r['増減率']==np.inf else "減少")) # 無限大の場合の表示
            ))

    def plot_results(self):
        for w in self.graph_frame.winfo_children():
            w.destroy()

        if self.merged_data is None or self.merged_data.empty:
            ttk.Label(self.graph_frame, text="表示するデータがありません。").pack(padx=10, pady=10)
            return

        top_n = 10 # 表示する上位商品数
        top = self.merged_data.head(top_n)
        pc_col = self.product_code_column.get()

        # 日本語フォント対応
        try:
            plt.rcParams['font.family'] = 'Yu Gothic' # Windowsで一般的に利用可能なフォント
            if not any(f.name == 'Yu Gothic' for f in plt.matplotlib.font_manager.fontManager.ttflist): #フォントが存在しない場合
                 plt.rcParams['font.family'] = 'MS Gothic' #代替フォント
        except:
             plt.rcParams['font.family'] = 'sans-serif' #それでもダメならデフォルト
        plt.rcParams['axes.unicode_minus'] = False # マイナス記号の文字化け対策


        fig = plt.Figure(figsize=(14, 12), dpi=100) # figsizeとdpi調整
        fig.subplots_adjust(hspace=0.5, wspace=0.3) # subplot間のスペース調整

        # --- 1. 過去売上構成比（円グラフ） ---
        ax1 = fig.add_subplot(221)
        nonzero_past = top[top['過去構成比'] > 0.01] # 0.01%未満は表示しないなど調整可能
        if not nonzero_past.empty:
            wedges, texts, autotexts = ax1.pie(
                nonzero_past['過去構成比'],
                labels=nonzero_past[pc_col],
                autopct='%1.1f%%',
                startangle=90,
                pctdistance=0.85, # パーセンテージ表示の位置
                wedgeprops={'edgecolor': 'white', 'linewidth': 0.5}, # 枠線
                textprops={'fontsize': 8} # ラベルフォントサイズ
            )
            plt.setp(autotexts, size=7, weight="bold", color="white") #パーセンテージのスタイル
        ax1.set_title(f'過去売上構成比 (上位{len(nonzero_past)}商品)', fontsize=10)
        ax1.axis('equal') # 真円にする

        # --- 2. 現在売上構成比（円グラフ） ---
        ax2 = fig.add_subplot(222)
        nonzero_current = top[top['現在構成比'] > 0.01]
        if not nonzero_current.empty:
            wedges, texts, autotexts = ax2.pie(
                nonzero_current['現在構成比'],
                labels=nonzero_current[pc_col],
                autopct='%1.1f%%',
                startangle=90,
                pctdistance=0.85,
                wedgeprops={'edgecolor': 'white', 'linewidth': 0.5},
                textprops={'fontsize': 8}
            )
            plt.setp(autotexts, size=7, weight="bold", color="white")
        ax2.set_title(f'現在売上構成比 (上位{len(nonzero_current)}商品)', fontsize=10)
        ax2.axis('equal')

        # --- 3. 過去と現在の売上比較（棒グラフ） ---
        ax3 = fig.add_subplot(223) # 2行2列の3番目
        x = np.arange(len(top))
        width = 0.35
        rects1 = ax3.bar(x - width/2, top['過去売上金額'], width, label='過去売上', color='skyblue')
        rects2 = ax3.bar(x + width/2, top['現在売上金額'], width, label='現在売上', color='lightcoral')

        ax3.set_ylabel('売上金額', fontsize=9)
        ax3.set_title(f'過去・現在 売上比較 (上位{top_n}商品)', fontsize=10)
        ax3.set_xticks(x)
        ax3.set_xticklabels(top[pc_col], rotation=45, ha='right', fontsize=8)
        ax3.legend(fontsize=8)
        ax3.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda val, loc: "{:,.0f}".format(val))) # Y軸カンマ区切り

        # 棒グラフの上に数値を表示
        def autolabel(rects, ax):
            for rect in rects:
                height = rect.get_height()
                ax.annotate('{:,.0f}'.format(height),
                            xy=(rect.get_x() + rect.get_width() / 2, height),
                            xytext=(0, 3),  # 3 points vertical offset
                            textcoords="offset points",
                            ha='center', va='bottom', fontsize=7)
        autolabel(rects1, ax3)
        autolabel(rects2, ax3)
        ax3.tick_params(axis='y', labelsize=8)


        # --- 4. 増減額 上位・下位商品（水平棒グラフ） ---
        ax4 = fig.add_subplot(224) # 2行2列の4番目
        # 増減額でソートし、絶対値の大きい順に表示（ただし、上位Nと下位Nを表示する方が良い場合も）
        # ここでは単純にtop_n件の増減額を表示
        sorted_by_change = top.sort_values('増減額', ascending=False)
        colors = ['green' if x > 0 else 'red' for x in sorted_by_change['増減額']]
        bars = ax4.barh(sorted_by_change[pc_col], sorted_by_change['増減額'], color=colors, edgecolor='grey')
        ax4.set_xlabel('増減額', fontsize=9)
        ax4.set_title(f'売上増減額 (上位{top_n}商品の現在売上ベース)', fontsize=10)
        ax4.invert_yaxis() # 上から大きい順
        ax4.get_xaxis().set_major_formatter(plt.FuncFormatter(lambda val, loc: "{:,.0f}".format(val)))
        ax4.tick_params(axis='y', labelsize=8)
        ax4.tick_params(axis='x', labelsize=8)

        # 水平棒グラフに数値を表示
        for bar in bars:
            width = bar.get_width()
            label_x_pos = width + (50 if width >=0 else -50) #数値表示位置のオフセット調整
            if width > 0:
                ha = 'left'
                offset = 5
            else:
                ha = 'right'
                offset = -5

            ax4.text(bar.get_width() + offset, bar.get_y() + bar.get_height()/2.,
                     '{:,.0f}'.format(width),
                     ha=ha, va='center', fontsize=7)


        fig.tight_layout(pad=2.0) # 全体的なレイアウト調整

        canvas = FigureCanvasTkAgg(fig, master=self.graph_frame)
        canvas.draw()
        canvas.get_tk_widget().pack(fill="both", expand=True)
        toolbar = NavigationToolbar2Tk(canvas, self.graph_frame)
        toolbar.update()


    def export_to_excel(self):
        if self.merged_data is None:
            messagebox.showerror("エラー", "先に分析を実行してください")
            return

        path = self.output_file_path.get()
        if not path:
            path = filedialog.asksaveasfilename(
                title="保存先を選択", defaultextension=".xlsx", filetypes=[("Excel files","*.xlsx"),("All","*.*")])
            if not path:
                return
        self.output_file_path.set(path) # ユーザーが選択したパスを記憶

        try:
            wb = openpyxl.Workbook()
            ws = wb.active
            ws.title = "売上分析結果"

            # タイトル・日付
            ws.merge_cells('A1:G1')
            ws['A1'] = "商品別売上分析"
            ws['A1'].font = Font(size=14, bold=True, color="000080") # 色変更
            ws['A1'].alignment = Alignment(horizontal='center')
            ws.merge_cells('A2:C2')
            ws['A2'] = f"作成日: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}" # 時刻も追加
            ws['A2'].font = Font(italic=True)


            headers = ["商品コード","過去売上金額","過去構成比","現在売上金額","現在構成比","増減額","増減率"]
            for i, h in enumerate(headers, start=1):
                c = ws.cell(row=4, column=i, value=h)
                c.font = Font(bold=True, color="FFFFFF") # 文字色白
                c.alignment = Alignment(horizontal='center', vertical='center')
                c.fill = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid") #濃い青

            # データ
            pc_col_name = self.product_code_column.get() # 正しい商品コード列名
            for idx, r_tuple in enumerate(self.merged_data.itertuples(index=False), start=5):
                # itertuplesでDataFrameの行を名前付きタプルとして取得
                # 商品コードの列名が動的なので、getattrで値を取得
                product_code_value = getattr(r_tuple, pc_col_name)
                ws.cell(row=idx, column=1, value=product_code_value)
                ws.cell(row=idx, column=2, value=r_tuple.過去売上金額)
                ws.cell(row=idx, column=3, value=r_tuple.過去構成比/100 if pd.notnull(r_tuple.過去構成比) else 0)
                ws.cell(row=idx, column=4, value=r_tuple.現在売上金額)
                ws.cell(row=idx, column=5, value=r_tuple.現在構成比/100 if pd.notnull(r_tuple.現在構成比) else 0)
                ws.cell(row=idx, column=6, value=r_tuple.増減額)
                ws.cell(row=idx, column=7, value=r_tuple.増減率/100 if pd.notnull(r_tuple.増減率) and np.isfinite(r_tuple.増減率) else 0) # 無限大は0として出力

                # 書式設定
                ws.cell(row=idx, column=1).alignment = Alignment(horizontal='left')
                for col_num in [2, 4, 6]: # 金額列
                    ws.cell(row=idx, column=col_num).number_format = '#,##0'
                    ws.cell(row=idx, column=col_num).alignment = Alignment(horizontal='right')
                for col_num in [3, 5, 7]: # 比率列
                    ws.cell(row=idx, column=col_num).number_format = '0.00%'
                    ws.cell(row=idx, column=col_num).alignment = Alignment(horizontal='right')

                # 条件付き書式（増減額・増減率がマイナスなら赤字）
                if r_tuple.増減額 < 0:
                    ws.cell(row=idx, column=6).font = Font(color="FF0000") # 赤
                    ws.cell(row=idx, column=7).font = Font(color="FF0000") # 赤
                elif r_tuple.増減額 > 0 :
                     ws.cell(row=idx, column=6).font = Font(color="008000") # 緑
                     ws.cell(row=idx, column=7).font = Font(color="008000") # 緑


            # 列幅自動調整の改善
            for i, column_cells in enumerate(ws.columns):
                max_length = 0
                column = get_column_letter(i + 1)
                # ヘッダーの長さを考慮
                if ws.cell(row=4, column=i+1).value:
                     max_length = max(max_length, len(str(ws.cell(row=4, column=i+1).value)))

                for cell in column_cells[4:]: # データ行のみ対象
                    try:
                        if cell.value:
                            cell_str_len = len(str(cell.value))
                            # 数値やパーセンテージの場合はフォーマット後の見た目も考慮（簡易的）
                            if isinstance(cell.value, (int, float)):
                                if '%' in cell.number_format:
                                    cell_str_len = len(f"{cell.value:.2%}")
                                elif ',' in cell.number_format:
                                     cell_str_len = len(f"{cell.value:,}")
                            max_length = max(max_length, cell_str_len)
                    except:
                        pass
                adjusted_width = (max_length + 2)
                ws.column_dimensions[column].width = adjusted_width if adjusted_width < 50 else 50 # 最大幅制限


            # 合計行
            total_row = 5 + len(self.merged_data)
            ws.cell(row=total_row, column=1, value="合計").font = Font(bold=True)
            ws.cell(row=total_row, column=1).alignment = Alignment(horizontal='center')
            ws.cell(row=total_row, column=1).fill = PatternFill(start_color="DDDDDD", end_color="DDDDDD", fill_type="solid")


            for col_idx, key in [(2, '過去売上金額'), (4, '現在売上金額'), (6, '増減額')]:
                val = self.merged_data[key].sum()
                cell = ws.cell(row=total_row, column=col_idx, value=val)
                cell.font = Font(bold=True)
                cell.number_format = '#,##0'
                cell.alignment = Alignment(horizontal='right')
                cell.fill = PatternFill(start_color="DDDDDD", end_color="DDDDDD", fill_type="solid")


            # グラフシートは今回はExcelへの直接描画は行わず、アプリでの確認を促すメッセージのみ
            chart_sheet = wb.create_sheet(title="グラフについて")
            chart_sheet['A1'] = "グラフの確認は、分析アプリ本体の「グラフ」タブで行ってください。"
            chart_sheet['A1'].font = Font(size=12, bold=True)
            chart_sheet.merge_cells('A1:E1')


            wb.save(path)
            messagebox.showinfo("完了", f"Excelファイルを保存しました: {path}")
        except PermissionError:
            messagebox.showerror("エラー", f"ファイル '{path}' への書き込み権限がありません。\nファイルが開かれていないか確認してください。")
        except Exception as e:
            import traceback
            traceback.print_exc()
            messagebox.showerror("エラー", f"Excelファイル出力中にエラーが発生しました: {e}")


    def clear_results(self):
        for i in self.tree.get_children():
            self.tree.delete(i)
        for w in self.graph_frame.winfo_children():
            w.destroy() # グラフタブの子ウィジェットをすべて削除
        ttk.Label(self.graph_frame, text="分析を実行するとここにグラフが表示されます。").pack(padx=10,pady=10) # 初期メッセージ再表示
        self.past_summary = None
        self.current_summary = None
        self.merged_data = None
        # ファイルパスや列名はクリアしないでおく（再分析のため）
        messagebox.showinfo("クリア", "分析結果をクリアしました")


def main():
    root = tk.Tk()
    app = SalesAnalysisApp(root)
    # 初期状態でグラフタブにメッセージを表示
    ttk.Label(app.graph_frame, text="分析を実行するとここにグラフが表示されます。").pack(padx=10,pady=10)
    root.mainloop()

if __name__ == "__main__":
    main()

過去ファイルを読み込み中: C:/Users/kkt040/OneDrive - トルク株式会社/デスクトップ/85期上期評価/【坂口】８４期 上期 ｻｶｸﾞﾁ 群馬 栃木 売上照会.xlsx, シート: ８４期 上期 ｻｶｸﾞﾁ 群馬 栃木 売上照会.
過去ファイルの列名: ['取引先', '金額', '売上数', '原価', '手配区分', '単価', '受注者', '配送便名', '取引先名', '納入先名', '旧ｺｰﾄﾞ連結', '商品名', '売上№', 'サイズ', '仕入先名', '直送仕入先名', '中分類１', '売上日', '粗利率', '仕入先コード', '直送仕入先', '来勘区分名', '商品', '取引区分', '売上行№', '来勘区分', '受注区分', '摘要名', '取引形態', '客先発注№', '納入先', '事業部', '売上数(バラ)', '事業部名', '部', '部名', 'グループ', 'グループ名', '担当者', '担当者名', '倉庫', '倉庫名', '受注№', '受注行№', '受注日', '受注数', '売上数(ケース)', 'ケース入数', '売上数（小箱）', '小箱入数', '単位', '納品書メモ', '仮単価区分', '赤黒', '税抜金額', '消費税', '税区分', '摘要', '配送便', '前回訂正', '一連№', '受注者.1', 'ﾋﾟｯｷﾝｸﾞﾒﾓ', '住所１', '住所２', '住所３']
現在ファイルを読み込み中: C:/Users/kkt040/OneDrive - トルク株式会社/デスクトップ/85期上期評価/【坂口】85期 坂口 売上照会.xlsx, シート: 85期 坂口 売上照会.
現在ファイルの列名: ['取引先', '金額', '売上数', '原価', '手配区分', '単価', '受注者', '配送便名', '取引先名', '納入先名', '旧ｺｰﾄﾞ連結', '商品名', '売上№', 'サイズ', '仕入先名', '直送仕入先名', '中分類１', '売上日', '粗利率', '仕入先コード', '直送仕入先', '来勘区分名', '商品', '取引区分', '売上行№', '来勘区分', '受注区分', '摘要名', '取引形態', '客先発注№

In [4]:
#チャットGPT作成 2025 05 24

import pandas as pd 
import tkinter as tk
from tkinter import filedialog, ttk, messagebox
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
import openpyxl
from openpyxl.styles import Font, Alignment, PatternFill
from openpyxl.utils import get_column_letter
import re

class SalesAnalysisApp:
    def __init__(self, root):
        self.root = root
        self.root.title("売上照会分析アプリ")
        self.root.geometry("1000x700")
        
        # ファイルパス用変数
        self.past_file_path = tk.StringVar()
        self.current_file_path = tk.StringVar()
        self.output_file_path = tk.StringVar()
        
        # 列名設定用変数（デフォルト値）
        self.product_code_column = tk.StringVar(value="旧ｺｰﾄﾞ連結")
        self.amount_column = tk.StringVar(value="金額")
        
        # データ保存用
        self.past_summary = None
        self.current_summary = None
        self.merged_data = None
        
        self.create_widgets()
    
    def create_widgets(self):
        main_frame = ttk.Frame(self.root)
        main_frame.pack(fill="both", expand=True, padx=10, pady=10)
        
        # ファイル選択
        input_frame = ttk.LabelFrame(main_frame, text="ファイル選択")
        input_frame.pack(fill="x", pady=5)
        ttk.Label(input_frame, text="過去の売上照会ファイル:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        ttk.Entry(input_frame, textvariable=self.past_file_path, width=50).grid(row=0, column=1, padx=5, pady=5)
        ttk.Button(input_frame, text="参照", command=lambda: self.browse_file(self.past_file_path)).grid(row=0, column=2, padx=5)
        
        ttk.Label(input_frame, text="現在の売上照会ファイル:").grid(row=1, column=0, padx=5, pady=5, sticky="w")
        ttk.Entry(input_frame, textvariable=self.current_file_path, width=50).grid(row=1, column=1, padx=5, pady=5)
        ttk.Button(input_frame, text="参照", command=lambda: self.browse_file(self.current_file_path)).grid(row=1, column=2, padx=5)
        
        ttk.Label(input_frame, text="分析結果の出力先:").grid(row=2, column=0, padx=5, pady=5, sticky="w")
        ttk.Entry(input_frame, textvariable=self.output_file_path, width=50).grid(row=2, column=1, padx=5, pady=5)
        ttk.Button(input_frame, text="参照", command=self.browse_output_file).grid(row=2, column=2, padx=5)
        
        # 列名設定
        column_frame = ttk.LabelFrame(main_frame, text="列名設定")
        column_frame.pack(fill="x", pady=5)
        ttk.Label(column_frame, text="商品コード列名:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        ttk.Entry(column_frame, textvariable=self.product_code_column, width=20).grid(row=0, column=1, padx=5, pady=5)
        ttk.Label(column_frame, text="金額列名:").grid(row=0, column=2, padx=5, pady=5, sticky="w")
        ttk.Entry(column_frame, textvariable=self.amount_column, width=20).grid(row=0, column=3, padx=5, pady=5)
        ttk.Label(column_frame, text="※ファイル選択後に「分析実行」を押すとシート選択が可能").grid(row=1, column=0, columnspan=4, padx=5, pady=5)
        
        # 実行ボタン
        buttons_frame = ttk.Frame(main_frame)
        buttons_frame.pack(fill="x", pady=5)
        ttk.Button(buttons_frame, text="分析実行", command=self.analyze_sales, width=20).pack(side="left", padx=5)
        ttk.Button(buttons_frame, text="Excelに出力", command=self.export_to_excel, width=20).pack(side="left", padx=5)
        ttk.Button(buttons_frame, text="クリア", command=self.clear_results, width=20).pack(side="left", padx=5)
        
        # タブ
        tab_control = ttk.Notebook(main_frame)
        table_tab = ttk.Frame(tab_control)
        graph_tab = ttk.Frame(tab_control)
        tab_control.add(table_tab, text='テーブル')
        tab_control.add(graph_tab, text='グラフ')
        tab_control.pack(expand=1, fill="both")
        
        # テーブル表示
        cols = ("商品コード", "過去売上金額", "過去構成比", "現在売上金額", "現在構成比", "増減額", "増減率")
        self.tree = ttk.Treeview(table_tab, columns=cols, show="headings")
        for c in cols:
            self.tree.heading(c, text=c)
            self.tree.column(c, width=150 if c=="商品コード" else 120, anchor="center")
        ysb = ttk.Scrollbar(table_tab, orient="vertical", command=self.tree.yview)
        xsb = ttk.Scrollbar(table_tab, orient="horizontal", command=self.tree.xview)
        self.tree.configure(yscrollcommand=ysb.set, xscrollcommand=xsb.set)
        self.tree.pack(side="left", fill="both", expand=True)
        ysb.pack(side="right", fill="y")
        xsb.pack(side="bottom", fill="x")
        
        # グラフフレーム
        self.graph_frame = graph_tab

        # グラフ切り替えボタン
        graph_buttons_frame = ttk.Frame(graph_tab)
        graph_buttons_frame.pack(fill="x", pady=5)
        ttk.Button(graph_buttons_frame, text="デフォルト", command=self.plot_results).pack(side="left", padx=5)
        ttk.Button(graph_buttons_frame, text="売上増減率", command=self.plot_increase_rate).pack(side="left", padx=5)
        ttk.Button(graph_buttons_frame, text="全体売上比較", command=self.plot_total_comparison).pack(side="left", padx=5)
        ttk.Button(graph_buttons_frame, text="構成比推移", command=self.plot_share_shift).pack(side="left", padx=5)

    def browse_file(self, var):
        f = filedialog.askopenfilename(title="Excelファイルを選択", filetypes=[("Excel files","*.xlsx *.xls"),("All","*.*")])
        if f:
            var.set(f)

    def browse_output_file(self):
        f = filedialog.asksaveasfilename(title="保存先を選択", defaultextension=".xlsx", filetypes=[("Excel files","*.xlsx"),("All","*.*")])
        if f:
            self.output_file_path.set(f)

    def get_excel_sheets(self, path):
        try:
            return pd.ExcelFile(path).sheet_names
        except Exception as e:
            messagebox.showerror("エラー", f"シート取得失敗: {e}")
            return []
            
    def normalize_column_name(self, name):
        if not isinstance(name, str):
            return str(name)
        name = name.replace('　', ' ').strip()
        return name
        
    def find_best_match_column(self, df, target_name):
        if target_name in df.columns:
            return target_name
        norm_target = self.normalize_column_name(target_name)
        for col in df.columns:
            if self.normalize_column_name(col) == norm_target:
                return col
        best_match = None
        best_score = 0
        for col in df.columns:
            norm_col = self.normalize_column_name(col)
            if norm_target in norm_col or norm_col in norm_target:
                match_len = len(set(norm_target) & set(norm_col))
                if match_len > best_score:
                    best_score = match_len
                    best_match = col
        return best_match

    def show_sheet_selection_dialog(self, past_file, current_file):
        dialog = tk.Toplevel(self.root)
        dialog.title("シート選択と列設定")
        dialog.geometry("500x400")
        dialog.transient(self.root)
        dialog.grab_set()

        past_sheets = self.get_excel_sheets(past_file)
        current_sheets = self.get_excel_sheets(current_file)

        ttk.Label(dialog, text="過去ファイルのシート:").pack(anchor="w", padx=10, pady=5)
        past_sheet_var = tk.StringVar(value=past_sheets[0] if past_sheets else "")
        ttk.Combobox(dialog, textvariable=past_sheet_var, values=past_sheets, state="readonly", width=30).pack(anchor="w", padx=10)

        ttk.Label(dialog, text="現在ファイルのシート:").pack(anchor="w", padx=10, pady=5)
        current_sheet_var = tk.StringVar(value=current_sheets[0] if current_sheets else "")
        ttk.Combobox(dialog, textvariable=current_sheet_var, values=current_sheets, state="readonly", width=30).pack(anchor="w", padx=10)

        preview_frame = ttk.LabelFrame(dialog, text="列名プレビュー")
        preview_frame.pack(fill="both", expand=True, padx=10, pady=10)
        product_code_var = tk.StringVar(value=self.product_code_column.get())
        amount_var = tk.StringVar(value=self.amount_column.get())

        ttk.Label(preview_frame, text="商品コード列:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        product_code_combo = ttk.Combobox(preview_frame, textvariable=product_code_var, width=30)
        product_code_combo.grid(row=0, column=1, padx=5, pady=5)
        ttk.Label(preview_frame, text="金額列:").grid(row=1, column=0, padx=5, pady=5, sticky="w")
        amount_combo = ttk.Combobox(preview_frame, textvariable=amount_var, width=30)
        amount_combo.grid(row=1, column=1, padx=5, pady=5)
        preview_text = tk.Text(preview_frame, height=8, width=40)
        preview_text.grid(row=3, column=0, columnspan=2, padx=5, pady=5)

        def preview_columns():
            try:
                df = pd.read_excel(past_file, sheet_name=past_sheet_var.get(), nrows=5)
                cols = df.columns.tolist()
                product_code_combo['values'] = cols
                amount_combo['values'] = cols
                best_product_match = self.find_best_match_column(df, product_code_var.get())
                best_amount_match = self.find_best_match_column(df, amount_var.get())
                if best_product_match:
                    product_code_var.set(best_product_match)
                if best_amount_match:
                    amount_var.set(best_amount_match)
                preview_text.delete("1.0", tk.END)
                preview_text.insert(tk.END, "使用可能な列一覧:\n" + "\n".join(f"{i+1}. {c}" for i, c in enumerate(cols)))
                if len(df) > 0:
                    preview_text.insert(tk.END, "\n\n最初の行のデータ例:\n")
                    for col in cols[:5]:
                        val = df.iloc[0][col]
                        preview_text.insert(tk.END, f"{col}: {val}\n")
            except Exception as e:
                messagebox.showerror("エラー", f"列プレビュー失敗: {e}")

        ttk.Button(preview_frame, text="列名プレビュー表示", command=preview_columns).grid(row=2, column=0, columnspan=2, pady=5)

        def on_ok():
            self.past_sheet = past_sheet_var.get()
            self.current_sheet = current_sheet_var.get()
            self.product_code_column.set(product_code_var.get())
            self.amount_column.set(amount_var.get())
            dialog.destroy()
            self.perform_analysis()

        ttk.Button(dialog, text="OK", command=on_ok).pack(pady=10)
        self.root.wait_window(dialog)

    def analyze_sales(self):
        if not self.past_file_path.get() or not self.current_file_path.get():
            messagebox.showerror("エラー", "ファイルを選択してください")
            return
        self.show_sheet_selection_dialog(self.past_file_path.get(), self.current_file_path.get())

    def perform_analysis(self):
        try:
            print(f"過去ファイルを読み込み中: {self.past_file_path.get()}, シート: {self.past_sheet}")
            past = pd.read_excel(self.past_file_path.get(), sheet_name=self.past_sheet)
            print(f"過去ファイルの列名: {past.columns.tolist()}")
            print(f"現在ファイルを読み込み中: {self.current_file_path.get()}, シート: {self.current_sheet}")
            curr = pd.read_excel(self.current_file_path.get(), sheet_name=self.current_sheet)
            print(f"現在ファイルの列名: {curr.columns.tolist()}")
            pc = self.product_code_column.get()
            amt = self.amount_column.get()
            print(f"使用する商品コード列: {pc}")
            print(f"使用する金額列: {amt}")
            if pc not in past.columns:
                best_match = self.find_best_match_column(past, pc)
                if best_match:
                    messagebox.showinfo("列名の自動修正", f"過去ファイルで '{pc}' が見つからないため、最も近い '{best_match}' を使用します")
                    pc_past = best_match
                else:
                    raise ValueError(f"商品コード列 '{pc}' が過去ファイルで見つかりません")
            else:
                pc_past = pc
            if pc not in curr.columns:
                best_match = self.find_best_match_column(curr, pc)
                if best_match:
                    messagebox.showinfo("列名の自動修正", f"現在ファイルで '{pc}' が見つからないため、最も近い '{best_match}' を使用します")
                    pc_curr = best_match
                else:
                    raise ValueError(f"商品コード列 '{pc}' が現在ファイルで見つかりません")
            else:
                pc_curr = pc
            if amt not in past.columns:
                best_match = self.find_best_match_column(past, amt)
                if best_match:
                    messagebox.showinfo("列名の自動修正", f"過去ファイルで '{amt}' が見つからないため、最も近い '{best_match}' を使用します")
                    amt_past = best_match
                else:
                    raise ValueError(f"金額列 '{amt}' が過去ファイルで見つかりません")
            else:
                amt_past = amt
            if amt not in curr.columns:
                best_match = self.find_best_match_column(curr, amt)
                if best_match:
                    messagebox.showinfo("列名の自動修正", f"現在ファイルで '{amt}' が見つからないため、最も近い '{best_match}' を使用します")
                    amt_curr = best_match
                else:
                    raise ValueError(f"金額列 '{amt}' が現在ファイルで見つかりません")
            else:
                amt_curr = amt
            for df, col in [(past, amt_past), (curr, amt_curr)]:
                if df[col].dtype == object:
                    df[col] = df[col].astype(str).str.replace(',', '').str.replace(' ', '')
                    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
            self.past_summary = past.groupby(pc_past)[amt_past].sum().reset_index()
            self.past_summary.rename(columns={pc_past: pc, amt_past: amt}, inplace=True)
            self.past_summary['構成比'] = self.past_summary[amt] / self.past_summary[amt].sum() * 100
            self.current_summary = curr.groupby(pc_curr)[amt_curr].sum().reset_index()
            self.current_summary.rename(columns={pc_curr: pc, amt_curr: amt}, inplace=True)
            self.current_summary['構成比'] = self.current_summary[amt] / self.current_summary[amt].sum() * 100
            self.prepare_merged_data()
            self.display_results()
            self.plot_results()
            messagebox.showinfo("完了", "分析が完了しました")
        except Exception as e:
            import traceback
            traceback.print_exc()
            messagebox.showerror("エラー", f"分析中にエラー: {e}")

    def prepare_merged_data(self):
        pc = self.product_code_column.get()
        amt = self.amount_column.get()
        past_temp = self.past_summary.copy()
        current_temp = self.current_summary.copy()
        past_temp[pc] = past_temp[pc].astype(str)
        current_temp[pc] = current_temp[pc].astype(str)
        m = pd.merge(past_temp, current_temp, on=pc, how='outer', suffixes=('_過去','_現在')).fillna(0)
        m.rename(columns={
            f'{amt}_過去': '過去売上金額',
            f'{amt}_現在': '現在売上金額',
            '構成比_過去': '過去構成比',
            '構成比_現在': '現在構成比'
        }, inplace=True)
        m['増減額'] = m['現在売上金額'] - m['過去売上金額']
        m['増減率'] = m.apply(lambda r: 0 if r['過去売上金額']==0 else r['増減額']/r['過去売上金額']*100, axis=1)
        self.merged_data = m.sort_values('現在売上金額', ascending=False)

    def display_results(self):
        for i in self.tree.get_children():
            self.tree.delete(i)
        pc = self.product_code_column.get()
        for _, r in self.merged_data.iterrows():
            self.tree.insert('', 'end', values=(
                r[pc],
                f"{int(r['過去売上金額']):,}",
                f"{r['過去構成比']:.2f}%",
                f"{int(r['現在売上金額']):,}",
                f"{r['現在構成比']:.2f}%",
                f"{int(r['増減額']):,}",
                f"{r['増減率']:.2f}%"
            ))

    # デフォルトグラフ（円グラフ＋棒グラフ）
    def plot_results(self):
        for w in self.graph_frame.winfo_children():
            w.destroy()
        top = self.merged_data.head(10)
        fig = plt.Figure(figsize=(12,8))
        plt.rcParams['font.family'] = 'MS Gothic'
        ax1 = fig.add_subplot(221)
        nonzero_past = top[top['過去構成比'] > 0]
        if len(nonzero_past) > 0:
            ax1.pie(nonzero_past['過去構成比'], labels=nonzero_past[self.product_code_column.get()], 
                   autopct='%1.1f%%', startangle=90)
        ax1.set_title('過去売上構成比（上位10商品）')
        ax2 = fig.add_subplot(222)
        nonzero_current = top[top['現在構成比'] > 0]
        if len(nonzero_current) > 0:
            ax2.pie(nonzero_current['現在構成比'], labels=nonzero_current[self.product_code_column.get()], 
                   autopct='%1.1f%%', startangle=90)
        ax2.set_title('現在売上構成比（上位10商品）')
        ax3 = fig.add_subplot(212)
        x = range(len(top))
        w = 0.35
        ax3.bar([i-w/2 for i in x], top['過去売上金額'], w, label='過去')
        ax3.bar([i+w/2 for i in x], top['現在売上金額'], w, label='現在')
        ax3.set_xticks(x)
        ax3.set_xticklabels(top[self.product_code_column.get()], rotation=45, ha='right')
        ax3.set_title('過去と現在の売上比較（上位10商品）')
        ax3.legend()
        ax3.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))
        fig.tight_layout()
        self.show_figure_on_canvas(fig)

    # 売上増減率ランキング
    def plot_increase_rate(self):
        for w in self.graph_frame.winfo_children():
            w.destroy()
        top = self.merged_data.copy()
        top = top.sort_values('増減率', ascending=False).head(10)
        fig, ax = plt.subplots(figsize=(10, 6))
        bars = ax.bar(top[self.product_code_column.get()], top['増減率'])
        ax.set_ylabel("増減率（%）")
        ax.set_title("売上増減率トップ10商品")
        ax.set_xticklabels(top[self.product_code_column.get()], rotation=45, ha='right')
        ax.axhline(0, color='black', linewidth=0.8)
        fig.tight_layout()
        self.show_figure_on_canvas(fig)

    # 全体売上比較グラフ
    def plot_total_comparison(self):
        for w in self.graph_frame.winfo_children():
            w.destroy()
        fig, ax = plt.subplots(figsize=(5, 5))
        labels = ['過去合計', '現在合計']
        values = [
            self.merged_data['過去売上金額'].sum(),
            self.merged_data['現在売上金額'].sum()
        ]
        ax.bar(labels, values)
        ax.set_ylabel("売上金額")
        ax.set_title("全体売上金額の比較")
        ax.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))
        fig.tight_layout()
        self.show_figure_on_canvas(fig)

    # 構成比推移グラフ
    def plot_share_shift(self):
        for w in self.graph_frame.winfo_children():
            w.destroy()
        top = self.merged_data.head(10)
        fig, ax = plt.subplots(figsize=(10, 6))
        width = 0.35
        x = range(len(top))
        ax.bar([i - width/2 for i in x], top['過去構成比'], width, label='過去')
        ax.bar([i + width/2 for i in x], top['現在構成比'], width, label='現在')
        ax.set_xticks(list(x))
        ax.set_xticklabels(top[self.product_code_column.get()], rotation=45, ha='right')
        ax.set_ylabel("構成比（%）")
        ax.set_title("上位10商品の構成比推移")
        ax.legend()
        fig.tight_layout()
        self.show_figure_on_canvas(fig)

    def show_figure_on_canvas(self, fig):
        for w in self.graph_frame.winfo_children():
            w.destroy()
        canvas = FigureCanvasTkAgg(fig, master=self.graph_frame)
        canvas.draw()
        canvas.get_tk_widget().pack(fill="both", expand=True)
        toolbar = NavigationToolbar2Tk(canvas, self.graph_frame)
        toolbar.update()

    def export_to_excel(self):
        if self.merged_data is None:
            messagebox.showerror("エラー", "先に分析を実行してください")
            return
        path = self.output_file_path.get() or filedialog.asksaveasfilename(
            title="保存先を選択", defaultextension=".xlsx", filetypes=[("Excel files","*.xlsx"),("All","*.*")])
        if not path:
            return
        self.output_file_path.set(path)
        wb = openpyxl.Workbook()
        ws = wb.active
        ws.title = "売上分析結果"
        ws.merge_cells('A1:G1')
        ws['A1'] = "商品別売上分析"
        ws['A1'].font = Font(size=14, bold=True)
        ws.merge_cells('A2:C2')
        ws['A2'] = f"作成日: {pd.Timestamp.now().strftime('%Y-%m-%d')}"
        headers = ["商品コード","過去売上金額","過去構成比","現在売上金額","現在構成比","増減額","増減率"]
        for i, h in enumerate(headers, start=1):
            c = ws.cell(row=4, column=i, value=h)
            c.font = Font(bold=True)
            c.alignment = Alignment(horizontal='center')
            c.fill = PatternFill(start_color="DDDDDD", end_color="DDDDDD", fill_type="solid")
        for idx, (_, r) in enumerate(self.merged_data.iterrows(), start=5):
            ws.cell(row=idx, column=1, value=r[self.product_code_column.get()])
            ws.cell(row=idx, column=2, value=r['過去売上金額'])
            ws.cell(row=idx, column=3, value=r['過去構成比']/100)
            ws.cell(row=idx, column=4, value=r['現在売上金額'])
            ws.cell(row=idx, column=5, value=r['現在構成比']/100)
            ws.cell(row=idx, column=6, value=r['増減額'])
            ws.cell(row=idx, column=7, value=r['増減率']/100)
            if r['増減額'] < 0:
                ws.cell(row=idx, column=6).font = Font(color="FF0000")
                ws.cell(row=idx, column=7).font = Font(color="FF0000")
        for row in range(5, 5 + len(self.merged_data)):
            ws.cell(row=row, column=2).number_format = '#,##0'
            ws.cell(row=row, column=3).number_format = '0.00%'
            ws.cell(row=row, column=4).number_format = '#,##0'
            ws.cell(row=row, column=5).number_format = '0.00%'
            ws.cell(row=row, column=6).number_format = '#,##0'
            ws.cell(row=row, column=7).number_format = '0.00%'
        for col in range(1, 8):
            ws.column_dimensions[get_column_letter(col)].width = 20 if col==1 else 15
        total_row = 5 + len(self.merged_data)
        ws.cell(row=total_row, column=1, value="合計").font = Font(bold=True)
        for col_idx, key in [(2, '過去売上金額'), (4, '現在売上金額'), (6, '増減額')]:
            val = self.merged_data[key].sum()
            ws.cell(row=total_row, column=col_idx, value=val).font = Font(bold=True)
            ws.cell(row=total_row, column=col_idx).number_format = '#,##0'
        chart_sheet = wb.create_sheet(title="グラフ")
        chart_sheet['A1'] = "※Excelでは自動グラフ作成はできません。アプリ上でグラフを確認してください。"
        wb.save(path)
        messagebox.showinfo("完了", f"Excelファイルを保存しました: {path}")

    def clear_results(self):
        for i in self.tree.get_children():
            self.tree.delete(i)
        for w in self.graph_frame.winfo_children():
            w.destroy()
        self.past_summary = self.current_summary = self.merged_data = None
        messagebox.showinfo("クリア", "分析結果をクリアしました")

def main():
    root = tk.Tk()
    app = SalesAnalysisApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()


過去ファイルを読み込み中: C:/Users/kkt040/Downloads/83期 日の出ネジ 売上照会.xlsx, シート: 83期 日の出ネジ 売上照会.
過去ファイルの列名: ['取引先', '金額', '売上数', '原価', '手配区分', '単価', '受注者', '配送便名', '取引先名', '納入先名', '旧ｺｰﾄﾞ連結', '商品名', '売上№', 'サイズ', '仕入先名', '直送仕入先名', '中分類１', '売上日', '粗利率', '仕入先コード', '直送仕入先', '来勘区分名', '商品', '取引区分', '売上行№', '来勘区分', '受注区分', '摘要名', '取引形態', '客先発注№', '納入先', '事業部', '売上数(バラ)', '事業部名', '部', '部名', 'グループ', 'グループ名', '担当者', '担当者名', '倉庫', '倉庫名', '受注№', '受注行№', '受注日', '受注数', '売上数(ケース)', 'ケース入数', '売上数（小箱）', '小箱入数', '単位', '納品書メモ', '仮単価区分', '赤黒', '税抜金額', '消費税', '税区分', '摘要', '配送便', '前回訂正', '一連№', '受注者.1', 'ﾋﾟｯｷﾝｸﾞﾒﾓ', '住所１', '住所２', '住所３']
現在ファイルを読み込み中: C:/Users/kkt040/Downloads/84期 日の出ネジ 売上照会.xlsx, シート: 84期 日の出ネジ 売上照会.
現在ファイルの列名: ['取引先', '金額', '売上数', '原価', '手配区分', '単価', '受注者', '配送便名', '取引先名', '納入先名', '旧ｺｰﾄﾞ連結', '商品名', '売上№', 'サイズ', '仕入先名', '直送仕入先名', '中分類１', '売上日', '粗利率', '仕入先コード', '直送仕入先', '来勘区分名', '商品', '取引区分', '売上行№', '来勘区分', '受注区分', '摘要名', '取引形態', '客先発注№', '納入先', '事業部', '売上数(バラ)', '事業部名', '部', '部名', 'グループ', 'グループ名', '担当者

In [3]:
#次回それぞれのグラフを作成する。（2025、5，24） Claud作成
import pandas as pd 
import tkinter as tk
from tkinter import filedialog, ttk, messagebox
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
import openpyxl
from openpyxl.styles import Font, Alignment, PatternFill
from openpyxl.utils import get_column_letter
import re
import numpy as np
import seaborn as sns
from matplotlib import font_manager
import platform

class SalesAnalysisApp:
    def __init__(self, root):
        self.root = root
        self.root.title("売上照会分析アプリ（改良版）")
        self.root.geometry("1200x800")
        
        # ファイルパス用変数
        self.past_file_path = tk.StringVar()
        self.current_file_path = tk.StringVar()
        self.output_file_path = tk.StringVar()
        
        # 列名設定用変数（デフォルト値）
        self.product_code_column = tk.StringVar(value="旧ｺｰﾄﾞ連結")
        self.amount_column = tk.StringVar(value="金額")
        
        # グラフ設定用変数
        self.graph_type = tk.StringVar(value="総合")
        self.show_top_n = tk.IntVar(value=10)
        self.color_scheme = tk.StringVar(value="カラフル")
        
        # データ保存用
        self.past_summary = None
        self.current_summary = None
        self.merged_data = None
        
        # 日本語フォント設定
        self.setup_japanese_font()
        
        self.create_widgets()
    
    def setup_japanese_font(self):
        """日本語フォントを自動設定"""
        try:
            system = platform.system()
            if system == "Windows":
                # Windowsの場合
                fonts = ['MS Gothic', 'Meiryo', 'Yu Gothic']
            elif system == "Darwin":  # macOS
                fonts = ['Hiragino Sans', 'Arial Unicode MS']
            else:  # Linux
                fonts = ['DejaVu Sans', 'Liberation Sans']
            
            # 利用可能なフォントをチェック
            available_fonts = [f.name for f in font_manager.fontManager.ttflist]
            self.japanese_font = None
            
            for font in fonts:
                if font in available_fonts:
                    self.japanese_font = font
                    break
            
            if self.japanese_font:
                plt.rcParams['font.family'] = [self.japanese_font]
                plt.rcParams['axes.unicode_minus'] = False
            else:
                # フォールバック
                plt.rcParams['font.family'] = ['DejaVu Sans']
                
        except Exception as e:
            print(f"フォント設定エラー: {e}")
            plt.rcParams['font.family'] = ['DejaVu Sans']
    
    def create_widgets(self):
        main_frame = ttk.Frame(self.root)
        main_frame.pack(fill="both", expand=True, padx=10, pady=10)
        
        # ファイル選択
        input_frame = ttk.LabelFrame(main_frame, text="ファイル選択")
        input_frame.pack(fill="x", pady=5)
        ttk.Label(input_frame, text="過去の売上照会ファイル:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        ttk.Entry(input_frame, textvariable=self.past_file_path, width=50).grid(row=0, column=1, padx=5, pady=5)
        ttk.Button(input_frame, text="参照", command=lambda: self.browse_file(self.past_file_path)).grid(row=0, column=2, padx=5)
        
        ttk.Label(input_frame, text="現在の売上照会ファイル:").grid(row=1, column=0, padx=5, pady=5, sticky="w")
        ttk.Entry(input_frame, textvariable=self.current_file_path, width=50).grid(row=1, column=1, padx=5, pady=5)
        ttk.Button(input_frame, text="参照", command=lambda: self.browse_file(self.current_file_path)).grid(row=1, column=2, padx=5)
        
        ttk.Label(input_frame, text="分析結果の出力先:").grid(row=2, column=0, padx=5, pady=5, sticky="w")
        ttk.Entry(input_frame, textvariable=self.output_file_path, width=50).grid(row=2, column=1, padx=5, pady=5)
        ttk.Button(input_frame, text="参照", command=self.browse_output_file).grid(row=2, column=2, padx=5)
        
        # 列名設定
        column_frame = ttk.LabelFrame(main_frame, text="列名設定")
        column_frame.pack(fill="x", pady=5)
        ttk.Label(column_frame, text="商品コード列名:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        ttk.Entry(column_frame, textvariable=self.product_code_column, width=20).grid(row=0, column=1, padx=5, pady=5)
        ttk.Label(column_frame, text="金額列名:").grid(row=0, column=2, padx=5, pady=5, sticky="w")
        ttk.Entry(column_frame, textvariable=self.amount_column, width=20).grid(row=0, column=3, padx=5, pady=5)
        ttk.Label(column_frame, text="※ファイル選択後に「分析実行」を押すとシート選択が可能").grid(row=1, column=0, columnspan=4, padx=5, pady=5)
        
        # グラフ設定
        graph_config_frame = ttk.LabelFrame(main_frame, text="グラフ設定")
        graph_config_frame.pack(fill="x", pady=5)
        
        ttk.Label(graph_config_frame, text="グラフ種類:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        graph_types = ["総合", "比較棒グラフ", "パイチャート", "増減分析", "トレンド分析", "散布図"]
        ttk.Combobox(graph_config_frame, textvariable=self.graph_type, values=graph_types, 
                    state="readonly", width=15).grid(row=0, column=1, padx=5, pady=5)
        
        ttk.Label(graph_config_frame, text="表示件数:").grid(row=0, column=2, padx=5, pady=5, sticky="w")
        ttk.Spinbox(graph_config_frame, from_=5, to=20, textvariable=self.show_top_n, 
                   width=10).grid(row=0, column=3, padx=5, pady=5)
        
        ttk.Label(graph_config_frame, text="配色:").grid(row=0, column=4, padx=5, pady=5, sticky="w")
        color_schemes = ["カラフル", "ブルー系", "グリーン系", "暖色系", "モノクロ"]
        ttk.Combobox(graph_config_frame, textvariable=self.color_scheme, values=color_schemes, 
                    state="readonly", width=12).grid(row=0, column=5, padx=5, pady=5)
        
        ttk.Button(graph_config_frame, text="グラフ更新", command=self.update_graph).grid(row=0, column=6, padx=5, pady=5)
        
        # 実行ボタン
        buttons_frame = ttk.Frame(main_frame)
        buttons_frame.pack(fill="x", pady=5)
        ttk.Button(buttons_frame, text="分析実行", command=self.analyze_sales, width=20).pack(side="left", padx=5)
        ttk.Button(buttons_frame, text="Excelに出力", command=self.export_to_excel, width=20).pack(side="left", padx=5)
        ttk.Button(buttons_frame, text="グラフ保存", command=self.save_graph, width=20).pack(side="left", padx=5)
        ttk.Button(buttons_frame, text="クリア", command=self.clear_results, width=20).pack(side="left", padx=5)
        
        # タブ
        tab_control = ttk.Notebook(main_frame)
        table_tab = ttk.Frame(tab_control)
        graph_tab = ttk.Frame(tab_control)
        tab_control.add(table_tab, text='テーブル')
        tab_control.add(graph_tab, text='グラフ')
        tab_control.pack(expand=1, fill="both")
        
        # テーブル表示
        cols = ("商品コード", "過去売上金額", "過去構成比", "現在売上金額", "現在構成比", "増減額", "増減率")
        self.tree = ttk.Treeview(table_tab, columns=cols, show="headings")
        for c in cols:
            self.tree.heading(c, text=c)
            self.tree.column(c, width=150 if c=="商品コード" else 120, anchor="center")
        ysb = ttk.Scrollbar(table_tab, orient="vertical", command=self.tree.yview)
        xsb = ttk.Scrollbar(table_tab, orient="horizontal", command=self.tree.xview)
        self.tree.configure(yscrollcommand=ysb.set, xscrollcommand=xsb.set)
        self.tree.pack(side="left", fill="both", expand=True)
        ysb.pack(side="right", fill="y")
        xsb.pack(side="bottom", fill="x")
        
        # グラフフレーム
        self.graph_frame = graph_tab

    def get_color_palette(self):
        """配色テーマに基づいてカラーパレットを返す"""
        schemes = {
            "カラフル": plt.cm.Set3(np.linspace(0, 1, 12)),
            "ブルー系": plt.cm.Blues(np.linspace(0.3, 0.9, 10)),
            "グリーン系": plt.cm.Greens(np.linspace(0.3, 0.9, 10)),
            "暖色系": plt.cm.Reds(np.linspace(0.3, 0.9, 10)),
            "モノクロ": plt.cm.gray(np.linspace(0.2, 0.8, 10))
        }
        return schemes.get(self.color_scheme.get(), plt.cm.Set3(np.linspace(0, 1, 12)))

    def browse_file(self, var):
        f = filedialog.askopenfilename(title="Excelファイルを選択", filetypes=[("Excel files","*.xlsx *.xls"),("All","*.*")])
        if f:
            var.set(f)

    def browse_output_file(self):
        f = filedialog.asksaveasfilename(title="保存先を選択", defaultextension=".xlsx", filetypes=[("Excel files","*.xlsx"),("All","*.*")])
        if f:
            self.output_file_path.set(f)

    def get_excel_sheets(self, path):
        try:
            return pd.ExcelFile(path).sheet_names
        except Exception as e:
            messagebox.showerror("エラー", f"シート取得失敗: {e}")
            return []
            
    def normalize_column_name(self, name):
        """列名を正規化して比較しやすくする"""
        if not isinstance(name, str):
            return str(name)
        name = name.replace('　', ' ').strip()
        return name
        
    def find_best_match_column(self, df, target_name):
        """最も近い列名を見つける"""
        if target_name in df.columns:
            return target_name
            
        norm_target = self.normalize_column_name(target_name)
        
        for col in df.columns:
            if self.normalize_column_name(col) == norm_target:
                return col
                
        best_match = None
        best_score = 0
        
        for col in df.columns:
            norm_col = self.normalize_column_name(col)
            if norm_target in norm_col or norm_col in norm_target:
                match_len = len(set(norm_target) & set(norm_col))
                if match_len > best_score:
                    best_score = match_len
                    best_match = col
                    
        return best_match

    def show_sheet_selection_dialog(self, past_file, current_file):
        dialog = tk.Toplevel(self.root)
        dialog.title("シート選択と列設定")
        dialog.geometry("500x400")
        dialog.transient(self.root)
        dialog.grab_set()

        past_sheets = self.get_excel_sheets(past_file)
        current_sheets = self.get_excel_sheets(current_file)

        ttk.Label(dialog, text="過去ファイルのシート:").pack(anchor="w", padx=10, pady=5)
        past_sheet_var = tk.StringVar(value=past_sheets[0] if past_sheets else "")
        ttk.Combobox(dialog, textvariable=past_sheet_var, values=past_sheets, state="readonly", width=30).pack(anchor="w", padx=10)

        ttk.Label(dialog, text="現在ファイルのシート:").pack(anchor="w", padx=10, pady=5)
        current_sheet_var = tk.StringVar(value=current_sheets[0] if current_sheets else "")
        ttk.Combobox(dialog, textvariable=current_sheet_var, values=current_sheets, state="readonly", width=30).pack(anchor="w", padx=10)

        preview_frame = ttk.LabelFrame(dialog, text="列名プレビュー")
        preview_frame.pack(fill="both", expand=True, padx=10, pady=10)
        product_code_var = tk.StringVar(value=self.product_code_column.get())
        amount_var = tk.StringVar(value=self.amount_column.get())

        ttk.Label(preview_frame, text="商品コード列:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        product_code_combo = ttk.Combobox(preview_frame, textvariable=product_code_var, width=30)
        product_code_combo.grid(row=0, column=1, padx=5, pady=5)

        ttk.Label(preview_frame, text="金額列:").grid(row=1, column=0, padx=5, pady=5, sticky="w")
        amount_combo = ttk.Combobox(preview_frame, textvariable=amount_var, width=30)
        amount_combo.grid(row=1, column=1, padx=5, pady=5)

        preview_text = tk.Text(preview_frame, height=8, width=40)
        preview_text.grid(row=3, column=0, columnspan=2, padx=5, pady=5)

        def preview_columns():
            try:
                df = pd.read_excel(past_file, sheet_name=past_sheet_var.get(), nrows=5)
                cols = df.columns.tolist()
                product_code_combo['values'] = cols
                amount_combo['values'] = cols
                
                best_product_match = self.find_best_match_column(df, product_code_var.get())
                best_amount_match = self.find_best_match_column(df, amount_var.get())
                
                if best_product_match:
                    product_code_var.set(best_product_match)
                if best_amount_match:
                    amount_var.set(best_amount_match)
                
                preview_text.delete("1.0", tk.END)
                preview_text.insert(tk.END, "使用可能な列一覧:\n" + "\n".join(f"{i+1}. {c}" for i, c in enumerate(cols)))
                
                if len(df) > 0:
                    preview_text.insert(tk.END, "\n\n最初の行のデータ例:\n")
                    for col in cols[:5]:
                        val = df.iloc[0][col]
                        preview_text.insert(tk.END, f"{col}: {val}\n")
                        
            except Exception as e:
                messagebox.showerror("エラー", f"列プレビュー失敗: {e}")

        ttk.Button(preview_frame, text="列名プレビュー表示", command=preview_columns).grid(row=2, column=0, columnspan=2, pady=5)

        def on_ok():
            self.past_sheet = past_sheet_var.get()
            self.current_sheet = current_sheet_var.get()
            self.product_code_column.set(product_code_var.get())
            self.amount_column.set(amount_var.get())
            dialog.destroy()
            self.perform_analysis()

        ttk.Button(dialog, text="OK", command=on_ok).pack(pady=10)
        self.root.wait_window(dialog)

    def analyze_sales(self):
        if not self.past_file_path.get() or not self.current_file_path.get():
            messagebox.showerror("エラー", "ファイルを選択してください")
            return
        self.show_sheet_selection_dialog(self.past_file_path.get(), self.current_file_path.get())

    def perform_analysis(self):
        try:
            print(f"過去ファイルを読み込み中: {self.past_file_path.get()}, シート: {self.past_sheet}")
            past = pd.read_excel(self.past_file_path.get(), sheet_name=self.past_sheet)
            print(f"過去ファイルの列名: {past.columns.tolist()}")
            
            print(f"現在ファイルを読み込み中: {self.current_file_path.get()}, シート: {self.current_sheet}")
            curr = pd.read_excel(self.current_file_path.get(), sheet_name=self.current_sheet)
            print(f"現在ファイルの列名: {curr.columns.tolist()}")
            
            pc = self.product_code_column.get()
            amt = self.amount_column.get()
            
            print(f"使用する商品コード列: {pc}")
            print(f"使用する金額列: {amt}")
            
            # 列が存在するか確認と自動修正
            if pc not in past.columns:
                best_match = self.find_best_match_column(past, pc)
                if best_match:
                    messagebox.showinfo("列名の自動修正", f"過去ファイルで '{pc}' が見つからないため、最も近い '{best_match}' を使用します")
                    pc_past = best_match
                else:
                    raise ValueError(f"商品コード列 '{pc}' が過去ファイルで見つかりません")
            else:
                pc_past = pc
                
            if pc not in curr.columns:
                best_match = self.find_best_match_column(curr, pc)
                if best_match:
                    messagebox.showinfo("列名の自動修正", f"現在ファイルで '{pc}' が見つからないため、最も近い '{best_match}' を使用します")
                    pc_curr = best_match
                else:
                    raise ValueError(f"商品コード列 '{pc}' が現在ファイルで見つかりません")
            else:
                pc_curr = pc
                
            if amt not in past.columns:
                best_match = self.find_best_match_column(past, amt)
                if best_match:
                    messagebox.showinfo("列名の自動修正", f"過去ファイルで '{amt}' が見つからないため、最も近い '{best_match}' を使用します")
                    amt_past = best_match
                else:
                    raise ValueError(f"金額列 '{amt}' が過去ファイルで見つかりません")
            else:
                amt_past = amt
                
            if amt not in curr.columns:
                best_match = self.find_best_match_column(curr, amt)
                if best_match:
                    messagebox.showinfo("列名の自動修正", f"現在ファイルで '{amt}' が見つからないため、最も近い '{best_match}' を使用します")
                    amt_curr = best_match
                else:
                    raise ValueError(f"金額列 '{amt}' が現在ファイルで見つかりません")
            else:
                amt_curr = amt

            # 金額列のデータ型変換
            for df, col in [(past, amt_past), (curr, amt_curr)]:
                if df[col].dtype == object:
                    df[col] = df[col].astype(str).str.replace(',', '').str.replace(' ', '')
                    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

            # グループ集計
            self.past_summary = past.groupby(pc_past)[amt_past].sum().reset_index()
            self.past_summary.rename(columns={pc_past: pc, amt_past: amt}, inplace=True)
            self.past_summary['構成比'] = self.past_summary[amt] / self.past_summary[amt].sum() * 100
            
            self.current_summary = curr.groupby(pc_curr)[amt_curr].sum().reset_index()
            self.current_summary.rename(columns={pc_curr: pc, amt_curr: amt}, inplace=True)
            self.current_summary['構成比'] = self.current_summary[amt] / self.current_summary[amt].sum() * 100

            self.prepare_merged_data()
            self.display_results()
            self.plot_results()
            messagebox.showinfo("完了", "分析が完了しました")
        except Exception as e:
            import traceback
            traceback.print_exc()
            messagebox.showerror("エラー", f"分析中にエラー: {e}")

    def prepare_merged_data(self):
        pc = self.product_code_column.get()
        amt = self.amount_column.get()
        
        print(f"過去データ: {self.past_summary[pc].dtype}")
        print(f"現在データ: {self.current_summary[pc].dtype}")
        
        past_temp = self.past_summary.copy()
        current_temp = self.current_summary.copy()
        past_temp[pc] = past_temp[pc].astype(str)
        current_temp[pc] = current_temp[pc].astype(str)
        
        m = pd.merge(past_temp, current_temp, on=pc, how='outer', suffixes=('_過去','_現在')).fillna(0)
        m.rename(columns={
            f'{amt}_過去': '過去売上金額',
            f'{amt}_現在': '現在売上金額',
            '構成比_過去': '過去構成比',
            '構成比_現在': '現在構成比'
        }, inplace=True)
        m['増減額'] = m['現在売上金額'] - m['過去売上金額']
        m['増減率'] = m.apply(lambda r: 0 if r['過去売上金額']==0 else r['増減額']/r['過去売上金額']*100, axis=1)
        self.merged_data = m.sort_values('現在売上金額', ascending=False)

    def display_results(self):
        for i in self.tree.get_children():
            self.tree.delete(i)
        pc = self.product_code_column.get()
        for _, r in self.merged_data.iterrows():
            self.tree.insert('', 'end', values=(
                r[pc],
                f"{int(r['過去売上金額']):,}",
                f"{r['過去構成比']:.2f}%",
                f"{int(r['現在売上金額']):,}",
                f"{r['現在構成比']:.2f}%",
                f"{int(r['増減額']):,}",
                f"{r['増減率']:.2f}%"
            ))

    def plot_results(self):
        """グラフタイプに応じてグラフを描画"""
        for w in self.graph_frame.winfo_children():
            w.destroy()

        if self.merged_data is None:
            return

        graph_type = self.graph_type.get()
        
        if graph_type == "総合":
            self.plot_comprehensive()
        elif graph_type == "比較棒グラフ":
            self.plot_comparison_bar()
        elif graph_type == "パイチャート":
            self.plot_pie_charts()
        elif graph_type == "増減分析":
            self.plot_change_analysis()
        elif graph_type == "トレンド分析":
            self.plot_trend_analysis()
        elif graph_type == "散布図":
            self.plot_scatter()

    def plot_comprehensive(self):
        """総合ダッシュボード"""
        top = self.merged_data.head(self.show_top_n.get())
        colors = self.get_color_palette()
        
        fig = plt.Figure(figsize=(16, 12))
        
        # 2x2のサブプロット
        ax1 = fig.add_subplot(221)
        nonzero_past = top[top['過去構成比'] > 0]
        if len(nonzero_past) > 0:
            wedges, texts, autotexts = ax1.pie(nonzero_past['過去構成比'], 
                                             labels=nonzero_past[self.product_code_column.get()], 
                                             autopct='%1.1f%%', startangle=90, colors=colors)
            # フォントサイズ調整
            for autotext in autotexts:
                autotext.set_color('white')
                autotext.set_fontweight('bold')
        ax1.set_title('過去売上構成比（上位商品）', fontsize=14, fontweight='bold')

        ax2 = fig.add_subplot(222)
        nonzero_current = top[top['現在構成比'] > 0]
        if len(nonzero_current) > 0:
            wedges, texts, autotexts = ax2.pie(nonzero_current['現在構成比'], 
                                             labels=nonzero_current[self.product_code_column.get()], 
                                             autopct='%1.1f%%', startangle=90, colors=colors)
            for autotext in autotexts:
                autotext.set_color('white')
                autotext.set_fontweight('bold')
        ax2.set_title('現在売上構成比（上位商品）', fontsize=14, fontweight='bold')

        # 比較棒グラフ
        ax3 = fig.add_subplot(223)
        x = np.arange(len(top))
        width = 0.35
        bars1 = ax3.bar(x - width/2, top['過去売上金額'], width, label='過去', 
                       color=colors[0], alpha=0.8)
        bars2 = ax3.bar(x + width/2, top['現在売上金額'], width, label='現在', 
                       color=colors[1], alpha=0.8)
        
        ax3.set_xlabel('商品コード', fontweight='bold')
        ax3.set_ylabel('売上金額', fontweight='bold')
        ax3.set_title('過去と現在の売上比較', fontsize=14, fontweight='bold')
        ax3.set_xticks(x)
        ax3.set_xticklabels(top[self.product_code_column.get()], rotation=45, ha='right')
        ax3.legend()
        ax3.grid(True, alpha=0.3)
        ax3.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: f"{int(x):,}"))
        
        # 値をバーの上に表示
        for bar in bars1:
            height = bar.get_height()
            ax3.text(bar.get_x() + bar.get_width()/2., height,
                    f'{int(height):,}', ha='center', va='bottom', fontsize=8)
        for bar in bars2:
            height = bar.get_height()
            ax3.text(bar.get_x() + bar.get_width()/2., height,
                    f'{int(height):,}', ha='center', va='bottom', fontsize=8)

        # 増減率分析
        ax4 = fig.add_subplot(224)
        change_data = top[top['過去売上金額'] > 0]  # 過去売上がある商品のみ
        if len(change_data) > 0:
            positive_changes = change_data[change_data['増減率'] >= 0]
            negative_changes = change_data[change_data['増減率'] < 0]
            
            if len(positive_changes) > 0:
                ax4.barh(positive_changes[self.product_code_column.get()], 
                        positive_changes['増減率'], color='green', alpha=0.7, label='増加')
            if len(negative_changes) > 0:
                ax4.barh(negative_changes[self.product_code_column.get()], 
                        negative_changes['増減率'], color='red', alpha=0.7, label='減少')
        
        ax4.set_xlabel('増減率（%）', fontweight='bold')
        ax4.set_title('売上増減率分析', fontsize=14, fontweight='bold')
        ax4.legend()
        ax4.grid(True, alpha=0.3)
        ax4.axvline(x=0, color='black', linestyle='-', alpha=0.3)
        
        fig.tight_layout()
        self.display_graph(fig)

    def plot_comparison_bar(self):
        """詳細比較棒グラフ"""
        top = self.merged_data.head(self.show_top_n.get())
        colors = self.get_color_palette()
        
        fig = plt.Figure(figsize=(14, 8))
        ax = fig.add_subplot(111)
        
        x = np.arange(len(top))
        width = 0.35
        
        bars1 = ax.bar(x - width/2, top['過去売上金額'], width, 
                      label='過去売上', color=colors[0], alpha=0.8)
        bars2 = ax.bar(x + width/2, top['現在売上金額'], width, 
                      label='現在売上', color=colors[1], alpha=0.8)
        
        # データラベル追加
        for i, (bar1, bar2) in enumerate(zip(bars1, bars2)):
            height1 = bar1.get_height()
            height2 = bar2.get_height()
            
            ax.text(bar1.get_x() + bar1.get_width()/2., height1,
                   f'{int(height1):,}', ha='center', va='bottom', fontsize=9, fontweight='bold')
            ax.text(bar2.get_x() + bar2.get_width()/2., height2,
                   f'{int(height2):,}', ha='center', va='bottom', fontsize=9, fontweight='bold')
            
            # 増減率を表示
            change_rate = top.iloc[i]['増減率']
            change_color = 'green' if change_rate >= 0 else 'red'
            ax.text(x[i], max(height1, height2) * 1.1,
                   f'{change_rate:+.1f}%', ha='center', va='bottom', 
                   color=change_color, fontweight='bold', fontsize=10)
        
        ax.set_xlabel('商品コード', fontsize=12, fontweight='bold')
        ax.set_ylabel('売上金額', fontsize=12, fontweight='bold')
        ax.set_title('商品別売上比較（詳細版）', fontsize=16, fontweight='bold')
        ax.set_xticks(x)
        ax.set_xticklabels(top[self.product_code_column.get()], rotation=45, ha='right')
        ax.legend(fontsize=12)
        ax.grid(True, alpha=0.3)
        ax.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: f"{int(x):,}"))
        
        fig.tight_layout()
        self.display_graph(fig)

    def plot_pie_charts(self):
        """改良版パイチャート"""
        top = self.merged_data.head(self.show_top_n.get())
        colors = self.get_color_palette()
        
        fig = plt.Figure(figsize=(16, 8))
        
        # 過去売上パイチャート
        ax1 = fig.add_subplot(121)
        nonzero_past = top[top['過去構成比'] > 0]
        if len(nonzero_past) > 0:
            wedges, texts, autotexts = ax1.pie(
                nonzero_past['過去構成比'], 
                labels=nonzero_past[self.product_code_column.get()],
                autopct=lambda pct: f'{pct:.1f}%\n({int(nonzero_past.iloc[int(pct/100*len(nonzero_past))]["過去売上金額"]):,})',
                startangle=90, colors=colors, textprops={'fontsize': 9}
            )
            
            # 見た目を改善
            for autotext in autotexts:
                autotext.set_color('white')
                autotext.set_fontweight('bold')
                autotext.set_bbox(dict(boxstyle="round,pad=0.3", facecolor='black', alpha=0.5))
        
        ax1.set_title('過去売上構成比', fontsize=14, fontweight='bold')
        
        # 現在売上パイチャート
        ax2 = fig.add_subplot(122)
        nonzero_current = top[top['現在構成比'] > 0]
        if len(nonzero_current) > 0:
            wedges, texts, autotexts = ax2.pie(
                nonzero_current['現在構成比'], 
                labels=nonzero_current[self.product_code_column.get()],
                autopct=lambda pct: f'{pct:.1f}%\n({int(nonzero_current.iloc[int(pct/100*len(nonzero_current))]["現在売上金額"]):,})',
                startangle=90, colors=colors, textprops={'fontsize': 9}
            )
            
            for autotext in autotexts:
                autotext.set_color('white')
                autotext.set_fontweight('bold')
                autotext.set_bbox(dict(boxstyle="round,pad=0.3", facecolor='black', alpha=0.5))
        
        ax2.set_title('現在売上構成比', fontsize=14, fontweight='bold')
        
        fig.tight_layout()
        self.display_graph(fig)

    def plot_change_analysis(self):
        """増減分析グラフ"""
        top = self.merged_data.head(self.show_top_n.get())
        
        fig = plt.Figure(figsize=(14, 10))
        
        # 増減額分析
        ax1 = fig.add_subplot(211)
        change_amounts = top['増減額']
        product_names = top[self.product_code_column.get()]
        
        colors = ['green' if x >= 0 else 'red' for x in change_amounts]
        bars = ax1.barh(product_names, change_amounts, color=colors, alpha=0.7)
        
        # データラベル
        for i, (bar, amount) in enumerate(zip(bars, change_amounts)):
            width = bar.get_width()
            ax1.text(width, bar.get_y() + bar.get_height()/2,
                    f'{int(amount):,}', ha='left' if width >= 0 else 'right', 
                    va='center', fontweight='bold')
        
        ax1.set_xlabel('増減額', fontsize=12, fontweight='bold')
        ax1.set_title('商品別売上増減額', fontsize=14, fontweight='bold')
        ax1.grid(True, alpha=0.3)
        ax1.axvline(x=0, color='black', linestyle='-', alpha=0.5)
        ax1.get_xaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: f"{int(x):,}"))
        
        # 増減率分析
        ax2 = fig.add_subplot(212)
        valid_data = top[top['過去売上金額'] > 0]  # 過去売上がある商品のみ
        
        if len(valid_data) > 0:
            change_rates = valid_data['増減率']
            product_names_valid = valid_data[self.product_code_column.get()]
            
            colors = ['green' if x >= 0 else 'red' for x in change_rates]
            bars = ax2.barh(product_names_valid, change_rates, color=colors, alpha=0.7)
            
            # データラベル
            for bar, rate in zip(bars, change_rates):
                width = bar.get_width()
                ax2.text(width, bar.get_y() + bar.get_height()/2,
                        f'{rate:+.1f}%', ha='left' if width >= 0 else 'right', 
                        va='center', fontweight='bold')
        
        ax2.set_xlabel('増減率（%）', fontsize=12, fontweight='bold')
        ax2.set_title('商品別売上増減率', fontsize=14, fontweight='bold')
        ax2.grid(True, alpha=0.3)
        ax2.axvline(x=0, color='black', linestyle='-', alpha=0.5)
        
        fig.tight_layout()
        self.display_graph(fig)

    def plot_trend_analysis(self):
        """トレンド分析（過去→現在の変化を線グラフで）"""
        top = self.merged_data.head(self.show_top_n.get())
        
        fig = plt.Figure(figsize=(14, 8))
        ax = fig.add_subplot(111)
        
        x_positions = np.arange(len(top))
        
        # 過去と現在の点をプロット
        ax.scatter(x_positions, top['過去売上金額'], color='blue', s=100, 
                  label='過去売上', alpha=0.7, zorder=3)
        ax.scatter(x_positions, top['現在売上金額'], color='red', s=100, 
                  label='現在売上', alpha=0.7, zorder=3)
        
        # 各商品の変化を線で結ぶ
        for i, (past, current) in enumerate(zip(top['過去売上金額'], top['現在売上金額'])):
            color = 'green' if current >= past else 'red'
            alpha = 0.6
            linewidth = 2
            ax.plot([i, i], [past, current], color=color, alpha=alpha, 
                   linewidth=linewidth, zorder=2)
            
            # 矢印を追加
            if abs(current - past) > 0:
                ax.annotate('', xy=(i, current), xytext=(i, past),
                           arrowprops=dict(arrowstyle='->', color=color, lw=2))
        
        # 商品名を表示
        ax.set_xticks(x_positions)
        ax.set_xticklabels(top[self.product_code_column.get()], rotation=45, ha='right')
        
        ax.set_ylabel('売上金額', fontsize=12, fontweight='bold')
        ax.set_title('売上トレンド分析（過去→現在）', fontsize=16, fontweight='bold')
        ax.legend(fontsize=12)
        ax.grid(True, alpha=0.3)
        ax.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: f"{int(x):,}"))
        
        # 値をラベルとして表示
        for i, (past, current, code) in enumerate(zip(top['過去売上金額'], 
                                                     top['現在売上金額'], 
                                                     top[self.product_code_column.get()])):
            ax.text(i, past, f'{int(past):,}', ha='center', va='bottom', 
                   fontsize=8, color='blue', fontweight='bold')
            ax.text(i, current, f'{int(current):,}', ha='center', va='bottom', 
                   fontsize=8, color='red', fontweight='bold')
        
        fig.tight_layout()
        self.display_graph(fig)

    def plot_scatter(self):
        """散布図（過去売上 vs 現在売上）"""
        fig = plt.Figure(figsize=(12, 10))
        
        # メイン散布図
        ax1 = fig.add_subplot(221)
        
        scatter = ax1.scatter(self.merged_data['過去売上金額'], 
                             self.merged_data['現在売上金額'],
                             c=self.merged_data['増減率'], 
                             cmap='RdYlGn', s=60, alpha=0.7)
        
        # 対角線（変化なしライン）を追加
        max_val = max(self.merged_data[['過去売上金額', '現在売上金額']].max())
        ax1.plot([0, max_val], [0, max_val], 'k--', alpha=0.5, label='変化なし')
        
        ax1.set_xlabel('過去売上金額', fontweight='bold')
        ax1.set_ylabel('現在売上金額', fontweight='bold')
        ax1.set_title('過去売上 vs 現在売上', fontsize=14, fontweight='bold')
        ax1.grid(True, alpha=0.3)
        ax1.legend()
        
        # カラーバー追加
        cbar = fig.colorbar(scatter, ax=ax1)
        cbar.set_label('増減率（%）', fontweight='bold')
        
        # 上位商品の詳細散布図
        ax2 = fig.add_subplot(222)
        top_data = self.merged_data.head(self.show_top_n.get())
        
        scatter2 = ax2.scatter(top_data['過去売上金額'], 
                              top_data['現在売上金額'],
                              c=top_data['増減率'], 
                              cmap='RdYlGn', s=100, alpha=0.8)
        
        # 商品名をラベルとして追加
        for i, code in enumerate(top_data[self.product_code_column.get()]):
            ax2.annotate(code, 
                        (top_data.iloc[i]['過去売上金額'], top_data.iloc[i]['現在売上金額']),
                        xytext=(5, 5), textcoords='offset points', fontsize=8)
        
        max_val_top = max(top_data[['過去売上金額', '現在売上金額']].max())
        ax2.plot([0, max_val_top], [0, max_val_top], 'k--', alpha=0.5)
        
        ax2.set_xlabel('過去売上金額', fontweight='bold')
        ax2.set_ylabel('現在売上金額', fontweight='bold')
        ax2.set_title(f'上位{self.show_top_n.get()}商品の詳細', fontsize=14, fontweight='bold')
        ax2.grid(True, alpha=0.3)
        
        # 構成比散布図
        ax3 = fig.add_subplot(223)
        ax3.scatter(self.merged_data['過去構成比'], 
                   self.merged_data['現在構成比'],
                   alpha=0.6, c='blue')
        
        max_ratio = max(self.merged_data[['過去構成比', '現在構成比']].max())
        ax3.plot([0, max_ratio], [0, max_ratio], 'k--', alpha=0.5)
        
        ax3.set_xlabel('過去構成比（%）', fontweight='bold')
        ax3.set_ylabel('現在構成比（%）', fontweight='bold')
        ax3.set_title('構成比の変化', fontsize=14, fontweight='bold')
        ax3.grid(True, alpha=0.3)
        
        # 統計情報
        ax4 = fig.add_subplot(224)
        ax4.axis('off')
        
        # 統計テキスト
        stats_text = f"""
        【分析統計】
        
        総商品数: {len(self.merged_data)}
        
        過去売上合計: {int(self.merged_data['過去売上金額'].sum()):,}
        現在売上合計: {int(self.merged_data['現在売上金額'].sum()):,}
        
        総増減額: {int(self.merged_data['増減額'].sum()):,}
        総増減率: {(self.merged_data['増減額'].sum() / self.merged_data['過去売上金額'].sum() * 100):.2f}%
        
        売上増加商品数: {len(self.merged_data[self.merged_data['増減額'] > 0])}
        売上減少商品数: {len(self.merged_data[self.merged_data['増減額'] < 0])}
        売上変化なし: {len(self.merged_data[self.merged_data['増減額'] == 0])}
        
        最大増加商品: {self.merged_data.loc[self.merged_data['増減額'].idxmax(), self.product_code_column.get()]}
        最大減少商品: {self.merged_data.loc[self.merged_data['増減額'].idxmin(), self.product_code_column.get()]}
        """
        
        ax4.text(0.1, 0.9, stats_text, transform=ax4.transAxes, fontsize=10,
                verticalalignment='top', fontfamily='monospace',
                bbox=dict(boxstyle="round,pad=0.5", facecolor='lightgray', alpha=0.8))
        
        fig.tight_layout()
        self.display_graph(fig)

    def display_graph(self, fig):
        """グラフを表示"""
        canvas = FigureCanvasTkAgg(fig, master=self.graph_frame)
        canvas.draw()
        canvas.get_tk_widget().pack(fill="both", expand=True)
        toolbar = NavigationToolbar2Tk(canvas, self.graph_frame)
        toolbar.update()
        
        # 現在のfigを保存（保存機能用）
        self.current_fig = fig

    def update_graph(self):
        """グラフ設定変更時の更新"""
        if self.merged_data is not None:
            self.plot_results()

    def save_graph(self):
        """グラフを画像として保存"""
        if hasattr(self, 'current_fig'):
            file_path = filedialog.asksaveasfilename(
                title="グラフを保存",
                defaultextension=".png",
                filetypes=[("PNG files", "*.png"), ("PDF files", "*.pdf"), 
                          ("SVG files", "*.svg"), ("All files", "*.*")]
            )
            if file_path:
                self.current_fig.savefig(file_path, dpi=300, bbox_inches='tight')
                messagebox.showinfo("保存完了", f"グラフを保存しました: {file_path}")
        else:
            messagebox.showwarning("警告", "保存するグラフがありません。先に分析を実行してください。")

    def export_to_excel(self):
        if self.merged_data is None:
            messagebox.showerror("エラー", "先に分析を実行してください")
            return

        path = self.output_file_path.get() or filedialog.asksaveasfilename(
            title="保存先を選択", defaultextension=".xlsx", filetypes=[("Excel files","*.xlsx"),("All","*.*")])
        if not path:
            return
        self.output_file_path.set(path)

        wb = openpyxl.Workbook()
        ws = wb.active
        ws.title = "売上分析結果"

        # タイトル・日付
        ws.merge_cells('A1:G1')
        ws['A1'] = "商品別売上分析"
        ws['A1'].font = Font(size=14, bold=True)
        ws.merge_cells('A2:C2')
        ws['A2'] = f"作成日: {pd.Timestamp.now().strftime('%Y-%m-%d')}"

        headers = ["商品コード","過去売上金額","過去構成比","現在売上金額","現在構成比","増減額","増減率"]
        for i, h in enumerate(headers, start=1):
            c = ws.cell(row=4, column=i, value=h)
            c.font = Font(bold=True)
            c.alignment = Alignment(horizontal='center')
            c.fill = PatternFill(start_color="DDDDDD", end_color="DDDDDD", fill_type="solid")

        # データ
        for idx, (_, r) in enumerate(self.merged_data.iterrows(), start=5):
            ws.cell(row=idx, column=1, value=r[self.product_code_column.get()])
            ws.cell(row=idx, column=2, value=r['過去売上金額'])
            ws.cell(row=idx, column=3, value=r['過去構成比']/100)
            ws.cell(row=idx, column=4, value=r['現在売上金額'])
            ws.cell(row=idx, column=5, value=r['現在構成比']/100)
            ws.cell(row=idx, column=6, value=r['増減額'])
            ws.cell(row=idx, column=7, value=r['増減率']/100)
            if r['増減額'] < 0:
                ws.cell(row=idx, column=6).font = Font(color="FF0000")
                ws.cell(row=idx, column=7).font = Font(color="FF0000")

        # 書式・列幅
        for row in range(5, 5 + len(self.merged_data)):
            ws.cell(row=row, column=2).number_format = '#,##0'
            ws.cell(row=row, column=3).number_format = '0.00%'
            ws.cell(row=row, column=4).number_format = '#,##0'
            ws.cell(row=row, column=5).number_format = '0.00%'
            ws.cell(row=row, column=6).number_format = '#,##0'
            ws.cell(row=row, column=7).number_format = '0.00%'
        for col in range(1, 8):
            ws.column_dimensions[get_column_letter(col)].width = 20 if col==1 else 15

        # 合計行
        total_row = 5 + len(self.merged_data)
        ws.cell(row=total_row, column=1, value="合計").font = Font(bold=True)
        for col_idx, key in [(2, '過去売上金額'), (4, '現在売上金額'), (6, '増減額')]:
            val = self.merged_data[key].sum()
            ws.cell(row=total_row, column=col_idx, value=val).font = Font(bold=True)
            ws.cell(row=total_row, column=col_idx).number_format = '#,##0'

        # グラフシート追加
        chart_sheet = wb.create_sheet(title="グラフ")
        chart_sheet['A1'] = "※Excelでは自動グラフ作成はできません。アプリ上でグラフを確認してください。"
        chart_sheet['A2'] = "グラフ画像を保存したい場合は、アプリの「グラフ保存」ボタンをご利用ください。"
        
        wb.save(path)
        messagebox.showinfo("完了", f"Excelファイルを保存しました: {path}")

    def clear_results(self):
        for i in self.tree.get_children():
            self.tree.delete(i)
        for w in self.graph_frame.winfo_children():
            w.destroy()
        self.past_summary = self.current_summary = self.merged_data = None
        if hasattr(self, 'current_fig'):
            delattr(self, 'current_fig')
        messagebox.showinfo("クリア", "分析結果をクリアしました")

def main():
    root = tk.Tk()
    app = SalesAnalysisApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()

過去ファイルを読み込み中: C:/Users/kkt040/OneDrive - トルク株式会社/デスクトップ/85期上期評価/【坂口】８４期 上期 ｻｶｸﾞﾁ 群馬 栃木 売上照会.xlsx, シート: ８４期 上期 ｻｶｸﾞﾁ 群馬 栃木 売上照会.
過去ファイルの列名: ['取引先', '金額', '売上数', '原価', '手配区分', '単価', '受注者', '配送便名', '取引先名', '納入先名', '旧ｺｰﾄﾞ連結', '商品名', '売上№', 'サイズ', '仕入先名', '直送仕入先名', '中分類１', '売上日', '粗利率', '仕入先コード', '直送仕入先', '来勘区分名', '商品', '取引区分', '売上行№', '来勘区分', '受注区分', '摘要名', '取引形態', '客先発注№', '納入先', '事業部', '売上数(バラ)', '事業部名', '部', '部名', 'グループ', 'グループ名', '担当者', '担当者名', '倉庫', '倉庫名', '受注№', '受注行№', '受注日', '受注数', '売上数(ケース)', 'ケース入数', '売上数（小箱）', '小箱入数', '単位', '納品書メモ', '仮単価区分', '赤黒', '税抜金額', '消費税', '税区分', '摘要', '配送便', '前回訂正', '一連№', '受注者.1', 'ﾋﾟｯｷﾝｸﾞﾒﾓ', '住所１', '住所２', '住所３']
現在ファイルを読み込み中: C:/Users/kkt040/OneDrive - トルク株式会社/デスクトップ/85期上期評価/【坂口】85期 坂口 売上照会.xlsx, シート: 85期 坂口 売上照会.
現在ファイルの列名: ['取引先', '金額', '売上数', '原価', '手配区分', '単価', '受注者', '配送便名', '取引先名', '納入先名', '旧ｺｰﾄﾞ連結', '商品名', '売上№', 'サイズ', '仕入先名', '直送仕入先名', '中分類１', '売上日', '粗利率', '仕入先コード', '直送仕入先', '来勘区分名', '商品', '取引区分', '売上行№', '来勘区分', '受注区分', '摘要名', '取引形態', '客先発注№

In [8]:
#次回それぞれのグラフを作成する。（2025、５，１１）
import pandas as pd 
import tkinter as tk
from tkinter import filedialog, ttk, messagebox
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
import openpyxl
from openpyxl.styles import Font, Alignment, PatternFill
from openpyxl.utils import get_column_letter
import re

class SalesAnalysisApp:
    def __init__(self, root):
        self.root = root
        self.root.title("売上照会分析アプリ")
        self.root.geometry("1000x700")
        
        # ファイルパス用変数
        self.past_file_path = tk.StringVar()
        self.current_file_path = tk.StringVar()
        self.output_file_path = tk.StringVar()
        
        # 列名設定用変数（デフォルト値）
        self.product_code_column = tk.StringVar(value="旧ｺｰﾄﾞ連結")  # 半角カタカナに修正
        self.amount_column = tk.StringVar(value="金額")
        
        # データ保存用
        self.past_summary = None
        self.current_summary = None
        self.merged_data = None
        
        self.create_widgets()
    
    def create_widgets(self):
        main_frame = ttk.Frame(self.root)
        main_frame.pack(fill="both", expand=True, padx=10, pady=10)
        
        # ファイル選択
        input_frame = ttk.LabelFrame(main_frame, text="ファイル選択")
        input_frame.pack(fill="x", pady=5)
        ttk.Label(input_frame, text="過去の売上照会ファイル:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        ttk.Entry(input_frame, textvariable=self.past_file_path, width=50).grid(row=0, column=1, padx=5, pady=5)
        ttk.Button(input_frame, text="参照", command=lambda: self.browse_file(self.past_file_path)).grid(row=0, column=2, padx=5)
        
        ttk.Label(input_frame, text="現在の売上照会ファイル:").grid(row=1, column=0, padx=5, pady=5, sticky="w")
        ttk.Entry(input_frame, textvariable=self.current_file_path, width=50).grid(row=1, column=1, padx=5, pady=5)
        ttk.Button(input_frame, text="参照", command=lambda: self.browse_file(self.current_file_path)).grid(row=1, column=2, padx=5)
        
        ttk.Label(input_frame, text="分析結果の出力先:").grid(row=2, column=0, padx=5, pady=5, sticky="w")
        ttk.Entry(input_frame, textvariable=self.output_file_path, width=50).grid(row=2, column=1, padx=5, pady=5)
        ttk.Button(input_frame, text="参照", command=self.browse_output_file).grid(row=2, column=2, padx=5)
        
        # 列名設定
        column_frame = ttk.LabelFrame(main_frame, text="列名設定")
        column_frame.pack(fill="x", pady=5)
        ttk.Label(column_frame, text="商品コード列名:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        ttk.Entry(column_frame, textvariable=self.product_code_column, width=20).grid(row=0, column=1, padx=5, pady=5)
        ttk.Label(column_frame, text="金額列名:").grid(row=0, column=2, padx=5, pady=5, sticky="w")
        ttk.Entry(column_frame, textvariable=self.amount_column, width=20).grid(row=0, column=3, padx=5, pady=5)
        ttk.Label(column_frame, text="※ファイル選択後に「分析実行」を押すとシート選択が可能").grid(row=1, column=0, columnspan=4, padx=5, pady=5)
        
        # 実行ボタン
        buttons_frame = ttk.Frame(main_frame)
        buttons_frame.pack(fill="x", pady=5)
        ttk.Button(buttons_frame, text="分析実行", command=self.analyze_sales, width=20).pack(side="left", padx=5)
        ttk.Button(buttons_frame, text="Excelに出力", command=self.export_to_excel, width=20).pack(side="left", padx=5)
        ttk.Button(buttons_frame, text="クリア", command=self.clear_results, width=20).pack(side="left", padx=5)
        
        # タブ
        tab_control = ttk.Notebook(main_frame)
        table_tab = ttk.Frame(tab_control)
        graph_tab = ttk.Frame(tab_control)
        tab_control.add(table_tab, text='テーブル')
        tab_control.add(graph_tab, text='グラフ')
        tab_control.pack(expand=1, fill="both")
        
        # テーブル表示
        cols = ("商品コード", "過去売上金額", "過去構成比", "現在売上金額", "現在構成比", "増減額", "増減率")
        self.tree = ttk.Treeview(table_tab, columns=cols, show="headings")
        for c in cols:
            self.tree.heading(c, text=c)
            self.tree.column(c, width=150 if c=="商品コード" else 120, anchor="center")
        ysb = ttk.Scrollbar(table_tab, orient="vertical", command=self.tree.yview)
        xsb = ttk.Scrollbar(table_tab, orient="horizontal", command=self.tree.xview)
        self.tree.configure(yscrollcommand=ysb.set, xscrollcommand=xsb.set)
        self.tree.pack(side="left", fill="both", expand=True)
        ysb.pack(side="right", fill="y")
        xsb.pack(side="bottom", fill="x")
        
        # グラフフレーム
        self.graph_frame = graph_tab

    def browse_file(self, var):
        f = filedialog.askopenfilename(title="Excelファイルを選択", filetypes=[("Excel files","*.xlsx *.xls"),("All","*.*")])
        if f:
            var.set(f)

    def browse_output_file(self):
        f = filedialog.asksaveasfilename(title="保存先を選択", defaultextension=".xlsx", filetypes=[("Excel files","*.xlsx"),("All","*.*")])
        if f:
            self.output_file_path.set(f)

    def get_excel_sheets(self, path):
        try:
            return pd.ExcelFile(path).sheet_names
        except Exception as e:
            messagebox.showerror("エラー", f"シート取得失敗: {e}")
            return []
            
    def normalize_column_name(self, name):
        """列名を正規化して比較しやすくする"""
        if not isinstance(name, str):
            return str(name)
        # 全角→半角、大文字→小文字、スペース除去
        name = name.replace('　', ' ').strip()
        return name
        
    def find_best_match_column(self, df, target_name):
        """最も近い列名を見つける"""
        if target_name in df.columns:
            return target_name
            
        norm_target = self.normalize_column_name(target_name)
        
        # 完全一致を試す
        for col in df.columns:
            if self.normalize_column_name(col) == norm_target:
                return col
                
        # 部分一致で最長のものを探す
        best_match = None
        best_score = 0
        
        for col in df.columns:
            norm_col = self.normalize_column_name(col)
            if norm_target in norm_col or norm_col in norm_target:
                # スコアは一致文字の長さ
                match_len = len(set(norm_target) & set(norm_col))
                if match_len > best_score:
                    best_score = match_len
                    best_match = col
                    
        return best_match

    def show_sheet_selection_dialog(self, past_file, current_file):
        dialog = tk.Toplevel(self.root)
        dialog.title("シート選択と列設定")
        dialog.geometry("500x400")
        dialog.transient(self.root)
        dialog.grab_set()

        past_sheets = self.get_excel_sheets(past_file)
        current_sheets = self.get_excel_sheets(current_file)

        ttk.Label(dialog, text="過去ファイルのシート:").pack(anchor="w", padx=10, pady=5)
        past_sheet_var = tk.StringVar(value=past_sheets[0] if past_sheets else "")
        ttk.Combobox(dialog, textvariable=past_sheet_var, values=past_sheets, state="readonly", width=30).pack(anchor="w", padx=10)

        ttk.Label(dialog, text="現在ファイルのシート:").pack(anchor="w", padx=10, pady=5)
        current_sheet_var = tk.StringVar(value=current_sheets[0] if current_sheets else "")
        ttk.Combobox(dialog, textvariable=current_sheet_var, values=current_sheets, state="readonly", width=30).pack(anchor="w", padx=10)

        preview_frame = ttk.LabelFrame(dialog, text="列名プレビュー")
        preview_frame.pack(fill="both", expand=True, padx=10, pady=10)
        product_code_var = tk.StringVar(value=self.product_code_column.get())
        amount_var = tk.StringVar(value=self.amount_column.get())

        ttk.Label(preview_frame, text="商品コード列:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        product_code_combo = ttk.Combobox(preview_frame, textvariable=product_code_var, width=30)
        product_code_combo.grid(row=0, column=1, padx=5, pady=5)

        ttk.Label(preview_frame, text="金額列:").grid(row=1, column=0, padx=5, pady=5, sticky="w")
        amount_combo = ttk.Combobox(preview_frame, textvariable=amount_var, width=30)
        amount_combo.grid(row=1, column=1, padx=5, pady=5)

        preview_text = tk.Text(preview_frame, height=8, width=40)
        preview_text.grid(row=3, column=0, columnspan=2, padx=5, pady=5)

        def preview_columns():
            try:
                df = pd.read_excel(past_file, sheet_name=past_sheet_var.get(), nrows=5)
                cols = df.columns.tolist()
                product_code_combo['values'] = cols
                amount_combo['values'] = cols
                
                # 最適な列の推測
                best_product_match = self.find_best_match_column(df, product_code_var.get())
                best_amount_match = self.find_best_match_column(df, amount_var.get())
                
                if best_product_match:
                    product_code_var.set(best_product_match)
                if best_amount_match:
                    amount_var.set(best_amount_match)
                
                preview_text.delete("1.0", tk.END)
                preview_text.insert(tk.END, "使用可能な列一覧:\n" + "\n".join(f"{i+1}. {c}" for i, c in enumerate(cols)))
                
                # 列の内容プレビュー
                if len(df) > 0:
                    preview_text.insert(tk.END, "\n\n最初の行のデータ例:\n")
                    for col in cols[:5]:  # 最初の5列だけ表示
                        val = df.iloc[0][col]
                        preview_text.insert(tk.END, f"{col}: {val}\n")
                        
            except Exception as e:
                messagebox.showerror("エラー", f"列プレビュー失敗: {e}")

        ttk.Button(preview_frame, text="列名プレビュー表示", command=preview_columns).grid(row=2, column=0, columnspan=2, pady=5)

        def on_ok():
            self.past_sheet = past_sheet_var.get()
            self.current_sheet = current_sheet_var.get()
            self.product_code_column.set(product_code_var.get())
            self.amount_column.set(amount_var.get())
            dialog.destroy()
            self.perform_analysis()

        ttk.Button(dialog, text="OK", command=on_ok).pack(pady=10)
        self.root.wait_window(dialog)

    def analyze_sales(self):
        if not self.past_file_path.get() or not self.current_file_path.get():
            messagebox.showerror("エラー", "ファイルを選択してください")
            return
        self.show_sheet_selection_dialog(self.past_file_path.get(), self.current_file_path.get())

    def perform_analysis(self):
        try:
            # ファイル読み込み時に列名を直接表示してデバッグ
            print(f"過去ファイルを読み込み中: {self.past_file_path.get()}, シート: {self.past_sheet}")
            past = pd.read_excel(self.past_file_path.get(), sheet_name=self.past_sheet)
            print(f"過去ファイルの列名: {past.columns.tolist()}")
            
            print(f"現在ファイルを読み込み中: {self.current_file_path.get()}, シート: {self.current_sheet}")
            curr = pd.read_excel(self.current_file_path.get(), sheet_name=self.current_sheet)
            print(f"現在ファイルの列名: {curr.columns.tolist()}")
            
            pc = self.product_code_column.get()
            amt = self.amount_column.get()
            
            print(f"使用する商品コード列: {pc}")
            print(f"使用する金額列: {amt}")
            
            # 列が存在するか確認
            if pc not in past.columns:
                best_match = self.find_best_match_column(past, pc)
                if best_match:
                    messagebox.showinfo("列名の自動修正", f"過去ファイルで '{pc}' が見つからないため、最も近い '{best_match}' を使用します")
                    pc_past = best_match
                else:
                    raise ValueError(f"商品コード列 '{pc}' が過去ファイルで見つかりません")
            else:
                pc_past = pc
                
            if pc not in curr.columns:
                best_match = self.find_best_match_column(curr, pc)
                if best_match:
                    messagebox.showinfo("列名の自動修正", f"現在ファイルで '{pc}' が見つからないため、最も近い '{best_match}' を使用します")
                    pc_curr = best_match
                else:
                    raise ValueError(f"商品コード列 '{pc}' が現在ファイルで見つかりません")
            else:
                pc_curr = pc
                
            if amt not in past.columns:
                best_match = self.find_best_match_column(past, amt)
                if best_match:
                    messagebox.showinfo("列名の自動修正", f"過去ファイルで '{amt}' が見つからないため、最も近い '{best_match}' を使用します")
                    amt_past = best_match
                else:
                    raise ValueError(f"金額列 '{amt}' が過去ファイルで見つかりません")
            else:
                amt_past = amt
                
            if amt not in curr.columns:
                best_match = self.find_best_match_column(curr, amt)
                if best_match:
                    messagebox.showinfo("列名の自動修正", f"現在ファイルで '{amt}' が見つからないため、最も近い '{best_match}' を使用します")
                    amt_curr = best_match
                else:
                    raise ValueError(f"金額列 '{amt}' が現在ファイルで見つかりません")
            else:
                amt_curr = amt

            # 金額列のデータ型変換
            for df, col in [(past, amt_past), (curr, amt_curr)]:
                if df[col].dtype == object:
                    # カンマやスペースを削除し数値に変換
                    df[col] = df[col].astype(str).str.replace(',', '').str.replace(' ', '')
                    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

            # グループ集計
            self.past_summary = past.groupby(pc_past)[amt_past].sum().reset_index()
            self.past_summary.rename(columns={pc_past: pc, amt_past: amt}, inplace=True)
            self.past_summary['構成比'] = self.past_summary[amt] / self.past_summary[amt].sum() * 100
            
            self.current_summary = curr.groupby(pc_curr)[amt_curr].sum().reset_index()
            self.current_summary.rename(columns={pc_curr: pc, amt_curr: amt}, inplace=True)
            self.current_summary['構成比'] = self.current_summary[amt] / self.current_summary[amt].sum() * 100

            self.prepare_merged_data()
            self.display_results()
            self.plot_results()
            messagebox.showinfo("完了", "分析が完了しました")
        except Exception as e:
            import traceback
            traceback.print_exc()  # コンソールへの詳細エラー出力
            messagebox.showerror("エラー", f"分析中にエラー: {e}")

    def prepare_merged_data(self):
        pc = self.product_code_column.get()
        amt = self.amount_column.get()
        
        # 結合前にデータ型確認
        print(f"過去データ: {self.past_summary[pc].dtype}")
        print(f"現在データ: {self.current_summary[pc].dtype}")
        
        # 全てのデータを文字列に変換して結合
        past_temp = self.past_summary.copy()
        current_temp = self.current_summary.copy()
        past_temp[pc] = past_temp[pc].astype(str)
        current_temp[pc] = current_temp[pc].astype(str)
        
        m = pd.merge(past_temp, current_temp, on=pc, how='outer', suffixes=('_過去','_現在')).fillna(0)
        m.rename(columns={
            f'{amt}_過去': '過去売上金額',
            f'{amt}_現在': '現在売上金額',
            '構成比_過去': '過去構成比',
            '構成比_現在': '現在構成比'
        }, inplace=True)
        m['増減額'] = m['現在売上金額'] - m['過去売上金額']
        m['増減率'] = m.apply(lambda r: 0 if r['過去売上金額']==0 else r['増減額']/r['過去売上金額']*100, axis=1)
        self.merged_data = m.sort_values('現在売上金額', ascending=False)

    def display_results(self):
        for i in self.tree.get_children():
            self.tree.delete(i)
        pc = self.product_code_column.get()
        for _, r in self.merged_data.iterrows():
            self.tree.insert('', 'end', values=(
                r[pc],
                f"{int(r['過去売上金額']):,}",
                f"{r['過去構成比']:.2f}%",
                f"{int(r['現在売上金額']):,}",
                f"{r['現在構成比']:.2f}%",
                f"{int(r['増減額']):,}",
                f"{r['増減率']:.2f}%"
            ))

    def plot_results(self):
        for w in self.graph_frame.winfo_children():
            w.destroy()

        top = self.merged_data.head(10)
        fig = plt.Figure(figsize=(12,8))
        
        # 日本語フォント対応
        plt.rcParams['font.family'] = 'MS Gothic'  # Windowsの場合
        
        ax1 = fig.add_subplot(221)
        # 値が0の場合はパイチャートから除外
        nonzero_past = top[top['過去構成比'] > 0]
        if len(nonzero_past) > 0:
            ax1.pie(nonzero_past['過去構成比'], labels=nonzero_past[self.product_code_column.get()], 
                   autopct='%1.1f%%', startangle=90)
        ax1.set_title('過去売上構成比（上位10商品）')

        ax2 = fig.add_subplot(222)
        nonzero_current = top[top['現在構成比'] > 0]
        if len(nonzero_current) > 0:
            ax2.pie(nonzero_current['現在構成比'], labels=nonzero_current[self.product_code_column.get()], 
                   autopct='%1.1f%%', startangle=90)
        ax2.set_title('現在売上構成比（上位10商品）')

        ax3 = fig.add_subplot(212)
        x = range(len(top))
        w = 0.35
        ax3.bar([i-w/2 for i in x], top['過去売上金額'], w, label='過去')
        ax3.bar([i+w/2 for i in x], top['現在売上金額'], w, label='現在')
        ax3.set_xticks(x)
        ax3.set_xticklabels(top[self.product_code_column.get()], rotation=45, ha='right')
        ax3.set_title('過去と現在の売上比較（上位10商品）')
        ax3.legend()
        
        # Y軸を金額フォーマットに
        ax3.get_yaxis().set_major_formatter(
            plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))
            
        fig.tight_layout()

        canvas = FigureCanvasTkAgg(fig, master=self.graph_frame)
        canvas.draw()
        canvas.get_tk_widget().pack(fill="both", expand=True)
        toolbar = NavigationToolbar2Tk(canvas, self.graph_frame)
        toolbar.update()

    def export_to_excel(self):
        if self.merged_data is None:
            messagebox.showerror("エラー", "先に分析を実行してください")
            return

        path = self.output_file_path.get() or filedialog.asksaveasfilename(
            title="保存先を選択", defaultextension=".xlsx", filetypes=[("Excel files","*.xlsx"),("All","*.*")])
        if not path:
            return
        self.output_file_path.set(path)

        wb = openpyxl.Workbook()
        ws = wb.active
        ws.title = "売上分析結果"

        # タイトル・日付
        ws.merge_cells('A1:G1')
        ws['A1'] = "商品別売上分析"
        ws['A1'].font = Font(size=14, bold=True)
        ws.merge_cells('A2:C2')
        ws['A2'] = f"作成日: {pd.Timestamp.now().strftime('%Y-%m-%d')}"

        headers = ["商品コード","過去売上金額","過去構成比","現在売上金額","現在構成比","増減額","増減率"]
        for i, h in enumerate(headers, start=1):
            c = ws.cell(row=4, column=i, value=h)
            c.font = Font(bold=True)
            c.alignment = Alignment(horizontal='center')
            c.fill = PatternFill(start_color="DDDDDD", end_color="DDDDDD", fill_type="solid")

        # データ
        for idx, (_, r) in enumerate(self.merged_data.iterrows(), start=5):
            ws.cell(row=idx, column=1, value=r[self.product_code_column.get()])
            ws.cell(row=idx, column=2, value=r['過去売上金額'])
            ws.cell(row=idx, column=3, value=r['過去構成比']/100)
            ws.cell(row=idx, column=4, value=r['現在売上金額'])
            ws.cell(row=idx, column=5, value=r['現在構成比']/100)
            ws.cell(row=idx, column=6, value=r['増減額'])
            ws.cell(row=idx, column=7, value=r['増減率']/100)
            if r['増減額'] < 0:
                ws.cell(row=idx, column=6).font = Font(color="FF0000")
                ws.cell(row=idx, column=7).font = Font(color="FF0000")

        # 書式・列幅
        for row in range(5, 5 + len(self.merged_data)):
            ws.cell(row=row, column=2).number_format = '#,##0'
            ws.cell(row=row, column=3).number_format = '0.00%'
            ws.cell(row=row, column=4).number_format = '#,##0'
            ws.cell(row=row, column=5).number_format = '0.00%'
            ws.cell(row=row, column=6).number_format = '#,##0'
            ws.cell(row=row, column=7).number_format = '0.00%'
        for col in range(1, 8):
            ws.column_dimensions[get_column_letter(col)].width = 20 if col==1 else 15

        # 合計行
        total_row = 5 + len(self.merged_data)
        ws.cell(row=total_row, column=1, value="合計").font = Font(bold=True)
        for col_idx, key in [(2, '過去売上金額'), (4, '現在売上金額'), (6, '増減額')]:
            val = self.merged_data[key].sum()
            ws.cell(row=total_row, column=col_idx, value=val).font = Font(bold=True)
            ws.cell(row=total_row, column=col_idx).number_format = '#,##0'

        # グラフシート追加
        chart_sheet = wb.create_sheet(title="グラフ")
        chart_sheet['A1'] = "※Excelでは自動グラフ作成はできません。アプリ上でグラフを確認してください。"
        
        wb.save(path)
        messagebox.showinfo("完了", f"Excelファイルを保存しました: {path}")

    def clear_results(self):
        for i in self.tree.get_children():
            self.tree.delete(i)
        for w in self.graph_frame.winfo_children():
            w.destroy()
        self.past_summary = self.current_summary = self.merged_data = None
        messagebox.showinfo("クリア", "分析結果をクリアしました")

def main():
    root = tk.Tk()
    app = SalesAnalysisApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()

過去ファイルを読み込み中: C:/Users/kkt040/Downloads/2024 05 売上照会.xlsx, シート: 2024 05 売上照会.
過去ファイルの列名: ['取引先', '金額', '売上数', '原価', '手配区分', '単価', '受注者', '配送便名', '取引先名', '納入先名', '旧ｺｰﾄﾞ連結', '商品名', '売上№', 'サイズ', '仕入先名', '直送仕入先名', '中分類１', '売上日', '粗利率', '仕入先コード', '直送仕入先', '来勘区分名', '商品', '取引区分', '売上行№', '来勘区分', '受注区分', '摘要名', '取引形態', '客先発注№', '納入先', '事業部', '売上数(バラ)', '事業部名', '部', '部名', 'グループ', 'グループ名', '担当者', '担当者名', '倉庫', '倉庫名', '受注№', '受注行№', '受注日', '受注数', '売上数(ケース)', 'ケース入数', '売上数（小箱）', '小箱入数', '単位', '納品書メモ', '仮単価区分', '赤黒', '税抜金額', '消費税', '税区分', '摘要', '配送便', '前回訂正', '一連№', '受注者.1', 'ﾋﾟｯｷﾝｸﾞﾒﾓ', '住所１', '住所２', '住所３']
現在ファイルを読み込み中: C:/Users/kkt040/Downloads/売上照会.xlsx, シート: 売上照会.
現在ファイルの列名: ['取引先', '金額', '利益総額', '原価総額', '売上数', '原価', '手配区分', '単価', '受注者', '配送便名', '取引先名', '納入先名', '旧ｺｰﾄﾞ連結', '商品名', '売上№', 'サイズ', '仕入先名', '直送仕入先名', '中分類１', '売上日', '粗利率', '仕入先コード', '直送仕入先', '来勘区分名', '商品', '取引区分', '売上行№', '来勘区分', '受注区分', '摘要名', '取引形態', '客先発注№', '納入先', '事業部', '売上数(バラ)', '事業部名', '部', '部名', 'グループ', 'グループ名', '担当者', '担当者名

In [7]:
import pandas as pd 
import tkinter as tk
from tkinter import filedialog, ttk, messagebox
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
import openpyxl
from openpyxl.styles import Font, Alignment, PatternFill
from openpyxl.utils import get_column_letter

class SalesAnalysisApp:
    def __init__(self, root):
        self.root = root
        self.root.title("売上照会分析アプリ")
        self.root.geometry("1000x700")
        
        # ファイルパス用変数
        self.past_file_path = tk.StringVar()
        self.current_file_path = tk.StringVar()
        self.output_file_path = tk.StringVar()
        
        # 列名設定用変数（デフォルト値）
        self.product_code_column = tk.StringVar(value="旧コード連結")
        self.amount_column = tk.StringVar(value="金額")
        
        # データ保存用
        self.past_summary = None
        self.current_summary = None
        self.merged_data = None
        
        self.create_widgets()
    
    def create_widgets(self):
        main_frame = ttk.Frame(self.root)
        main_frame.pack(fill="both", expand=True, padx=10, pady=10)
        
        # ファイル選択
        input_frame = ttk.LabelFrame(main_frame, text="ファイル選択")
        input_frame.pack(fill="x", pady=5)
        ttk.Label(input_frame, text="過去の売上照会ファイル:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        ttk.Entry(input_frame, textvariable=self.past_file_path, width=50).grid(row=0, column=1, padx=5, pady=5)
        ttk.Button(input_frame, text="参照", command=lambda: self.browse_file(self.past_file_path)).grid(row=0, column=2, padx=5)
        
        ttk.Label(input_frame, text="現在の売上照会ファイル:").grid(row=1, column=0, padx=5, pady=5, sticky="w")
        ttk.Entry(input_frame, textvariable=self.current_file_path, width=50).grid(row=1, column=1, padx=5, pady=5)
        ttk.Button(input_frame, text="参照", command=lambda: self.browse_file(self.current_file_path)).grid(row=1, column=2, padx=5)
        
        ttk.Label(input_frame, text="分析結果の出力先:").grid(row=2, column=0, padx=5, pady=5, sticky="w")
        ttk.Entry(input_frame, textvariable=self.output_file_path, width=50).grid(row=2, column=1, padx=5, pady=5)
        ttk.Button(input_frame, text="参照", command=self.browse_output_file).grid(row=2, column=2, padx=5)
        
        # 列名設定
        column_frame = ttk.LabelFrame(main_frame, text="列名設定")
        column_frame.pack(fill="x", pady=5)
        ttk.Label(column_frame, text="商品コード列名:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        ttk.Entry(column_frame, textvariable=self.product_code_column, width=20).grid(row=0, column=1, padx=5, pady=5)
        ttk.Label(column_frame, text="金額列名:").grid(row=0, column=2, padx=5, pady=5, sticky="w")
        ttk.Entry(column_frame, textvariable=self.amount_column, width=20).grid(row=0, column=3, padx=5, pady=5)
        ttk.Label(column_frame, text="※ファイル選択後に「分析実行」を押すとシート選択が可能").grid(row=1, column=0, columnspan=4, padx=5, pady=5)
        
        # 実行ボタン
        buttons_frame = ttk.Frame(main_frame)
        buttons_frame.pack(fill="x", pady=5)
        ttk.Button(buttons_frame, text="分析実行", command=self.analyze_sales, width=20).pack(side="left", padx=5)
        ttk.Button(buttons_frame, text="Excelに出力", command=self.export_to_excel, width=20).pack(side="left", padx=5)
        ttk.Button(buttons_frame, text="クリア", command=self.clear_results, width=20).pack(side="left", padx=5)
        
        # タブ
        tab_control = ttk.Notebook(main_frame)
        table_tab = ttk.Frame(tab_control)
        graph_tab = ttk.Frame(tab_control)
        tab_control.add(table_tab, text='テーブル')
        tab_control.add(graph_tab, text='グラフ')
        tab_control.pack(expand=1, fill="both")
        
        # テーブル表示
        cols = ("商品コード", "過去売上金額", "過去構成比", "現在売上金額", "現在構成比", "増減額", "増減率")
        self.tree = ttk.Treeview(table_tab, columns=cols, show="headings")
        for c in cols:
            self.tree.heading(c, text=c)
            self.tree.column(c, width=150 if c=="商品コード" else 120, anchor="center")
        ysb = ttk.Scrollbar(table_tab, orient="vertical", command=self.tree.yview)
        xsb = ttk.Scrollbar(table_tab, orient="horizontal", command=self.tree.xview)
        self.tree.configure(yscrollcommand=ysb.set, xscrollcommand=xsb.set)
        self.tree.pack(side="left", fill="both", expand=True)
        ysb.pack(side="right", fill="y")
        xsb.pack(side="bottom", fill="x")
        
        # グラフフレーム
        self.graph_frame = graph_tab

    def browse_file(self, var):
        f = filedialog.askopenfilename(title="Excelファイルを選択", filetypes=[("Excel files","*.xlsx *.xls"),("All","*.*")])
        if f:
            var.set(f)

    def browse_output_file(self):
        f = filedialog.asksaveasfilename(title="保存先を選択", defaultextension=".xlsx", filetypes=[("Excel files","*.xlsx"),("All","*.*")])
        if f:
            self.output_file_path.set(f)

    def get_excel_sheets(self, path):
        try:
            return pd.ExcelFile(path).sheet_names
        except Exception as e:
            messagebox.showerror("エラー", f"シート取得失敗: {e}")
            return []

    def show_sheet_selection_dialog(self, past_file, current_file):
        dialog = tk.Toplevel(self.root)
        dialog.title("シート選択と列設定")
        dialog.geometry("500x400")
        dialog.transient(self.root)
        dialog.grab_set()

        past_sheets = self.get_excel_sheets(past_file)
        current_sheets = self.get_excel_sheets(current_file)

        ttk.Label(dialog, text="過去ファイルのシート:").pack(anchor="w", padx=10, pady=5)
        past_sheet_var = tk.StringVar(value=past_sheets[0] if past_sheets else "")
        ttk.Combobox(dialog, textvariable=past_sheet_var, values=past_sheets, state="readonly", width=30).pack(anchor="w", padx=10)

        ttk.Label(dialog, text="現在ファイルのシート:").pack(anchor="w", padx=10, pady=5)
        current_sheet_var = tk.StringVar(value=current_sheets[0] if current_sheets else "")
        ttk.Combobox(dialog, textvariable=current_sheet_var, values=current_sheets, state="readonly", width=30).pack(anchor="w", padx=10)

        preview_frame = ttk.LabelFrame(dialog, text="列名プレビュー")
        preview_frame.pack(fill="both", expand=True, padx=10, pady=10)
        product_code_var = tk.StringVar(value=self.product_code_column.get())
        amount_var = tk.StringVar(value=self.amount_column.get())

        ttk.Label(preview_frame, text="商品コード列:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        product_code_combo = ttk.Combobox(preview_frame, textvariable=product_code_var, width=30)
        product_code_combo.grid(row=0, column=1, padx=5, pady=5)

        ttk.Label(preview_frame, text="金額列:").grid(row=1, column=0, padx=5, pady=5, sticky="w")
        amount_combo = ttk.Combobox(preview_frame, textvariable=amount_var, width=30)
        amount_combo.grid(row=1, column=1, padx=5, pady=5)

        preview_text = tk.Text(preview_frame, height=8, width=40)
        preview_text.grid(row=3, column=0, columnspan=2, padx=5, pady=5)

        def preview_columns():
            try:
                df = pd.read_excel(past_file, sheet_name=past_sheet_var.get(), nrows=5)
                cols = df.columns.tolist()
                product_code_combo['values'] = cols
                amount_combo['values'] = cols
                preview_text.delete("1.0", tk.END)
                preview_text.insert(tk.END, "使用可能な列一覧:\n" + "\n".join(f"{i+1}. {c}" for i, c in enumerate(cols)))
            except Exception as e:
                messagebox.showerror("エラー", f"列プレビュー失敗: {e}")

        ttk.Button(preview_frame, text="列名プレビュー表示", command=preview_columns).grid(row=2, column=0, columnspan=2, pady=5)

        def on_ok():
            self.past_sheet = past_sheet_var.get()
            self.current_sheet = current_sheet_var.get()
            self.product_code_column.set(product_code_var.get())
            self.amount_column.set(amount_var.get())
            dialog.destroy()
            self.perform_analysis()

        ttk.Button(dialog, text="OK", command=on_ok).pack(pady=10)
        self.root.wait_window(dialog)

    def analyze_sales(self):
        if not self.past_file_path.get() or not self.current_file_path.get():
            messagebox.showerror("エラー", "ファイルを選択してください")
            return
        self.show_sheet_selection_dialog(self.past_file_path.get(), self.current_file_path.get())

    def perform_analysis(self):
        try:
            past = pd.read_excel(self.past_file_path.get(), sheet_name=self.past_sheet)
            curr = pd.read_excel(self.current_file_path.get(), sheet_name=self.current_sheet)
            pc, amt = self.product_code_column.get(), self.amount_column.get()

            for df in (past, curr):
                if amt not in df.columns:
                    raise ValueError(f"金額列 '{amt}' が見つかりません")
                if df[amt].dtype == object:
                    df[amt] = df[amt].str.replace(',', '').astype(float)

            self.past_summary = past.groupby(pc)[amt].sum().reset_index()
            self.past_summary['構成比'] = self.past_summary[amt] / self.past_summary[amt].sum() * 100
            self.current_summary = curr.groupby(pc)[amt].sum().reset_index()
            self.current_summary['構成比'] = self.current_summary[amt] / self.current_summary[amt].sum() * 100

            self.prepare_merged_data()
            self.display_results()
            self.plot_results()
            messagebox.showinfo("完了", "分析が完了しました")
        except Exception as e:
            messagebox.showerror("エラー", f"分析中にエラー: {e}")

    def prepare_merged_data(self):
        pc = self.product_code_column.get()
        amt = self.amount_column.get()
        m = pd.merge(self.past_summary, self.current_summary, on=pc, how='outer', suffixes=('_過去','_現在')).fillna(0)
        m.rename(columns={
            f'{amt}_過去': '過去売上金額',
            f'{amt}_現在': '現在売上金額',
            '構成比_過去': '過去構成比',
            '構成比_現在': '現在構成比'
        }, inplace=True)
        m['増減額'] = m['現在売上金額'] - m['過去売上金額']
        m['増減率'] = m.apply(lambda r: 0 if r['過去売上金額']==0 else r['増減額']/r['過去売上金額']*100, axis=1)
        self.merged_data = m.sort_values('現在売上金額', ascending=False)

    def display_results(self):
        for i in self.tree.get_children():
            self.tree.delete(i)
        pc = self.product_code_column.get()
        for _, r in self.merged_data.iterrows():
            self.tree.insert('', 'end', values=(
                r[pc],
                f"{int(r['過去売上金額']):,}",
                f"{r['過去構成比']:.2f}%",
                f"{int(r['現在売上金額']):,}",
                f"{r['現在構成比']:.2f}%",
                f"{int(r['増減額']):,}",
                f"{r['増減率']:.2f}%"
            ))

    def plot_results(self):
        for w in self.graph_frame.winfo_children():
            w.destroy()

        top = self.merged_data.head(10)
        fig = plt.Figure(figsize=(12,8))
        ax1 = fig.add_subplot(221)
        ax1.pie(top['過去構成比'], labels=top[self.product_code_column.get()], autopct='%1.1f%%', startangle=90)
        ax1.set_title('過去売上構成比（上位10商品）')

        ax2 = fig.add_subplot(222)
        ax2.pie(top['現在構成比'], labels=top[self.product_code_column.get()], autopct='%1.1f%%', startangle=90)
        ax2.set_title('現在売上構成比（上位10商品）')

        ax3 = fig.add_subplot(212)
        x = range(len(top))
        w = 0.35
        ax3.bar([i-w/2 for i in x], top['過去売上金額'], w, label='過去')
        ax3.bar([i+w/2 for i in x], top['現在売上金額'], w, label='現在')
        ax3.set_xticks(x)
        ax3.set_xticklabels(top[self.product_code_column.get()], rotation=45, ha='right')
        ax3.set_title('過去と現在の売上比較（上位10商品）')
        ax3.legend()
        fig.tight_layout()

        canvas = FigureCanvasTkAgg(fig, master=self.graph_frame)
        canvas.draw()
        canvas.get_tk_widget().pack(fill="both", expand=True)
        toolbar = NavigationToolbar2Tk(canvas, self.graph_frame)
        toolbar.update()

    def export_to_excel(self):
        if self.merged_data is None:
            messagebox.showerror("エラー", "先に分析を実行してください")
            return

        path = self.output_file_path.get() or filedialog.asksaveasfilename(
            title="保存先を選択", defaultextension=".xlsx", filetypes=[("Excel files","*.xlsx"),("All","*.*")])
        if not path:
            return
        self.output_file_path.set(path)

        wb = openpyxl.Workbook()
        ws = wb.active
        ws.title = "売上分析結果"

        # タイトル・日付
        ws.merge_cells('A1:G1')
        ws['A1'] = "商品別売上分析"
        ws['A1'].font = Font(size=14, bold=True)
        ws.merge_cells('A2:C2')
        ws['A2'] = f"作成日: {pd.Timestamp.now().strftime('%Y-%m-%d')}"

        headers = ["商品コード","過去売上金額","過去構成比","現在売上金額","現在構成比","増減額","増減率"]
        for i, h in enumerate(headers, start=1):
            c = ws.cell(row=4, column=i, value=h)
            c.font = Font(bold=True)
            c.alignment = Alignment(horizontal='center')
            c.fill = PatternFill(start_color="DDDDDD", end_color="DDDDDD", fill_type="solid")

        # データ
        for idx, (_, r) in enumerate(self.merged_data.iterrows(), start=5):
            ws.cell(row=idx, column=1, value=r[self.product_code_column.get()])
            ws.cell(row=idx, column=2, value=r['過去売上金額'])
            ws.cell(row=idx, column=3, value=r['過去構成比']/100)
            ws.cell(row=idx, column=4, value=r['現在売上金額'])
            ws.cell(row=idx, column=5, value=r['現在構成比']/100)
            ws.cell(row=idx, column=6, value=r['増減額'])
            ws.cell(row=idx, column=7, value=r['増減率']/100)
            if r['増減額'] < 0:
                ws.cell(row=idx, column=6).font = Font(color="FF0000")
                ws.cell(row=idx, column=7).font = Font(color="FF0000")

        # 書式・列幅
        for row in range(5, 5 + len(self.merged_data)):
            ws.cell(row=row, column=2).number_format = '#,##0'
            ws.cell(row=row, column=3).number_format = '0.00%'
            ws.cell(row=row, column=4).number_format = '#,##0'
            ws.cell(row=row, column=5).number_format = '0.00%'
            ws.cell(row=row, column=6).number_format = '#,##0'
            ws.cell(row=row, column=7).number_format = '0.00%'
        for col in range(1, 8):
            ws.column_dimensions[get_column_letter(col)].width = 20 if col==1 else 15

        # 合計行
        total_row = 5 + len(self.merged_data)
        ws.cell(row=total_row, column=1, value="合計").font = Font(bold=True)
        for col_idx, key in [(2, '過去売上金額'), (4, '現在売上金額'), (6, '増減額')]:
            val = self.merged_data[key].sum()
            ws.cell(row=total_row, column=col_idx, value=val).font = Font(bold=True)
            ws.cell(row=total_row, column=col_idx).number_format = '#,##0'

        # グラフ用シート（省略可）

        wb.save(path)
        messagebox.showinfo("完了", f"Excelファイルを保存しました: {path}")

    def clear_results(self):
        for i in self.tree.get_children():
            self.tree.delete(i)
        for w in self.graph_frame.winfo_children():
            w.destroy()
        self.past_summary = self.current_summary = self.merged_data = None
        messagebox.showinfo("クリア", "分析結果をクリアしました")

def main():
    root = tk.Tk()
    app = SalesAnalysisApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()
